In [ ]:
import pandas as pd
import polars as pl
import networkx as nx
import rtsvg
rt = rtsvg.RACETrack()
import os
import kagglehub
path = kagglehub.dataset_download("new-york-city/nyc-citywide-payroll-data")
os.listdir(path)
df = pl.read_csv(path+"/Citywide_Payroll_Data__Fiscal_Year_.csv")
# Remove spaces in first (or last) slot
for _column_ in ['Pay Basis', 'Work Location Borough']:
    df = df.with_columns(pl.col(_column_).str.strip_chars(' ').alias(_column_))
#df = rt.columnsAreTimestamps(df, 'Agency Start Date')
# To Upper Case
for _column_ in ['Agency Name', 'Last Name', 'First Name', 'Mid Init', 'Work Location Borough', 'Title Description']:
    df = df.with_columns(pl.col(_column_).str.to_uppercase().alias(_column_))
# Remove "$" sign... cast to float
for _column_ in ['Base Salary', 'Regular Gross Paid', 'Total OT Paid', 'Total Other Pay']:
    df = df.with_columns(pl.col(_column_).str.strip_chars_start('$').cast(pl.Float64).alias(_column_))
# Per Session Flag
def isPerSession(x):
    if x is None: return False
    return 'session' in x.lower() and 'per' in x.lower()
df = df.with_columns(pl.col('Title Description').str.to_lowercase().str.contains('session').alias('Per Session Pay Flag'))
for x in set(df['Title Description']):
    if isPerSession(x): print(x)
# Crude fixes for Title Description
_fix_ = {}
for x in set(df['Title Description']):
    _orig_ = x
    if x is None: x = "None"
    while x.startswith('*') or x.startswith('?'): x = x[1:]
    x = x.strip()
    _fix_[_orig_] = x
df = df.with_columns(pl.col('Title Description').replace_strict(_fix_).alias('Title Description'))
# Finer fix for Agency Name
# Produces the template below
#_len_ = 40
#for x in sorted(list(set(df['Agency Name']))):
#    _other_ = f'"{x}"'
#    _str_   = f'"{x.strip()}"'
#    print(f'{_other_:{_len_}}:({_str_:{_len_}},{_str_}),')
_fix_ = {
"ADMIN FOR CHILDREN'S SVCS     "        :("ADMIN FOR CHILDREN'S SERVICES"         ,"None",              "None"),
"ADMIN TRIALS AND HEARINGS     "        :("ADMIN TRIALS AND HEARINGS"             ,"None",              "None"),
"BOARD OF CORRECTION           "        :("BOARD OF CORRECTIONS"                  ,"None",              "None"),
"BOARD OF CORRECTIONS          "        :("BOARD OF CORRECTIONS"                  ,"None",              "None"),
"BOARD OF ELECTION             "        :("BOARD OF ELECTIONS"                    ,"None",              "None"),
"BOARD OF ELECTION POLL WORKERS"        :("BOARD OF ELECTIONS"                    ,"POLL WORKERS",      "None"),
"BOROUGH PRESIDENT-BRONX       "        :("PRESIDENT OF BOROUGH OFFICE",           "None"              ,"BRONX"),
"BOROUGH PRESIDENT-BROOKLYN    "        :("PRESIDENT OF BOROUGH OFFICE",           "None"              ,"BROOKLYN"),
"BOROUGH PRESIDENT-QUEENS      "        :("PRESIDENT OF BOROUGH OFFICE",           "None"              ,"QUEENS"),
"BOROUGH PRESIDENT-STATEN IS   "        :("PRESIDENT OF BOROUGH OFFICE",           "None"              ,"STATEN ISLAND"),
"BRONX COMMUNITY BOARD #1      "        :("COMMUNITY BOARD"                       ,"BOARD #1",          "BRONX"),
"BRONX COMMUNITY BOARD #10     "        :("COMMUNITY BOARD"                       ,"BOARD #10",         "BRONX"),
"BRONX COMMUNITY BOARD #11     "        :("COMMUNITY BOARD"                       ,"BOARD #11",         "BRONX"),
"BRONX COMMUNITY BOARD #12     "        :("COMMUNITY BOARD"                       ,"BOARD #12",         "BRONX"),
"BRONX COMMUNITY BOARD #2      "        :("COMMUNITY BOARD"                       ,"BOARD #2",          "BRONX"),
"BRONX COMMUNITY BOARD #3      "        :("COMMUNITY BOARD"                       ,"BOARD #3",          "BRONX"),
"BRONX COMMUNITY BOARD #4      "        :("COMMUNITY BOARD"                       ,"BOARD #4",          "BRONX"),
"BRONX COMMUNITY BOARD #5      "        :("COMMUNITY BOARD"                       ,"BOARD #5",          "BRONX"),
"BRONX COMMUNITY BOARD #6      "        :("COMMUNITY BOARD"                       ,"BOARD #6",          "BRONX"),
"BRONX COMMUNITY BOARD #7      "        :("COMMUNITY BOARD"                       ,"BOARD #7",          "BRONX"),
"BRONX COMMUNITY BOARD #8      "        :("COMMUNITY BOARD"                       ,"BOARD #8",          "BRONX"),
"BRONX COMMUNITY BOARD #9      "        :("COMMUNITY BOARD"                       ,"BOARD #9",          "BRONX"),
"BRONX DA       "                       :("DISTRICT ATTORNEY'S OFFICE",            "None"              ,"BRONX"),
"BRONX DISTRICT ATTORNEY       "        :("DISTRICT ATTORNEY'S OFFICE",            "None"              ,"BRONX"),
"BROOKLYN COMMUNITY BOARD #1   "        :("COMMUNITY BOARD"                       ,"BOARD #1",          "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #10  "        :("COMMUNITY BOARD"                       ,"BOARD #10",         "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #11  "        :("COMMUNITY BOARD"                       ,"BOARD #11",         "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #12  "        :("COMMUNITY BOARD"                       ,"BOARD #12",         "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #13  "        :("COMMUNITY BOARD"                       ,"BOARD #13",         "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #14  "        :("COMMUNITY BOARD"                       ,"BOARD #14",         "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #15  "        :("COMMUNITY BOARD"                       ,"BOARD #15",         "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #16  "        :("COMMUNITY BOARD"                       ,"BOARD #16",         "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #17  "        :("COMMUNITY BOARD"                       ,"BOARD #17",         "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #18  "        :("COMMUNITY BOARD"                       ,"BOARD #18",         "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #2   "        :("COMMUNITY BOARD"                       ,"BOARD #2",          "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #3   "        :("COMMUNITY BOARD"                       ,"BOARD #3",          "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #4   "        :("COMMUNITY BOARD"                       ,"BOARD #4",          "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #5   "        :("COMMUNITY BOARD"                       ,"BOARD #5",          "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #6   "        :("COMMUNITY BOARD"                       ,"BOARD #6",          "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #7   "        :("COMMUNITY BOARD"                       ,"BOARD #7",          "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #8   "        :("COMMUNITY BOARD"                       ,"BOARD #8",          "BROOKLYN"),
"BROOKLYN COMMUNITY BOARD #9   "        :("COMMUNITY BOARD"                       ,"BOARD #9",          "BROOKLYN"),
"BUSINESS INTEGRITY COMMISSION "        :("BUSINESS INTEGRITY COMMISSION"         ,"None",              "None"),
"CAMPAIGN FINANCE BOARD        "        :("CAMPAIGN FINANCE BOARD"                ,"None",              "None"),
"CITY CLERK                    "        :("CITY CLERK"                            ,"None",              "None"),
"CITY COUNCIL                  "        :("CITY COUNCIL"                          ,"None",              "None"),
"CIVIL SERVICE COMMISSION      "        :("CIVIL SERVICE COMMISSION"              ,"None",              "None"),
"CIVILIAN COMPLAINT REVIEW BD  "        :("CIVILIAN COMPLAINT REVIEW BOARD"       ,"None",              "None"),
"COMMUNITY COLLEGE (BRONX)     "        :("COMMUNITY COLLEGE"                     ,"None"              ,"BRONX"),
"COMMUNITY COLLEGE (HOSTOS)    "        :("COMMUNITY COLLEGE"                     ,"None"              ,"HOSTOS"),
"COMMUNITY COLLEGE (KINGSBORO) "        :("COMMUNITY COLLEGE"                     ,"None"              ,"KINGSBORO"),
"COMMUNITY COLLEGE (LAGUARDIA) "        :("COMMUNITY COLLEGE"                     ,"None"              ,"LAGUARDIA"),
"COMMUNITY COLLEGE (MANHATTAN) "        :("COMMUNITY COLLEGE"                     ,"None"              ,"MANHATTAN"),
"COMMUNITY COLLEGE (QUEENSBORO)"        :("COMMUNITY COLLEGE"                     ,"None"              ,"QUEENSBORO"),
"CONFLICTS OF INTEREST BOARD   "        :("CONFLICTS OF INTEREST BOARD"           ,"None",              "None"),
"CONSUMER AFFAIRS              "        :("CONSUMER AFFAIRS"                      ,"None",              "None"),
"CULTURAL AFFAIRS              "        :("CULTURAL AFFAIRS"                      ,"None",              "None"),
"CUNY CENTRAL OFFICE           "        :("CUNY CENTRAL OFFICE"                   ,"None",              "None"),
"DEPARTMENT FOR THE AGING      "        :("DEPARTMENT FOR THE AGING"              ,"None",              "None"),
"DEPARTMENT OF BUILDINGS       "        :("DEPARTMENT OF BUILDINGS"               ,"None",              "None"),
"DEPARTMENT OF BUSINESS SERV.  "        :("DEPARTMENT OF BUSINESS SERVICES"       ,"None",              "None"),
"DEPARTMENT OF CITY PLANNING   "        :("DEPARTMENT OF CITY PLANNING"           ,"None",              "None"),
"DEPARTMENT OF CORRECTION      "        :("DEPARTMENT OF CORRECTION"              ,"None",              "None"),
"DEPARTMENT OF EDUCATION ADMIN "        :("DEPARTMENT OF EDUCATION"               ,"Admin",             "None"),
"DEPARTMENT OF FINANCE         "        :("DEPARTMENT OF FINANCE"                 ,"None",              "None"),
"DEPARTMENT OF INVESTIGATION   "        :("DEPARTMENT OF INVESTIGATION"           ,"None",              "None"),
"DEPARTMENT OF JUVENILE JUSTICE"        :("DEPARTMENT OF JUVENILE JUSTICE"        ,"None",              "None"),
"DEPARTMENT OF PROBATION       "        :("DEPARTMENT OF PROBATION"               ,"None",              "None"),
"DEPARTMENT OF SANITATION      "        :("DEPARTMENT OF SANITATION"              ,"None",              "None"),
"DEPARTMENT OF TRANSPORTATION  "        :("DEPARTMENT OF TRANSPORTATION"          ,"None",              "None"),
"DEPT OF CITYWIDE ADMIN SVCS   "        :("DEPT OF CITYWIDE ADMIN SERVICES"       ,"None",              "None"),
"DEPT OF ED HRLY SUPPORT STAFF "        :("DEPT OF EDUCATION"                     ,"HOURLY SUPPORT STAFF", "None"),
"DEPT OF ED PARA PROFESSIONALS "        :("DEPT OF EDUCATION"                     ,"PARA PROFESSIONALS",   "None"),
"DEPT OF ED PEDAGOGICAL        "        :("DEPT OF EDUCATION"                     ,"PEDAGOGICAL",          "None"),
"DEPT OF ED PER DIEM TEACHERS  "        :("DEPT OF EDUCATION"                     ,"PER DIEM TEACHERS",    "None"),
"DEPT OF ED PER SESSION TEACHER"        :("DEPT OF EDUCATION"                     ,"PER SESSION TEACHER",  "None"),
"DEPT OF ENVIRONMENT PROTECTION"        :("DEPT OF ENVIRONMENT PROTECTION"        ,"None",                 "None"),
"DEPT OF HEALTH/MENTAL HYGIENE "        :("DEPT OF HEALTH/MENTAL HYGIENE"         ,"None",                 "None"),
"DEPT OF INFO TECH & TELECOMM  "        :("DEPT OF INFO TECH & TELECOMM"          ,"None",                 "None"),
"DEPT OF PARKS & RECREATION    "        :("DEPT OF PARKS & RECREATION"            ,"None",                 "None"),
"DEPT OF RECORDS & INFO SERVICE"        :("DEPT OF RECORDS & INFO SERVICE"        ,"None",                 "None"),
"DEPT OF YOUTH & COMM DEV SRVS "        :("DEPT OF YOUTH & COMM DEV SRVS"         ,"None",                 "None"),
"DEPT. OF DESIGN & CONSTRUCTION"        :("DEPT OF DESIGN & CONSTRUCTION"         ,"None",                 "None"),
"DEPT. OF HOMELESS SERVICES    "        :("DEPT OF HOMELESS SERVICES"             ,"None",                 "None"),
"DISTRICT ATTORNEY KINGS COUNTY"        :("DISTRICT ATTORNEY'S OFFICE"            ,"None"                 ,"BROOKLYN"), # apparently, kings county is actually brooklyn
"DISTRICT ATTORNEY QNS COUNTY  "        :("DISTRICT ATTORNEY'S OFFICE"            ,"None"                 ,"QNS COUNTY"),
"DISTRICT ATTORNEY RICHMOND COU"        :("DISTRICT ATTORNEY'S OFFICE"            ,"None"                 ,"RICHMOND COUNTY"),
"DISTRICT ATTORNEY-MANHATTAN   "        :("DISTRICT ATTORNEY'S OFFICE"            ,"None"                 ,"MANHATTAN"),
"DISTRICT ATTORNEY-SPECIAL NARC"        :("DISTRICT ATTORNEY'S OFFICE"            ,"None"                 ,"SPECIAL NARC"),
"DISTRICTING COMMISSION        "        :("DISTRICTING COMMISSION"                ,"None"                 ,"None"),
"DOE CUSTODIAL PAYROL          "        :("DOE CUSTODIAL PAYROL"                  ,"None"                 ,"None"),
"EQUAL EMPLOY PRACTICES COMM   "        :("EQUAL EMPLOY PRACTICES COMMISSION"     ,"None"                 ,"None"),
"FINANCIAL INFO SVCS AGENCY    "        :("FINANCIAL INFO SERVICES AGENCY"        ,"None"                 ,"None"),
"FIRE DEPARTMENT               "        :("FIRE DEPARTMENT"                       ,"None"                 ,"None"),
"GUTTMAN COMMUNITY COLLEGE     "        :("GUTTMAN COMMUNITY COLLEGE"             ,"None"                 ,"None"),
"HOUSING PRESERVATION & DVLPMNT"        :("HOUSING PRESERVATION & DEVELOPMENT"    ,"None"                 ,"None"),
"HRA/DEPT OF SOCIAL SERVICES   "        :("DEPT OF SOCIAL SERVICES"               ,"HRA"                  ,"None"),
"HUMAN RIGHTS COMMISSION       "        :("HUMAN RIGHTS COMMISSION"               ,"None"                 ,"None"),
"HUNTER COLLEGE HIGH SCHOOL    "        :("HUNTER COLLEGE HIGH SCHOOL"            ,"None"                 ,"None"),
"INDEPENDENT BUDGET OFFICE     "        :("INDEPENDENT BUDGET OFFICE"             ,"None"                 ,"None"),
"KINGS DA"                              :("DISTRICT ATTORNEY'S OFFICE"            ,"None"                 ,"BROOKLYN"), # apparently, kings county is actually brooklyn
"LANDMARKS PRESERVATION COMM   "        :("LANDMARKS PRESERVATION COMM"           ,"None"                 ,"None"),
"LAW DEPARTMENT                "        :("LAW DEPARTMENT"                        ,"None"                 ,"None"),
"MANHATTAN COMMUNITY BOARD #1  "        :("COMMUNITY BOARD"                       ,"BOARD #1"             ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #10 "        :("COMMUNITY BOARD"                       ,"BOARD #10"            ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #11 "        :("COMMUNITY BOARD"                       ,"BOARD #11"            ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #12 "        :("COMMUNITY BOARD"                       ,"BOARD #12"            ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #2  "        :("COMMUNITY BOARD"                       ,"BOARD #2"             ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #3  "        :("COMMUNITY BOARD"                       ,"BOARD #3"             ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #4  "        :("COMMUNITY BOARD"                       ,"BOARD #4"             ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #5  "        :("COMMUNITY BOARD"                       ,"BOARD #5"             ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #6  "        :("COMMUNITY BOARD"                       ,"BOARD #6"             ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #7  "        :("COMMUNITY BOARD"                       ,"BOARD #7"             ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #8  "        :("COMMUNITY BOARD"                       ,"BOARD #8"             ,"MANHATTAN"),
"MANHATTAN COMMUNITY BOARD #9  "        :("COMMUNITY BOARD"                       ,"BOARD #9"             ,"MANHATTAN"),
"MANHATTAN DA   "                       :("DISTRICT ATTORNEY'S OFFICE"            ,"None"                 ,"MANHATTAN"),
"MAYORS OFFICE OF CONTRACT SVCS"        :("MAYORS OFFICE OF CONTRACT SERVICES"    ,"None"                 ,"None"),
"MUNICIPAL WATER FIN AUTHORITY "        :("MUNICIPAL WATER FINANCIAL AUTHORITY"   ,"None"                 ,"None"),
"NYC DEPT OF VETERANS' SERVICES"        :("DEPT OF VETERANS' SERVICES"            ,"None"                 ,"None"),
"NYC EMPLOYEES RETIREMENT SYS  "        :("EMPLOYEES RETIREMENT SYSTEM"           ,"None"                 ,"None"),
"NYC HOUSING AUTHORITY         "        :("HOUSING AUTHORITY"                     ,"None"                 ,"None"),
"NYC POLICE PENSION FUND       "        :("POLICE DEPARTMENT}"                    ,"PENSION FUND"         ,"None"),
"OFF OF PAYROLL ADMINISTRATION "        :("OFFICE OF PAYROLL ADMINISTRATION"      ,"None"                 ,"None"),
"OFFICE OF COLLECTIVE BARGAININ"        :("OFFICE OF COLLECTIVE BARGAININ"        ,"None"                 ,"None"),
"OFFICE OF EMERGENCY MANAGEMENT"        :("OFFICE OF EMERGENCY MANAGEMENT"        ,"None"                 ,"None"),
"OFFICE OF LABOR RELATIONS     "        :("OFFICE OF LABOR RELATIONS"             ,"None"                 ,"None"),
"OFFICE OF MANAGEMENT & BUDGET "        :("OFFICE OF MANAGEMENT & BUDGET"         ,"None"                 ,"None"),
"OFFICE OF THE ACTUARY         "        :("OFFICE OF THE ACTUARY"                 ,"None"                 ,"None"),
"OFFICE OF THE COMPTROLLER     "        :("OFFICE OF THE COMPTROLLER"             ,"None"                 ,"None"),
"OFFICE OF THE MAYOR           "        :("OFFICE OF THE MAYOR"                   ,"None"                 ,"None"),
"PERSONNEL MONITORS            "        :("PERSONNEL MONITORS"                    ,"None"                 ,"None"),
"POLICE DEPARTMENT"                     :("POLICE DEPARTMENT"                     ,"None"                 ,"None"),
"POLICE DEPARTMENT             "        :("POLICE DEPARTMENT"                     ,"None"                 ,"None"),
"PRESIDENT BOROUGH OF MANHATTAN"        :("PRESIDENT OF BOROUGH OFFICE"           ,"None"                 ,"MANHATTAN"),
"PUBLIC ADMINISTRATOR-BRONX    "        :("PUBLIC ADMINISTRATION OFFICE"          ,"None"                 ,"BRONX"),
"PUBLIC ADMINISTRATOR-KINGS    "        :("PUBLIC ADMINISTRATION OFFICE"          ,"None"                 ,"BROOKLYN"),
"PUBLIC ADMINISTRATOR-NEW YORK "        :("PUBLIC ADMINISTRATION OFFICE"          ,"None"                 ,"None"),     # Not sure what to do here... do they mean all of NYC?
"PUBLIC ADMINISTRATOR-QUEENS   "        :("PUBLIC ADMINISTRATION OFFICE"          ,"None"                 ,"QUEENS"),
"PUBLIC ADMINISTRATOR-RICHMOND "        :("PUBLIC ADMINISTRATION OFFICE"          ,"None"                 ,"RICHMOND"),
"PUBLIC ADVOCATE               "        :("PUBLIC ADVOCATE'S OFFICE"              ,"None"                 ,"None"),
"PUBLIC SERVICE CORPS          "        :("PUBLIC SERVICE CORPS"                  ,"None"                 ,"None"),
"QUEENS COMMUNITY BOARD #1     "        :("COMMUNITY BOARD"                       ,"BOARD #1",             "QUEENS"),
"QUEENS COMMUNITY BOARD #10    "        :("COMMUNITY BOARD"                       ,"BOARD #10",            "QUEENS"),
"QUEENS COMMUNITY BOARD #11    "        :("COMMUNITY BOARD"                       ,"BOARD #11",            "QUEENS"),
"QUEENS COMMUNITY BOARD #12    "        :("COMMUNITY BOARD"                       ,"BOARD #12",            "QUEENS"),
"QUEENS COMMUNITY BOARD #13    "        :("COMMUNITY BOARD"                       ,"BOARD #13",            "QUEENS"),
"QUEENS COMMUNITY BOARD #14    "        :("COMMUNITY BOARD"                       ,"BOARD #14",            "QUEENS"),
"QUEENS COMMUNITY BOARD #2     "        :("COMMUNITY BOARD"                       ,"BOARD #2",             "QUEENS"),
"QUEENS COMMUNITY BOARD #3     "        :("COMMUNITY BOARD"                       ,"BOARD #3",             "QUEENS"),
"QUEENS COMMUNITY BOARD #4     "        :("COMMUNITY BOARD"                       ,"BOARD #4",             "QUEENS"),
"QUEENS COMMUNITY BOARD #5     "        :("COMMUNITY BOARD"                       ,"BOARD #5",             "QUEENS"),
"QUEENS COMMUNITY BOARD #6     "        :("COMMUNITY BOARD"                       ,"BOARD #6",             "QUEENS"),
"QUEENS COMMUNITY BOARD #7     "        :("COMMUNITY BOARD"                       ,"BOARD #7",             "QUEENS"),
"QUEENS COMMUNITY BOARD #8     "        :("COMMUNITY BOARD"                       ,"BOARD #8",             "QUEENS"),
"QUEENS COMMUNITY BOARD #9     "        :("COMMUNITY BOARD"                       ,"BOARD #9",             "QUEENS"),
"QUEENS DA  "                           :("DISTRICT ATTORNEY'S OFFICE"            ,"None"                 ,"QUEENS"),
"RICHMOND DA"                           :("DISTRICT ATTORNEY'S OFFICE"            ,"None"                 ,"RICHMOND"),
"SPEC NARCS-DA"                         :("DISTRICT ATTORNEY'S OFFICE"            ,"SPEC NARCS",           "None"),
"STATEN ISLAND COMMUNITY BD #1 "        :("COMMUNITY BOARD"                       ,"BOARD #1",             "STATEN ISLAND"),
"STATEN ISLAND COMMUNITY BD #2 "        :("COMMUNITY BOARD"                       ,"BOARD #2",             "STATEN ISLAND"),
"STATEN ISLAND COMMUNITY BD #3 "        :("COMMUNITY BOARD"                       ,"BOARD #3",             "STATEN ISLAND"),
"TAX COMMISSION                "        :("TAX COMMISSION"                        ,"None",                 "None"),
"TAXI & LIMOUSINE COMMISSION   "        :("TAXI & LIMOUSINE COMMISSION"           ,"None",                 "None"),
"TEACHERS RETIREMENT SYSTEM    "        :("DEPT OF EDUCATION"                     ,"TEACHERS RETIREMENT SYSTEM", "None"),
}
for i in range(3):
    if i == 0: _alias_ = "Agency Name Primary"
    if i == 1: _alias_ = "Agency Name Secondary"
    if i == 2: _alias_ = "Agency Name Borough"
    _lu_ = {}
    for x in _fix_.keys(): _lu_[x] = _fix_[x][i]
    for x in df['Agency Name']: 
        if x not in _lu_.keys(): raise Exception(x) # Make sure everything is covered
    df = df.with_columns(pl.col('Agency Name').replace_strict(_lu_).alias(_alias_))
print(f'{len(df):_}  |  {len(set(df['Agency Name Primary']))=}  |  {len(set(df['Title Description']))=}')
df.columns

In [ ]:
#_parms_ = {'color_by':'Pay Basis'}
#_parms_ = {'color_by':'Per Session Pay Flag', 'w':256, 'h':512}
#_tiles_ = []
#_list_  = list(df.columns)
#_list_  = ['Pay Basis', 'Title Description', 'Agency Name', 'Base Salary']
#for i in range(len(_list_)): _tiles_.append(rt.histogram(df.filter(pl.col('Pay Basis') == 'per Day'), bin_by=_list_[i], **_parms_))
#rt.tile(_tiles_)

In [ ]:
#rt.xy(df, x_field='Agency Name Primary', y_field='Base Salary', w=512, color_by='Pay Basis')

In [ ]:
print(set(df['Agency Name Primary']) & set(df['Title Description'])) # in a perfect world, this should be empty -- so that it's a bipartite graph
df.columns

In [ ]:
#rt.xy(df, x_field='Base Salary', y_field='Regular Hours', color_by='Title Description', dot_size='small', w=1024, h=512, render_x_distribution=100, render_y_distribution=100)

In [ ]:
_relates_ = [('Title Description', 'Agency Name Primary')]
g         = rt.createNetworkXGraph(df, _relates_)
#_pos_     = rt.hyperTreeLayout(g)
#ln_params = {'relationships':_relates_, 'pos':_pos_}
#rt.chordDiagram(df, _relates_, link_style='bundled', skeleton_algorithm='kmeans', w=800, h=800)
#_igl_     = rt.interactiveGraphLayout(df, ln_params, w=1600, h=1200)
#_igl_

In [ ]:
_community_max_sz_    = 1
_communities_         = []
for x in nx.community.louvain_communities(g):
    if len(x) > 160:
        _df_       = df.filter(pl.col('Title Description').is_in(x) & pl.col('Agency Name Primary').is_in(x))
        _subgraph_ = rt.createNetworkXGraph(_df_, _relates_)
        for y in nx.community.louvain_communities(_subgraph_): 
            _community_max_sz_ = max(_community_max_sz_, len(y))
            _communities_.append(y) 
    else:
        _community_max_sz_ = max(_community_max_sz_, len(x)) 
        _communities_.append(x)
_tiles_               = []
_intra_community_dfs_ = []
_node_to_widget_id_   = {}
for _community_ in _communities_:
    _df_  = df.filter(pl.col('Title Description').is_in(_community_) & pl.col('Agency Name Primary').is_in(_community_))

    _df_intra_ = pl.concat([df.filter(~pl.col('Title Description').is_in(_community_) &  pl.col('Agency Name Primary').is_in(_community_)),
                            df.filter( pl.col('Title Description').is_in(_community_) & ~pl.col('Agency Name Primary').is_in(_community_))])
    _intra_community_dfs_.append(_df_intra_)
    _dim_ = 400 + 300*len(_community_)/_community_max_sz_
    _cd_  = rt.chordDiagram(_df_, _relates_, link_style='bundled', skeleton_algorithm='kmeans', draw_labels=True, label_style='circular', draw_background=True, w=_dim_, h=_dim_)
    for _node_ in _community_: _node_to_widget_id_[_node_] = _cd_.widget_id
    _tiles_.append(_cd_)
#rt.table(_tiles_, per_row=4)

In [ ]:
_df_  = pl.concat(_intra_community_dfs_)
#g     = rt.createNetworkXGraph(_df_, _relates_)
#print(len(g.nodes))
#_pos_ = nx.kamada_kawai_layout(g)

In [ ]:
#
# Example Cleanup of Title Description
# 

#_strs_ = []
#for x in set(df['Title Description']):
#    if 'police' in x.lower():
#        _dquoted_ = f'"{x}"'
#        _len_     = 60
#        _strs_.append(f'{_dquoted_:{_len_}}: ({_dquoted_:{_len_}}, "None", "None", "None"),')
#sorted(_strs_)

#ORIGINAL STRING                                               "JOB ROLE"                                "MANAGER"  "QUALIFIER"           "RANK/SENIORITY"
_examples_ = {
"ASSISTANT TO POLICE COMMISSIONOR"                          : ("POLICE COMMISSIONER"                   , False,    "ASSISTANT OF",       "None"),
"COUNSEL TO THE POLICE COMMISSIONER"                        : ("POLICE COMMISSIONER"                   , False,    "COUNSEL OF",         "None"),
"ENVIRONMENTAL POLICE OFFICER"                              : ("ENVIRONMENTAL POLICE OFFICER"          , False,    "None",               "None"),
"ENVIRONMENTAL POLICE OFFICER-MANAGERIAL"                   : ("ENVIRONMENTAL POLICE OFFICER"          , True,     "None",               "None"),
"POLICE ADMINISTRATIVE AIDE"                                : ("POLICE ADMINISTRATIVE AIDE"            , False,    "None",               "None"),
"SENIOR POLICE ADMINISTRATIVE AIDE"                         : ("POLICE ADMINISTRATIVE AIDE"            , False,    "None",               "SENIOR"),
"POLICE ATTENDANT"                                          : ("POLICE ATTENDANT"                      , False,    "None",               "None"),
"POLICE CADET"                                              : ("POLICE CADET"                          , False,    "None",               "None"),
"POLICE OFFICER D/A DETECTIVE 1ST GR"                       : ("POLICE OFFICER"                        , False,    "D/A DETECTIVE",      "GRADE 1"),
"POLICE OFFICER D/A DETECTIVE 2ND GR"                       : ("POLICE OFFICER"                        , False,    "D/A DETECTIVE",      "GRADE 2"),
"POLICE OFFICER"                                            : ("POLICE OFFICER"                        , False,    "None",               "None"),
"POLICE OFFICER, SPECIAL ASSIGNMENT"                        : ("POLICE OFFICER"                        , False,    "SPECIAL ASSIGNMENT", "None"),
"POLICE SURGEON"                                            : ("POLICE SURGEON"                        , False,    "None",               "None"),
"POLICE COMMUNICATIONS TECHNICIAN"                          : ("POLICE COMMUNICATIONS TECHNICIAN"      , False,    "None",               "None"),
"PRINCIPAL POLICE COMMUNICATION TECHNICIAN"                 : ("POLICE COMMUNICATIONS TECHNICIAN"      , False,    "None",               "PRINCIPAL"),
"SUPERVISING POLICE COMMUNICATIONS TECHNICIAN"              : ("POLICE COMMUNICATIONS TECHNICIAN"      , True,     "None",               "None"),
}

from ollama import chat
from ollama import ChatResponse
from pydantic import BaseModel

model = 'gemma3:27b'

def promptModel(title):
    class JobFormatter(BaseModel):
        title_description: str
        job_role:          str
        manager_flag:      bool
        qualifier:         str
        rank:              str
    prompt = f"""For the job title description "{title}", provide the job role, manager flag, qualifier, rank in JSON."""
    for k in _examples_.keys():
        job_role, manager_flag, qualifier, rank = _examples_[k]
        prompt += f"""\nExample: "{k}":
        {{
            "title_description": "{k}",
            "job_role":          "{job_role}",
            "manager_flag":      {manager_flag},
            "qualifier":         "{qualifier}",
            "rank":              "{rank}"
        }}
        """
    response: ChatResponse = chat(model=model, messages=[{ 'role': 'user', 'content':  prompt,},], format=JobFormatter.model_json_schema())
    _formatter_ = JobFormatter.model_validate_json(response['message']['content'])
    return _formatter_.title_description, _formatter_.job_role, _formatter_.manager_flag, _formatter_.qualifier, _formatter_.rank

# _results_lu_ = {}
# for _job_title_ in df['Title Description'].unique():
#     _title_description_, _job_role_, _manager_flag_, _qualifier_, _rank_ = list(promptModel(_job_title_))
#     _results_lu_[_job_title_] = (_title_description_, _job_role_, _manager_flag_, _qualifier_, _rank_)
#     _title_description_, _job_role_, _qualifier_, _rank_                 = f'"{_title_description_}"', f'"{_job_role_}"', f'"{_qualifier_}"', f'"{_rank_}"'
#     _manager_flag_ = (_manager_flag_ == 1)

# _as_list_ = []
# for k in _results_lu_.keys():
#     _title_description_, _job_role_, _manager_flag_, _qualifier_, _rank_ = _results_lu_[k]
#     _title_description_, _job_role_, _qualifier_, _rank_ = f'"{_title_description_}"', f'"{_job_role_}"', f'"{_qualifier_}"', f'"{_rank_}"'
#     _as_list_.append(f'{_title_description_:70}: ({_job_role_:50}, {_qualifier_:40}, {_rank_:25}, {_manager_flag_}),')
# _sorted_ = sorted(_as_list_)
# print('{')
# for x in _sorted_: print(x)
# print('}')

In [ ]:
_fixes_ = {
"12 MONTH SPECIAL EDUCATION ASST. PRINCIPAL"                          : ("SPECIAL EDUCATION"                               , "ASSISTANT PRINCIPAL"                   , "None"                   , False),
"12 MONTH SPECIAL EDUCATION SUPERVISOR"                               : ("SPECIAL EDUCATION"                               , "SUPERVISOR"                            , "None"                   , True),
"ACCOUNTANT"                                                          : ("ACCOUNTANT"                                      , "None"                                  , "None"                   , False),
"ACTUARIAL SPECIALIST LEVEL I"                                        : ("ACTUARIAL SPECIALIST"                            , "None"                                  , "LEVEL I"                , False),
"ACTUARIAL SPECIALIST LEVEL II, OJ"                                   : ("ACTUARIAL SPECIALIST"                            , "None"                                  , "LEVEL II"               , False),
"ADJUNCT ASSISTANT PROFESSOR"                                         : ("PROFESSOR"                                       , "ADJUNCT ASSISTANT"                     , "None"                   , False),
"ADJUNCT ASSOCIATE PROFESSOR"                                         : ("PROFESSOR"                                       , "ADJUNCT ASSOCIATE"                     , "None"                   , False),
"ADJUNCT COLLEGE LAB TECH"                                            : ("COLLEGE LAB TECH"                                , "ADJUNCT"                               , "None"                   , False),
"ADJUNCT LECTURER"                                                    : ("LECTURER"                                        , "ADJUNCT"                               , "None"                   , False),
"ADJUNCT PROFESSOR"                                                   : ("PROFESSOR"                                       , "ADJUNCT"                               , "None"                   , False),
"ADJUNCT SENIOR COLLEGE LAB TECH"                                     : ("COLLEGE LAB TECH"                                , "ADJUNCT"                               , "SENIOR"                 , False),
"ADM DIR FLEET MAINT-MGRL ASGMNT"                                     : ("FLEET MAINTENANCE"                               , "DIRECTOR"                              , "None"                   , True),
"ADM DIR FLEET MAINTENANCE - NM"                                      : ("FLEET MAINTENANCE"                               , "DIRECTOR"                              , "None"                   , True),
"ADM MANAGER"                                                         : ("MANAGER"                                         , "None"                                  , "None"                   , True),
"ADM MANAGER-NON-MGR"                                                 : ("MANAGER"                                         , "NON-MANAGER"                           , "None"                   , False),
"ADM MANAGER-NON-MGRL FROM M1/M2"                                     : ("MANAGER"                                         , "NON-MANAGER"                           , "M1/M2"                  , False),
"ADM MANAGER-NON-MGRL"                                                : ("MANAGER"                                         , "NON-MANAGER"                           , "None"                   , False),
"ADM SCHOOL SECURITY MANAGER"                                         : ("SCHOOL SECURITY MANAGER"                         , "None"                                  , "None"                   , True),
"ADM SCHOOL SECURITY MANAGER-U"                                       : ("SCHOOL SECURITY MANAGER"                         , "None"                                  , "None"                   , True),
"ADM TRAFFIC ENFRCMNT AGENT"                                          : ("TRAFFIC ENFORCEMENT AGENT"                       , "None"                                  , "None"                   , False),
"ADM TRAFFIC ENFRCMNT AGENT-DIRECTOR OF PARKING ENFRCMNT DIST"        : ("TRAFFIC ENFORCEMENT AGENT - PARKING ENFORCEMENT" , "DIRECTOR"                              , "None"                   , True),
"ADMIN ASST-CAMPAIGN FIN BOARD"                                       : ("CAMPAIGN FINANCE BOARD"                          , "ASSISTANT"                             , "None"                   , False),
"ADMIN COMMUNITY RELATIONS SPECIALIST"                                : ("COMMUNITY RELATIONS SPECIALIST"                  , "None"                                  , "None"                   , False),
"ADMIN CONSTRUCTION PROJECT MANAGER"                                  : ("CONSTRUCTION PROJECT MANAGER"                    , "MANAGER"                               , "None"                   , True),
"ADMIN CONTRACT SPECIALIST"                                           : ("CONTRACT SPECIALIST"                             , "None"                                  , "None"                   , False),
"ADMIN DIR OF LABORATORY"                                             : ("DIRECTOR OF LABORATORY"                          , "DIRECTOR"                              , "None"                   , True),
"ADMIN HOUSING DEVELOPMENT SPEC"                                      : ("HOUSING DEVELOPMENT SPECIALIST"                  , "None"                                  , "None"                   , False),
"ADMIN INSPECTOR"                                                     : ("INSPECTOR"                                       , "None"                                  , "None"                   , False),
"ADMIN JOB OPOR SPEC-MANAGERIAL"                                      : ("JOB OPPORTUNITY SPECIALIST"                      , "MANAGER"                               , "None"                   , True),
"ADMIN JOB OPPORTUNITY SPEC NM"                                       : ("JOB OPPORTUNITY SPECIALIST"                      , "None"                                  , "None"                   , False),
"ADMIN LANDMARKS PRESERVATIONIST"                                     : ("LANDMARKS PRESERVATIONIST"                       , "None"                                  , "None"                   , False),
"ADMIN SCHL SECUR MGR"                                                : ("SCHOOL SECURITY MANAGER"                         , "MANAGER"                               , "None"                   , True),
"ADMIN SCHL SECUR MGR-MGL"                                            : ("SCHOOL SECURITY MANAGER"                         , "MANAGER"                               , "None"                   , True),
"ADMIN SCHOOL SECURITY"                                               : ("SCHOOL SECURITY"                                 , "None"                                  , "None"                   , False),
"ADMIN TESTS & MEAS SPEC"                                             : ("TESTS & MEASUREMENTS SPECIALIST"                 , "None"                                  , "None"                   , False),
"ADMIN TRAFFIC ENF AGNT"                                              : ("TRAFFIC ENFORCEMENT AGENT"                       , "None"                                  , "None"                   , False),
"ADMIN TRAFFIC ENF AGNT-UNION"                                        : ("TRAFFIC ENFORCEMENT AGENT"                       , "None"                                  , "UNION"                  , False),
"ADMIN TRAFFIC ENFRCMNT AGENT"                                        : ("TRAFFIC ENFORCEMENT AGENT"                       , "None"                                  , "None"                   , False),
"ADMINISTRATION PUBLIC RECORD OFFICER"                                : ("PUBLIC RECORD OFFICER"                           , "None"                                  , "None"                   , False),
"ADMINISTRATIVE ACCOUNTANT"                                           : ("ACCOUNTANT"                                      , "None"                                  , "None"                   , False),
"ADMINISTRATIVE ACTUARY"                                              : ("ACTUARY"                                         , "None"                                  , "None"                   , False),
"ADMINISTRATIVE ARCHITECT"                                            : ("ARCHITECT"                                       , "None"                                  , "None"                   , False),
"ADMINISTRATIVE ASSESSOR"                                             : ("ASSESSOR"                                        , "None"                                  , "None"                   , False),
"ADMINISTRATIVE ASSISTANT TO COMMUNITY EDUCATION COUNCIL"             : ("COMMUNITY EDUCATION COUNCILOR"                   , "ASSISTANT TO"                          , "None"                   , False),
"ADMINISTRATIVE ASSISTANT TO THE COMPTROLLER"                         : ("COMPTROLLER"                                     , "ASSISTANT TO"                          , "None"                   , False),
"ADMINISTRATIVE ASSISTANT TO THE MAYOR"                               : ("MAYOR"                                           , "ASSISTANT TO"                          , "None"                   , False),
"ADMINISTRATIVE ASSISTANT"                                            : ("ADMINISTRATIVE ASSISTANT"                        , "None"                                  , "None"                   , False),
"ADMINISTRATIVE ASSOCIATE"                                            : ("ADMINISTRATIVE ASSOCIATE"                        , "None"                                  , "None"                   , False),
"ADMINISTRATIVE ATTORNEY"                                             : ("ATTORNEY"                                        , "None"                                  , "None"                   , False),
"ADMINISTRATIVE BLASTING INSPECTOR"                                   : ("BLASTING INSPECTOR"                              , "None"                                  , "None"                   , False),
"ADMINISTRATIVE BOROUGH SUPERINTENDENT"                               : ("BOROUGH SUPERINTENDENT"                          , "SUPERINTENDENT"                        , "None"                   , True),
"ADMINISTRATIVE BUSINESS PROMOTION COORDINATOR"                       : ("BUSINESS PROMOTION COORDINATOR"                  , "None"                                  , "None"                   , False),
"ADMINISTRATIVE CHIEF"                                                : ("CHIEF"                                           , "None"                                  , "None"                   , True),
"ADMINISTRATIVE CITY PLANNER"                                         : ("CITY PLANNER"                                    , "None"                                  , "None"                   , False),
"ADMINISTRATIVE CLAIM EXAMINER"                                       : ("CLAIM EXAMINER"                                  , "None"                                  , "None"                   , False),
"ADMINISTRATIVE COMMUNITY RELATIONS SPECIALIST"                       : ("COMMUNITY RELATIONS SPECIALIST"                  , "None"                                  , "None"                   , False),
"ADMINISTRATIVE CONSTRUCTION PROJECT MANAGER"                         : ("CONSTRUCTION PROJECT MANAGER"                    , "None"                                  , "None"                   , False),
"ADMINISTRATIVE CONSULTANT"                                           : ("CONSULTANT"                                      , "None"                                  , "None"                   , False),
"ADMINISTRATIVE DEPUTY REGISTER"                                      : ("REGISTER"                                        , "DEPUTY"                                , "None"                   , False),
"ADMINISTRATIVE DIRECTOR OF LABORATORY"                               : ("LABORATORY DIRECTOR"                             , "DIRECTOR"                              , "None"                   , True),
"ADMINISTRATIVE DIRECTOR OF MARINE MAINTENANCE"                       : ("DIRECTOR OF MARINE MAINTENANCE"                  , "DIRECTOR"                              , "None"                   , True),
"ADMINISTRATIVE DIRECTOR OF RESIDENTIAL CHILD CARE"                   : ("DIRECTOR OF RESIDENTIAL CHILD CARE"              , "DIRECTOR"                              , "None"                   , True),
"ADMINISTRATIVE DIRECTOR OF SOCIAL SERVICES"                          : ("SOCIAL SERVICES DIRECTOR"                        , "DIRECTOR"                              , "None"                   , True),
"ADMINISTRATIVE EDUCATION ANALYST"                                    : ("EDUCATION ANALYST"                               , "None"                                  , "None"                   , False),
"ADMINISTRATIVE EDUCATION OFFICER"                                    : ("EDUCATION OFFICER"                               , "None"                                  , "None"                   , False),
"ADMINISTRATIVE ENGINEER"                                             : ("ENGINEER"                                        , "None"                                  , "None"                   , False),
"ADMINISTRATIVE FIRE PROTECTION INSPECTOR"                            : ("FIRE PROTECTION INSPECTOR"                       , "None"                                  , "None"                   , False),
"ADMINISTRATIVE GRAPHIC ARTIST"                                       : ("GRAPHIC ARTIST"                                  , "None"                                  , "None"                   , False),
"ADMINISTRATIVE HORTICULTURIST"                                       : ("HORTICULTURIST"                                  , "None"                                  , "None"                   , False),
"ADMINISTRATIVE HOUSING DEVELOPMENT SPECIALIST"                       : ("HOUSING DEVELOPMENT SPECIALIST"                  , "None"                                  , "None"                   , False),
"ADMINISTRATIVE HOUSING MANAGER"                                      : ("HOUSING MANAGER"                                 , "MANAGER"                               , "None"                   , True),
"ADMINISTRATIVE HOUSING SUPERINTENDENT"                               : ("HOUSING SUPERINTENDENT"                          , "SUPERINTENDENT"                        , "None"                   , True),
"ADMINISTRATIVE INSPECTOR"                                            : ("ADMINISTRATIVE INSPECTOR"                        , "None"                                  , "None"                   , False),
"ADMINISTRATIVE INVESTIGATOR"                                         : ("INVESTIGATOR"                                    , "None"                                  , "None"                   , False),
"ADMINISTRATIVE JUVENILE COUNSELOR"                                   : ("JUVENILE COUNSELOR"                              , "None"                                  , "None"                   , False),
"ADMINISTRATIVE LABOR RELATIONS ANALYST"                              : ("LABOR RELATIONS ANALYST"                         , "None"                                  , "None"                   , False),
"ADMINISTRATIVE LANDMARKS PRESERVATIONIST"                            : ("LANDMARKS PRESERVATIONIST"                       , "None"                                  , "None"                   , False),
"ADMINISTRATIVE LANDSCAPE ARCHITECT"                                  : ("LANDSCAPE ARCHITECT"                             , "None"                                  , "None"                   , False),
"ADMINISTRATIVE LAW JUDGE"                                            : ("LAW JUDGE"                                       , "None"                                  , "None"                   , False),
"ADMINISTRATIVE MANAGEMENT AUDITOR"                                   : ("MANAGEMENT AUDITOR"                              , "None"                                  , "None"                   , False),
"ADMINISTRATIVE MANAGER"                                              : ("ADMINISTRATIVE MANAGER"                          , "MANAGER"                               , "None"                   , True),
"ADMINISTRATIVE NUTRITIONIST"                                         : ("NUTRITIONIST"                                    , "None"                                  , "None"                   , False),
"ADMINISTRATIVE PARKS & RECREATION MANAGER"                           : ("PARKS & RECREATION MANAGER"                      , "MANAGER"                               , "None"                   , True),
"ADMINISTRATIVE PRINTING SERVICES MANAGER"                            : ("PRINTING SERVICES MANAGER"                       , "MANAGER"                               , "None"                   , True),
"ADMINISTRATIVE PROBATION OFFICER"                                    : ("PROBATION OFFICER"                               , "None"                                  , "None"                   , False),
"ADMINISTRATIVE PROCUREMENT ANALYST"                                  : ("PROCUREMENT ANALYST"                             , "None"                                  , "None"                   , False),
"ADMINISTRATIVE PROGRAM OFFICER"                                      : ("PROGRAM OFFICER"                                 , "None"                                  , "None"                   , False),
"ADMINISTRATIVE PROJECT DIRECTOR"                                     : ("PROJECT DIRECTOR"                                , "None"                                  , "None"                   , False),
"ADMINISTRATIVE PROJECT MANAGER"                                      : ("PROJECT MANAGER"                                 , "None"                                  , "None"                   , False),
"ADMINISTRATIVE PSYCHOLOGIST"                                         : ("PSYCHOLOGIST"                                    , "None"                                  , "None"                   , False),
"ADMINISTRATIVE PUBLIC HEALTH NURSE"                                  : ("PUBLIC HEALTH NURSE"                             , "None"                                  , "None"                   , False),
"ADMINISTRATIVE PUBLIC HEALTH SANITARIAN"                             : ("PUBLIC HEALTH SANITARIAN"                        , "None"                                  , "None"                   , False),
"ADMINISTRATIVE PUBLIC INFORMATION SPECIALIST"                        : ("PUBLIC INFORMATION SPECIALIST"                   , "None"                                  , "None"                   , False),
"ADMINISTRATIVE QUALITY ASSURANCE SPECIALIST"                         : ("QUALITY ASSURANCE SPECIALIST"                    , "None"                                  , "None"                   , False),
"ADMINISTRATIVE REAL PROPERTY MANAGER"                                : ("REAL PROPERTY MANAGER"                           , "None"                                  , "None"                   , False),
"ADMINISTRATIVE RETIREMENT BENEFITS SPECIALIST"                       : ("RETIREMENT BENEFITS SPECIALIST"                  , "None"                                  , "None"                   , False),
"ADMINISTRATIVE SANITATION ENFORCEMENT AGENT"                         : ("SANITATION ENFORCEMENT AGENT"                    , "None"                                  , "None"                   , False),
"ADMINISTRATIVE SCHOOL FOOD SERVICE MANAGER"                          : ("SCHOOL FOOD SERVICE MANAGER"                     , "MANAGER"                               , "None"                   , True),
"ADMINISTRATIVE SPACE ANALYST"                                        : ("SPACE ANALYST"                                   , "None"                                  , "None"                   , False),
"ADMINISTRATIVE STAFF ANALYST"                                        : ("STAFF ANALYST"                                   , "None"                                  , "None"                   , False),
"ADMINISTRATIVE STOREKEEPER"                                          : ("STOREKEEPER"                                     , "None"                                  , "None"                   , False),
"ADMINISTRATIVE SUPERINTENDENT OF BRIDGE OPERATIONS"                  : ("SUPERINTENDENT OF BRIDGE OPERATIONS"             , "SUPERINTENDENT"                        , "None"                   , True),
"ADMINISTRATIVE SUPERINTENDENT OF HIGHWAY OPERATIONS"                 : ("SUPERINTENDENT OF HIGHWAY OPERATIONS"            , "SUPERINTENDENT"                        , "None"                   , True),
"ADMINISTRATIVE SUPERVISOR OF BUILDING MAINTENANCE"                   : ("BUILDING MAINTENANCE SUPERVISOR"                 , "SUPERVISOR"                            , "None"                   , True),
"ADMINISTRATIVE SUPERVISOR OF SKILLED TRADES"                         : ("SKILLED TRADES SUPERVISOR"                       , "SUPERVISOR"                            , "None"                   , True),
"ADMINISTRATIVE TAX AUDITOR"                                          : ("TAX AUDITOR"                                     , "None"                                  , "None"                   , False),
"ADMINISTRATIVE TAXI & LIMOUSINE INSPECTOR"                           : ("INSPECTOR"                                       , "None"                                  , "None"                   , False),
"ADMINISTRATIVE TRANSPORTATION COORDINATOR"                           : ("TRANSPORTATION COORDINATOR"                      , "None"                                  , "None"                   , False),
"ADMINISTRATOR OF MEDICAL AFFAIRS"                                    : ("MEDICAL AFFAIRS ADMINISTRATOR"                   , "None"                                  , "None"                   , False),
"ADMINISTRATOR SUPT CAMPUS B/G"                                       : ("ADMINISTRATOR"                                   , "CAMPUS B/G"                            , "None"                   , False),
"ADMINISTRATOR"                                                       : ("ADMINISTRATOR"                                   , "None"                                  , "None"                   , False),
"ADULT ED-30 H"                                                       : ("ADULT EDUCATOR"                                  , "None"                                  , "30 H"                   , False),
"ADULT ED-35 H"                                                       : ("ADULT EDUCATOR"                                  , "None"                                  , "35 H"                   , False),
"ADULT EDUCAT TEACH - REG SUB"                                        : ("ADULT EDUCATOR"                                  , "SUBSTITUTE"                            , "None"                   , False),
"ADULT EDUCATION TEACHER"                                             : ("ADULT EDUCATOR"                                  , "None"                                  , "None"                   , False),
"ADULT EDUCATOR ASSISTANT COORD"                                      : ("ADULT EDUCATOR"                                  , "ASSISTANT COORDINATOR"                 , "None"                   , False),
"AGENCY ATTORNEY INTERN"                                              : ("ATTORNEY"                                        , "INTERN"                                , "None"                   , False),
"AGENCY ATTORNEY INTERNE"                                             : ("ATTORNEY"                                        , "INTERN"                                , "None"                   , False),
"AGENCY ATTORNEY"                                                     : ("ATTORNEY"                                        , "None"                                  , "None"                   , False),
"AGENCY CHIEF CONTRACTING OFFICER"                                    : ("CONTRACTING OFFICER"                             , "CHIEF"                                 , "None"                   , False),
"AGENCY DEPUTY MEDICAL DIRECTOR"                                      : ("MEDICAL DIRECTOR"                                , "DEPUTY DIRECTOR"                       , "None"                   , True),
"AGENCY MEDICAL DIRECTOR"                                             : ("MEDICAL DIRECTOR"                                , "DIRECTOR"                              , "None"                   , True),
"AGENCY SECURITY DIRECTOR"                                            : ("SECURITY DIRECTOR"                               , "DIRECTOR"                              , "None"                   , True),
"AIR POLLUTION INSPECTOR"                                             : ("AIR POLLUTION INSPECTOR"                         , "None"                                  , "None"                   , False),
"ANALYST"                                                             : ("ANALYST"                                         , "None"                                  , "None"                   , False),
"ANNUAL ED PARA"                                                      : ("PARAPROFESSIONAL EDUCATOR"                       , "ANNUAL"                                , "None"                   , False),
"APPRAISER"                                                           : ("APPRAISER"                                       , "None"                                  , "None"                   , False),
"APPRENTICE INSPECTOR"                                                : ("INSPECTOR"                                       , "APPRENTICE"                            , "None"                   , False),
"APPRENTICE"                                                          : ("APPRENTICE"                                      , "None"                                  , "None"                   , False),
"ARCHITECT"                                                           : ("ARCHITECT"                                       , "None"                                  , "None"                   , False),
"ARCHITECTURAL INTERN"                                                : ("ARCHITECT"                                       , "INTERN"                                , "None"                   , False),
"AREA MANAGER OF SCHOOL MAINTENANCE"                                  : ("AREA MANAGER OF SCHOOL MAINTENANCE"              , "None"                                  , "None"                   , False),
"AREA SUPERVISOR"                                                     : ("AREA SUPERVISOR"                                 , "SUPERVISOR"                            , "None"                   , True),
"ARTS PROGRAM SPECIALIST"                                             : ("ARTS PROGRAM SPECIALIST"                         , "None"                                  , "None"                   , False),
"ASBESTOS HANDLER SUPERVISOR"                                         : ("ASBESTOS HANDLER"                                , "SUPERVISOR"                            , "None"                   , True),
"ASBESTOS HANDLER"                                                    : ("ASBESTOS HANDLER"                                , "None"                                  , "None"                   , False),
"ASBESTOS HAZARD INVESTIGATOR"                                        : ("ASBESTOS HAZARD INVESTIGATOR"                    , "None"                                  , "None"                   , False),
"ASPIRING PRINCIPAL"                                                  : ("PRINCIPAL"                                       , "ASPIRING"                              , "None"                   , False),
"ASSISTANT CHIEF RACKET INVESTIGATOR"                                 : ("RACKET INVESTIGATOR"                             , "ASSISTANT CHIEF"                       , "None"                   , False),
"ASSISTANCE FINANCE OFFICER"                                          : ("FINANCE OFFICER"                                 , "ASSISTANCE"                            , "None"                   , False),
"ASSISTANT ADVOCATE-PD"                                               : ("ADVOCATE"                                        , "ASSISTANT"                             , "None"                   , False), # hopefully, it's assigned to the Police Department
"ASSISTANT CHIEF FIRE MARSHAL"                                        : ("CHIEF FIRE MARSHAL"                              , "ASSISTANT CHIEF"                       , "None"                   , False),
"ASSISTANT CHIEF OF DEPARTMENT"                                       : ("CHIEF OF DEPARTMENT"                             , "ASSISTANT CHIEF"                       , "None"                   , False),
"ASSISTANT COLLEGE SECURITY DIRECTOR"                                 : ("COLLEGE SECURITY"                                , "ASSISTANT DIRECTOR"                    , "None"                   , False),
"ASSISTANT COMMISSIONER FOR FACILITIES OPERATIONS"                    : ("COMMISSIONER"                                    , "ASSISTANT FACILITIES OPERATIONS"       , "None"                   , False),
"ASSISTANT COMMISSIONER FOR LABOR RELATIONS & ADMIN"                  : ("COMMISSIONER"                                    , "ASSISTANT LABOR RELATIONS"             , "None"                   , False),
"ASSISTANT COMMISSIONER FOR MANAGEMENT ACD-DOSS"                      : ("COMMISSIONER"                                    , "ASSISTANT MANAGEMENT ACD-DOSS"         , "None"                   , False),
"ASSISTANT COUNSEL-PD"                                                : ("COUNSEL"                                         , "ASSISTANT"                             , "None"                   , False), # hopefully, it's assigned to the Police Department
"ASSISTANT DEPUTY COMMISSIONER SS"                                    : ("COMMISSIONER"                                    , "ASSISTANT DEPUTY"                      , "None"                   , False),
"ASSISTANT DIRECTOR HEALTH PROGRAM"                                   : ("DIRECTOR HEALTH PROGRAM"                         , "ASSISTANT DIRECTOR"                    , "None"                   , False),
"ASSISTANT DIRECTOR OF ADMINISTRATION"                                : ("DIRECTOR OF ADMINISTRATION"                      , "ASSISTANT DIRECTOR"                    , "None"                   , False),
"ASSISTANT DIRECTOR OF FINANCE"                                       : ("DIRECTOR OF FINANCE"                             , "ASSISTANT DIRECTOR"                    , "None"                   , False),
"ASSISTANT DIRECTOR OF FORENSIC BIOLOGY"                              : ("DIRECTOR OF FORENSIC BIOLOGY"                    , "ASSISTANT DIRECTOR"                    , "None"                   , False),
"ASSISTANT DIRECTOR OF LEGAL SERVICES"                                : ("DIRECTOR OF LEGAL SERVICES"                      , "ASSISTANT DIRECTOR"                    , "None"                   , False),
"ASSISTANT DIRECTOR OF TOXICOLOGY"                                    : ("DIRECTOR OF TOXICOLOGY"                          , "ASSISTANT DIRECTOR"                    , "None"                   , False),
"ASSISTANT MECHANICAL ENGINEER"                                       : ("MECHANICAL ENGINEER"                             , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT MEDIA SERVICES TECHNICAN"                                  : ("MEDIA SERVICES TECHNICAN"                        , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT PRINCIPAL CUSTODIAL SUPERVISOR"                            : ("CUSTODIAL SUPERVISOR"                            , "ASSISTANT PRINCIPAL"                   , "None"                   , False),
"ASSISTANT PRINTING PRESS OPERATOR"                                   : ("PRINTING PRESS OPERATOR"                         , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT RESIDENT BUILDING SUPT"                                    : ("RESIDENT BUILDING SUPERINTENDENT"                , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT SERGEANT AT ARMS"                                          : ("SERGEANT AT ARMS"                                , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT SUPERINTENDENT OF SCHOOLS"                                 : ("SUPERINTENDENT OF SCHOOLS"                       , "ASSISTANT SUPERINTENDENT"              , "None"                   , False),
"ASIST SYSTMS ANALYST"                                                : ("SYSTEMS ANALYST"                                 , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT VICE PRESIDENT"                                            : ("PRESIDENT"                                       , "ASSISTANT VICE PRESIDENT"              , "None"                   , False),
"ASSIST COORDINATING MANAGER"                                         : ("COORDINATING MANAGER"                            , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT ACCOUNTANT"                                                : ("ACCOUNTANT"                                      , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT ADMINISTRATOR OF PUBLIC AFFAIRS"                           : ("ADMINISTRATOR OF PUBLIC AFFAIRS"                 , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT ADMINISTRATOR"                                             : ("ADMINISTRATOR"                                   , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT ADVOCATE"                                                  : ("ADVOCATE"                                        , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT ARCHITECT"                                                 : ("ARCHITECT"                                       , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT BUDGET ANALYST"                                            : ("BUDGET ANALYST"                                  , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT CAPTAIN"                                                   : ("CAPTAIN"                                         , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT CARETAKER"                                                 : ("CARETAKER"                                       , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT CHEMICAL ENGINEER"                                         : ("CHEMICAL ENGINEER"                               , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT CHEMIST"                                                   : ("CHEMIST"                                         , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT CITY ASSESSOR"                                             : ("CITY ASSESSOR"                                   , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT CITY HIGHWAY REPAIRER"                                     : ("CITY HIGHWAY REPAIRER"                           , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT CIVIL ENGINEER"                                            : ("CIVIL ENGINEER"                                  , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT COMMISSIONER OF LABOR RELATIONS"                           : ("COMMISSIONER"                                    , "ASSISTANT OF LABOR RELATIONS"          , "None"                   , False),
"ASSISTANT COMMISSIONER OF OPERATIONS"                                : ("COMMISSIONER"                                    , "ASSISTANT OF OPERATIONS"               , "None"                   , False),
"ASSISTANT COMMISSIONER"                                              : ("COMMISSIONER"                                    , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT CORPORATION COUNSEL"                                       : ("CORPORATION COUNSEL"                             , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT COUNSEL"                                                   : ("COUNSEL"                                         , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT DEAN"                                                      : ("DEAN"                                            , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT DEPUTY ADMINISTRATOR"                                      : ("ADMINISTRATOR"                                   , "ASSISTANT DEPUTY"                      , "None"                   , False),
"ASSISTANT DEPUTY COMMISSIONER"                                       : ("COMMISSIONER"                                    , "ASSISTANT DEPUTY"                      , "None"                   , False),
"ASSISTANT DIRECTOR OF INTERGROUP RELATIONS"                          : ("DIRECTOR OF INTERGROUP RELATIONS"                , "ASSISTANT DIRECTOR"                    , "None"                   , False),
"ASSISTANT DIRECTOR"                                                  : ("DIRECTOR"                                        , "ASSISTANT DIRECTOR"                    , "None"                   , False),
"ASSISTANT DISTRICT ATTORNEY"                                         : ("DISTRICT ATTORNEY"                               , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT DISTRICT MANAGER"                                          : ("DISTRICT MANAGER"                                , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT ELECTRICAL ENGINEER"                                       : ("ELECTRICAL ENGINEER"                             , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT ENVIRONMENTAL ENGINEER"                                    : ("ENVIRONMENTAL ENGINEER"                          , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT EXECUTIVE DIRECTOR"                                        : ("EXECUTIVE DIRECTOR"                              , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT HIGHWAY TRANSPORTATION SPECIALIST"                         : ("HIGHWAY TRANSPORTATION SPECIALIST"               , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT HOUSEKEEPER"                                               : ("HOUSEKEEPER"                                     , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT HOUSING MANAGER"                                           : ("HOUSING MANAGER"                                 , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT LANDSCAPE ARCHITECT"                                       : ("LANDSCAPE ARCHITECT"                             , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT LAUNDRY SUPERVISOR"                                        : ("LAUNDRY SUPERVISOR"                              , "ASSISTANT SUPERVISOR"                  , "None"                   , False),
"ASSISTANT LEGISLATIVE REPRESENTATIVE"                                : ("LEGISLATIVE REPRESENTATIVE"                      , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT PLAN EXAMINER"                                             : ("PLAN EXAMINER"                                   , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT PRINCIPAL ASSIGNED"                                        : ("PRINCIPAL"                                       , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT PRINCIPAL"                                                 : ("PRINCIPAL"                                       , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT PROFESSOR"                                                 : ("PROFESSOR"                                       , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT PURCHASING AGENT"                                          : ("PURCHASING AGENT"                                , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT RETIREMENT BENEFITS EXAMINER"                              : ("RETIREMENT BENEFITS EXAMINER"                    , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT SECRETARY TO THE COMPTROLLER"                              : ("COMPTROLLER"                                     , "ASSISTANT SECRETARY OF"                , "None"                   , False),
"ASSISTANT SUPERINTENDENT OF WELFARE SHELTERS"                        : ("SUPERINTENDENT OF WELFARE SHELTERS"              , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT SUPERINTENDENT"                                            : ("SUPERINTENDENT"                                  , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT TEACHER"                                                   : ("TEACHER"                                         , "ASSISTANT"                             , "None"                   , False),
"ASSISTANT TO CITY ADMINISTRATOR"                                     : ("CITY ADMINISTRATOR"                              , "ASSISTANT OF"                          , "None"                   , False),
"ASSISTANT TO HEO"                                                    : ("HEO"                                             , "ASSISTANT OF"                          , "None"                   , False),
"ASSISTANT TO POLICE COMMISSIONOR"                                    : ("POLICE COMMISSIONOR"                             , "ASSISTANT OF"                          , "None"                   , False),
"ASSISTANT TO THE CHAIRMAN"                                           : ("CHAIRMAN"                                        , "ASSISTANT OF"                          , "None"                   , False),
"ASSISTANT TO THE COMMISSIONER OF PARKS AND RECREATION"               : ("COMMISSIONER OF PARKS AND RECREATION"            , "ASSISTANT OF"                          , "None"                   , False),
"ASSISTANT TO THE COMMISSIONER"                                       : ("COMMISSIONER"                                    , "ASSISTANT OF"                          , "None"                   , False),
"ASSISTANT TO THE COMPTROLLER"                                        : ("COMPTROLLER"                                     , "ASSISTANT OF"                          , "None"                   , False),
"ASSISTANT TO THE DEPUTY COMMISSIONER"                                : ("DEPUTY COMMISSIONER"                             , "ASSISTANT OF"                          , "None"                   , False),
"ASSISTANT TO THE DEPUTY MAYOR"                                       : ("DEPUTY MAYOR"                                    , "ASSISTANT OF"                          , "None"                   , False),
"ASSISTANT TO THE MAYOR"                                              : ("MAYOR"                                           , "ASSISTANT OF"                          , "None"                   , False),
"ASSISTANT TO THE PRESIDENT"                                          : ("PRESIDENT"                                       , "ASSISTANT OF"                          , "None"                   , False),
"ASSISTANT TO THE PUBLIC ADVOCATE"                                    : ("PUBLIC ADVOCATE"                                 , "ASSISTANT OF"                          , "None"                   , False),
"ASSISTANT URBAN DESIGNER"                                            : ("URBAN DESIGNER"                                  , "ASSISTANT"                             , "None"                   , False),
"ASSOC INSPECTOR"                                                     : ("INSPECTOR"                                       , "ASSOCIATE"                             , "None"                   , False),
"ASSOC SPVR OF SCHOOL SECURITY"                                       : ("SCHOOL SECURITY"                                 , "ASSOCIATE SUPERVISOR"                  , "None"                   , True),
"ASSOC SUPVR OF SCHL SEC"                                             : ("SCHOOL SECURITY"                                 , "ASSOCIATE SUPERVISOR"                  , "None"                   , True),
"ASSOC WATER USE INSPECTOR-MGRL"                                      : ("WATER USE INSPECTOR"                             , "ASSOCIATE"                             , "None"                   , True),
"ASSOCIATE ADMINISTRATOR"                                             : ("ADMINISTRATOR"                                   , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE AIR POLLUTION INSPR"                                       : ("AIR POLLUTION INSPECTOR"                         , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE ARTS PROGRAMS SPECIALIST"                                  : ("ARTS PROGRAMS SPECIALIST"                        , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE CALL CENTER REPRESENTATIVE-NON-SPVR"                       : ("CALL CENTER REPRESENTATIVE"                      , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE CHEMIST"                                                   : ("CHEMIST"                                         , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE CHIEF RACKETS INVESTIGATOR"                                : ("CHIEF RACKETS INVESTIGATOR"                      , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE COMMISSIONER FOR ADULT SERVICES"                           : ("COMMISSIONER"                                    , "ASSOCIATE FOR ADULT SERVICES"          , "None"                   , False),
"ASSOCIATE COMMISSIONER FOR EMPLOYMENT SERVICE"                       : ("COMMISSIONER"                                    , "ASSOCIATE FOR EMPLOYMENT SERVICE"      , "None"                   , False),
"ASSOCIATE COMMISSIONER FOR PERSONNEL SERVICE"                        : ("COMMISSIONER"                                    , "ASSOCIATE FOR PERSONNEL SERVICE"       , "None"                   , False),
"ASSOCIATE COMMISSIONER"                                              : ("COMMISSIONER"                                    , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE CONTRACT SPECIALIST"                                       : ("CONTRACT SPECIALIST"                             , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE CORRECTIONAL COUNSELOR"                                    : ("CORRECTIONAL COUNSELOR"                          , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE COUNSEL"                                                   : ("COUNSEL"                                         , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE DEAN"                                                      : ("DEAN"                                            , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE DIRECTOR HEALTH PROGRAM"                                   : ("DIRECTOR"                                        , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE DIRECTOR OF NURSING SERVICES"                              : ("DIRECTOR OF NURSING SERVICES"                    , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE DIRECTOR-OPERATIONAL SERVICES"                             : ("DIRECTOR OF OPERATIONAL SERVICES"                , "ASSOCIATE"                             , "None"                   , True),
"ASSOCIATE EDUCATION ANALYST"                                         : ("EDUCATION ANALYST"                               , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE EDUCATION OFFICER"                                         : ("EDUCATION OFFICER"                               , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE ENGINEERING TECHNICIAN"                                    : ("ENGINEERING TECHNICIAN"                          , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE EXECUTIVE DIRECTOR"                                        : ("EXECUTIVE DIRECTOR"                              , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE FINGERPRINT TECHNICIAN"                                    : ("FINGERPRINT TECHNICIAN"                          , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE FIRE PROTECTION INSPECTOR"                                 : ("FIRE PROTECTION INSPECTOR"                       , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE FRAUD INVESTIGATOR"                                        : ("FRAUD INVESTIGATOR"                              , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE HOUSING DEVELOPMENT SPECIALIST"                            : ("HOUSING DEVELOPMENT SPECIALIST"                  , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE HUMAN RIGHTS SPECIALIST"                                   : ("HUMAN RIGHTS SPECIALIST"                         , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE INSPECTOR"                                                 : ("INSPECTOR"                                       , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE INVESTIGATOR"                                              : ("INVESTIGATOR"                                    , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE JOB OPPORTUNITY SPECIALIST"                                : ("JOB OPPORTUNITY SPECIALIST"                      , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE JUVENILE COUNSELOR"                                        : ("JUVENILE COUNSELOR"                              , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE LABOR RELATIONS ANALYST"                                   : ("LABOR RELATIONS ANALYST"                         , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE LABORATORY MICROBIOLOGIST"                                 : ("LABORATORY MICROBIOLOGIST"                       , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE LANDMARKS PRESERVATIONIST"                                 : ("LANDMARKS PRESERVATIONIST"                       , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE OPERATIONS COMMUNICATIONS SPECIALIST"                      : ("OPERATIONS COMMUNICATIONS SPECIALIST"            , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE PARK SERVICE WORKER"                                       : ("PARK SERVICE WORKER"                             , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE PARKING CONTROL SPECIALIST"                                : ("PARKING CONTROL SPECIALIST"                      , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE PROFESSOR"                                                 : ("PROFESSOR"                                       , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE PROJECT MANAGER"                                           : ("PROJECT MANAGER"                                 , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE PUBLIC HEALTH SANITARIAN"                                  : ("PUBLIC HEALTH SANITARIAN"                        , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE PUBLIC INFORMATION SPECIALIST"                             : ("PUBLIC INFORMATION SPECIALIST"                   , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE PUBLIC RECORDS OFFICER"                                    : ("PUBLIC RECORDS OFFICER"                          , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE QUALITY ASSURANCE SPECIALIST"                              : ("QUALITY ASSURANCE SPECIALIST"                    , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE REAL PROPERTY MANAGER"                                     : ("REAL PROPERTY MANAGER"                           , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE REHABILITATION SPECIALIST"                                 : ("REHABILITATION SPECIALIST"                       , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE RETIREMENT BENEFITS EXAMINER"                              : ("RETIREMENT BENEFITS EXAMINER"                    , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE SANITATION ENFORCEMENT AGENT"                              : ("SANITATION ENFORCEMENT AGENT"                    , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE SCHOOL FOOD SERVICE MANAGER"                               : ("SCHOOL FOOD SERVICE MANAGER"                     , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE STAFF ANALYST"                                             : ("STAFF ANALYST"                                   , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE TAXI & LIMOUSINE INSPECTOR"                                : ("TAXI & LIMOUSINE INSPECTOR"                      , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE TRAFFIC ENFORCEMENT AGENT"                                 : ("TRAFFIC ENFORCEMENT AGENT"                       , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE URBAN DESIGNER"                                            : ("URBAN DESIGNER"                                  , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE URBAN PARK RANGER"                                         : ("URBAN PARK RANGER"                               , "ASSOCIATE"                             , "None"                   , False),
"ASSOCIATE WATER USE INPECTOR"                                        : ("WATER USE INSPECTOR"                             , "ASSOCIATE"                             , "None"                   , False),
"ASST COMMISSIONER FOR AGENCY PREPAREDNESS AND RESPONSE"              : ("COMMISSIONER"                                    , "ASSISTANT FOR AGENCY PREPAREDNESS AND RESPONSE",      "None"      , False), 
"ASST COMMISSIONER FOR NON-SECURE DETENTION JUVENILE JUSTICE"         : ("COMMISSIONER"                                    , "ASSISTANT FOR NON-SECURE DETENTION JUVENILE JUSTICE", "None"      , False),
"ASST COMMISSIONER FOR PLANNING & PROGRAM DEV"                        : ("COMMISSIONER"                                    , "ASSISTANT FOR PLANNING & PROGRAM DEV",                "None"      , False),
"ASST COMMISSIONER FOR POLICY AND COMMUNITY RESILIENCE"               : ("COMMISSIONER"                                    , "ASSISTANT FOR POLICY AND COMMUNITY RESILIENCE",       "None"      , False),
"ASST COMMISSIONER"                                                   : ("COMMISSIONER"                                    , "ASSISTANT"                             , "None"                   , False),
"ASST DIR OF INTERGVNMENTAL RELTNS FOR THE ALBANY OFFICE"             : ("DIRECTOR OF INTERGOVERNMENTAL RELATIONS"         , "ASSISTANT"                             , "None"                   , False),
"ASST DIR OF INTERGVNMENTL RELATNS FOR THE WASHNGTN OFF"              : ("DIRECTOR OF INTERGOVERNMENTAL RELATIONS"         , "ASSISTANT"                             , "None"                   , False),
"ASST DIRECTOR OF INTERGOVERMENTAL REL FOR CITY LEGIST AFFS"          : ("DIRECTOR OF INTERGOVERNMENTAL RELATIONS"         , "ASSISTANT"                             , "None"                   , False),
"ASST PB HLTH ADV"                                                    : ("PUBLIC HEALTH ADVOCATE"                          , "ASSISTANT"                             , "None"                   , False),
"ASST PROJECT PLANNER"                                                : ("PROJECT PLANNER"                                 , "ASSISTANT"                             , "None"                   , False),
"ATTENDING DENTIST"                                                   : ("DENTIST"                                         , "ATTENDING"                             , "None"                   , False),
"ATTENDING PHYSICIAN"                                                 : ("PHYSICIAN"                                       , "ATTENDING"                             , "None"                   , False),
"ATTORNEY AT LAW"                                                     : ("ATTORNEY"                                        , "None"                                  , "None"                   , False),
"ATTORNEY-CAMPAIGN FIN BOARD"                                         : ("ATTORNEY"                                        , "CAMPAIGN FINANCE BOARD"                , "None"                   , False),
"AUDITOR"                                                             : ("AUDITOR"                                         , "None"                                  , "None"                   , False),
"AUTO BODY WORKER"                                                    : ("AUTO BODY WORKER"                                , "None"                                  , "None"                   , False),
"AUTO MACHINIST"                                                      : ("AUTO MACHINIST"                                  , "None"                                  , "None"                   , False),
"AUTO MECHANIC"                                                       : ("AUTO MECHANIC"                                   , "None"                                  , "None"                   , False),
"AUTOMOTIVE SERVICE WORKER"                                           : ("AUTOMOTIVE SERVICE WORKER"                       , "None"                                  , "None"                   , False),
"AUTOMOTIVE SPECIALIST"                                               : ("AUTOMOTIVE SPECIALIST"                           , "None"                                  , "None"                   , False),
"BAKER"                                                               : ("BAKER"                                           , "None"                                  , "None"                   , False),
"BATTALION CHIEF"                                                     : ("BATTALION CHIEF"                                 , "None"                                  , "None"                   , True),
"BIO-MEDICAL EQUIPMENT TECHNICIAN"                                    : ("BIO-MEDICAL EQUIPMENT TECHNICIAN"                , "None"                                  , "None"                   , False),
"BLACKSMITH"                                                          : ("BLACKSMITH"                                      , "None"                                  , "None"                   , False),
"BLACKSMITH'S HELPER"                                                 : ("BLACKSMITH"                                      , "HELPER"                                , "None"                   , False),
"BOILER MAKER"                                                        : ("BOILER MAKER"                                    , "None"                                  , "None"                   , False),
"BOOKBINDER"                                                          : ("BOOKBINDER"                                      , "None"                                  , "None"                   , False),
"BOOKKEEPER"                                                          : ("BOOKKEEPER"                                      , "None"                                  , "None"                   , False),
"BOROUGH COMMISSIONER"                                                : ("COMMISSIONER"                                    , "BOROUGH"                               , "None"                   , True),
"BOROUGH DIRECTOR OF RECREATION"                                      : ("DIRECTOR OF RECREATION"                          , "BOROUGH"                               , "None"                   , True),
"BOROUGH PRESIDENT"                                                   : ("BOROUGH PRESIDENT"                               , "None"                                  , "None"                   , True),
"BRICKLAYER"                                                          : ("BRICKLAYER"                                      , "None"                                  , "None"                   , False),
"BRIDGE OPERATOR"                                                     : ("BRIDGE OPERATOR"                                 , "None"                                  , "None"                   , False),
"BRIDGE PAINTER"                                                      : ("BRIDGE PAINTER"                                  , "None"                                  , "None"                   , False),
"BRIDGE REPAIRER AND RIVETER"                                         : ("BRIDGE REPAIRER & RIVETER"                       , "None"                                  , "None"                   , False),

"BUDGET ANALYST"                                                      : ("BUDGET ANALYST"                                  , "None"                                  , "None"                   , False),
"BUSINESS DATA AND REPORTING ANALYST"                                 : ("DATA AND REPORTING ANALYST"                      , "None"                                  , "None"                   , False),
"BUSINESS PROMOTION COORDINATOR"                                      : ("BUSINESS PROMOTION COORDINATOR"                  , "None"                                  , "None"                   , False),
"CALENDAR ASSISTANT"                                                  : ("CALENDAR ASSISTANT"                              , "None"                                  , "None"                   , False),
"CALL CENTER REPRESENTATIVE"                                          : ("CALL CENTER REPRESENTATIVE"                      , "None"                                  , "None"                   , False),
"CAMPUS PEACE OFFICER"                                                : ("CAMPUS PEACE OFFICER"                            , "None"                                  , "None"                   , False),
"CAMPUS PUBLIC SAFETY SERGEANT"                                       : ("CAMPUS PUBLIC SAFETY"                            , "None"                                  , "SERGEANT"               , False),
"CAMPUS SECURITY ASSISTANT"                                           : ("CAMPUS SECURITY"                                 , "ASSISTANT"                             , "None"                   , False),
"CAPT DET CHIEF OF DEPARTMENT"                                        : ("CHIEF"                                           , "None"                                  , "CAPTAIN"                , True),
"CAPT DET CHIEF OF INTERNAL AFFAIRS"                                  : ("CHIEF OF INTERNAL AFFAIRS"                       , "None"                                  , "CAPTAIN"                , True),
"CAPT DET CHIEF OF THE HOUSING BUREAU"                                : ("CHIEF OF THE HOUSING BUREAU"                     , "None"                                  , "CAPTAIN"                , True),
"CAPT DET CHIEF OF THE TRANSIT BUREAU"                                : ("CHIEF OF THE TRANSIT BUREAU"                     , "None"                                  , "CAPTAIN"                , True),

"CAPT-DET CHF SPEC OPERATIONS-M"                                      : ("CAPTAIN"                                         , "DET CHF SPEC OPERATIONS"               , "None"                   , True),
"CAPTAIN CHIEF OF COUNTERTERRORISM"                                   : ("CAPTAIN"                                         , "CHIEF OF COUNTERTERRORISM"             , "None"                   , True),
"CAPTAIN D/A CHIEF OF MANAGEMENT ANALYSIS AND PLANNING"               : ("CAPTAIN"                                         , "CHIEF OF MANAGEMENT ANALYSIS AND PLANNING", "None"                   , True),
"CAPTAIN D/A DEPUTY CHIEF INSPECTOR"                                  : ("CAPTAIN"                                         , "D/A DEPUTY CHIEF INSPECTOR"            , "None"                   , True),
"CAPTAIN D/A DEPUTY INSPECTOR"                                        : ("CAPTAIN"                                         , "D/A DEPUTY"                            , "None"                   , True),
"CAPTAIN D/A INSPECTOR"                                               : ("CAPTAIN"                                         , "D/A INSPECTOR"                         , "None"                   , True),
"CAPTAIN DETAILED AS ASSISTANT CHIEF INSPECTOR"                       : ("CAPTAIN"                                         , "ASSISTANT CHIEF INSPECTOR"             , "None"                   , False),
"CAPTAIN DETAILED AS CHIEF OF CRIME CONTROL STRATEGIES"               : ("CAPTAIN"                                         , "CHIEF OF CRIME CONTROL STRATEGIES"     , "None"                   , True),
"CAPTAIN DETAILED AS CHIEF OF DETECTIVES"                             : ("CAPTAIN"                                         , "CHIEF OF DETECTIVES"                   , "None"                   , True),
"CAPTAIN DETAILED AS CHIEF OF ORGANIZED CRIME CONTROL"                : ("CAPTAIN"                                         , "CHIEF OF ORGANIZED CRIME CONTROL"      , "None"                   , True),
"CAPTAIN DETAILED AS CHIEF OF PATROL"                                 : ("CAPTAIN"                                         , "CHIEF OF PATROL"                       , "None"                   , True),
"CAPTAIN DETAILED AS CHIEF OF PERSONNEL"                              : ("CAPTAIN"                                         , "CHIEF OF PERSONNEL"                    , "None"                   , True),
"CAPTAIN"                                                             : ("CAPTAIN"                                         , "None"                                  , "None"                   , True),
"CAPTAIN-CHIEF OF COMMUNITY AFFAIRS"                                  : ("CAPTAIN"                                         , "CHIEF OF COMMUNITY AFFAIRS"            , "None"                   , True),
"CAPTAIN-CHIEF OF INTELLIGENCE"                                       : ("INTELLIGENCE OFFICER"                            , "CAPTAIN"                               , "CHIEF"                  , True),
"CAPTAIN-CHIEF OF STAFF"                                              : ("CAPTAIN"                                         , "CHIEF OF STAFF"                        , "None"                   , True),
"CAPTAIN-MANAGERIAL DETAILS"                                          : ("CAPTAIN"                                         , "DETAILS"                               , "None"                   , True),
"CAPTAIN: CHIEF OF TRANSPORTATION BUREAU"                             : ("CAPTAIN"                                         , "CHIEF OF TRANSPORTATION BUREAU"        , "None"                   , True),
"CARETAKER"                                                           : ("CARETAKER"                                       , "None"                                  , "None"                   , False),
"CARPENTER"                                                           : ("CARPENTER"                                       , "None"                                  , "None"                   , False),
"CARRIAGE UPHOLSTERER"                                                : ("CARRIAGE UPHOLSTERER"                            , "None"                                  , "None"                   , False),
"CASE - MANAGEMENT NURSE"                                             : ("NURSE"                                           , "CASE-MANAGEMENT"                       , "None"                   , False),
"CASE MANAGEMENT NURSE"                                               : ("NURSE"                                           , "CASE MANAGEMENT"                       , "None"                   , False),
"CASE MGMT NURSE"                                                     : ("NURSE"                                           , "CASE MGMT"                             , "None"                   , False),
"CASEWORKER"                                                          : ("CASEWORKER"                                      , "None"                                  , "None"                   , False),
"CASHIER"                                                             : ("CASHIER"                                         , "None"                                  , "None"                   , False),
"CEMENT MASON"                                                        : ("CEMENT MASON"                                    , "None"                                  , "None"                   , False),
"CERTIFIED APPLICATIONS DEVELOPER"                                    : ("APPLICATIONS DEVELOPER"                          , "CERTIFIED"                             , "None"                   , False),
"CERTIFIED DATABASE ADMINISTRATOR"                                    : ("DATABASE ADMINISTRATOR"                          , "CERTIFIED"                             , "None"                   , False),
"CERTIFIED IT ADMINISTRATOR"                                          : ("IT ADMINISTRATOR"                                , "CERTIFIED"                             , "None"                   , False),
"CERTIFIED IT DEVELOPER"                                              : ("IT DEVELOPER"                                    , "CERTIFIED"                             , "None"                   , False),
"CERTIFIED LOCAL AREA NETWORK ADMINISTRATOR"                          : ("LOCAL AREA NETWORK ADMINISTRATOR"                , "CERTIFIED"                             , "None"                   , False),
"CERTIFIED WIDE AREA NETWORK ADMINISTRATOR"                           : ("WIDE AREA NETWORK ADMINISTRATOR"                 , "CERTIFIED"                             , "None"                   , False),
"CHAIR"                                                               : ("CHAIR"                                           , "None"                                  , "None"                   , True),
"CHAIR-LANDMARKS PRESERVATION COMMISSION"                             : ("LANDMARKS PRESERVATION COMMISSION"               , "CHAIR"                                 , "None"                   , True),
"CHAIRMAN COMMISSION ON HUMAN RIGHTS"                                 : ("COMMISSION ON HUMAN RIGHTS"                      , "CHAIRMAN"                              , "None"                   , True),
"CHAIRMAN"                                                            : ("CHAIRMAN"                                        , "None"                                  , "None"                   , True),
"CHANCELLOR"                                                          : ("CHANCELLOR"                                      , "None"                                  , "None"                   , True),
"CHAPLAIN"                                                            : ("CHAPLAIN"                                        , "None"                                  , "None"                   , False),
"CHAUFFER ATTENDANT"                                                  : ("CHAUFFEUR"                                       , "ATTENDANT OF"                          , "None"                   , False),
"CHAUFFEUR ATTENDANT"                                                 : ("ATTENDANT"                                       , "CHAUFFEUR"                             , "None"                   , False),
"CHAUFFEUR-ATTENDANT"                                                 : ("ATTENDANT"                                       , "CHAUFFEUR"                             , "None"                   , False),
"CHEMICAL ENGINEER"                                                   : ("CHEMICAL ENGINEER"                               , "None"                                  , "None"                   , False),
"CHEMICAL ENGINEERING INTERN"                                         : ("CHEMICAL ENGINEERING INTERN"                     , "None"                                  , "None"                   , False),
"CHIEF ACTUARY"                                                       : ("ACTUARY"                                         , "None"                                  , "CHIEF"                  , True),
"CHIEF ADMINISTRATIVE LAW JUDGE"                                      : ("ADMINISTRATIVE LAW JUDGE"                        , "CHIEF"                                 , "None"                   , True),
"CHIEF ADMINISTRATIVE SUPT CAMPUS B/G"                                : ("ADMINISTRATIVE SUPERINTENDENT"                   , "CAMPUS B/G"                            , "CHIEF"                  , True),
"CHIEF ADMINISTRATOR OF IMPARTIAL HEARING OFFICE"                     : ("IMPARTIAL HEARING OFFICE ADMINISTRATOR"          , "CHIEF"                                 , "None"                   , True),
"CHIEF ADMINISTRATOR OF IMPARTIAL HEARINGS"                           : ("ADMINISTRATOR"                                   , "CHIEF OF IMPARTIAL HEARINGS"           , "None"                   , True),
"CHIEF ADMINISTRATOR SUPT CAMPUS B/G"                                 : ("CHIEF ADMINISTRATOR"                             , "SUPT CAMPUS B/G"                       , "None"                   , True),
"CHIEF ASSET MANAGEMENT OFFICER"                                      : ("ASSET MANAGEMENT OFFICER"                        , "CHIEF"                                 , "None"                   , True),
"CHIEF CARETAKER"                                                     : ("CARETAKER"                                       , "None"                                  , "CHIEF"                  , True),
"CHIEF CITY MEDICAL EXAMINER"                                         : ("CITY MEDICAL EXAMINER"                           , "CHIEF"                                 , "None"                   , True),
"CHIEF CLERK"                                                         : ("CLERK"                                           , "None"                                  , "CHIEF"                  , True),
"CHIEF COLLEGE LAB TECHNICIAN"                                        : ("COLLEGE LAB TECHNICIAN"                          , "None"                                  , "CHIEF"                  , True),
"CHIEF CONSULTANT"                                                    : ("CONSULTANT"                                      , "None"                                  , "CHIEF"                  , True),
"CHIEF DOCKMASTER"                                                    : ("DOCKMASTER"                                      , "None"                                  , "CHIEF"                  , True),
"CHIEF ENGINEER"                                                      : ("ENGINEER"                                        , "None"                                  , "CHIEF"                  , True),
"CHIEF FIRE MARSHAL"                                                  : ("FIRE MARSHAL"                                    , "None"                                  , "CHIEF"                  , True),
"CHIEF INFORMATION TECHNOLOGY OFFICER"                                : ("INFORMATION TECHNOLOGY OFFICER"                  , "CHIEF"                                 , "None"                   , True),
"CHIEF INVESTIGATOR"                                                  : ("INVESTIGATOR"                                    , "CHIEF"                                 , "None"                   , True),
"CHIEF LIFEGUARD"                                                     : ("LIFEGUARD"                                       , "None"                                  , "CHIEF"                  , True),
"CHIEF MARINE ENGINEER"                                               : ("MARINE ENGINEER"                                 , "None"                                  , "CHIEF"                  , True),
"CHIEF OF DEPARTMENT"                                                 : ("DEPARTMENT HEAD"                                 , "None"                                  , "CHIEF"                  , True),
"CHIEF OF HOUSING COMMUNITY ACTIVITIES"                               : ("HOUSING COMMUNITY ACTIVITIES"                    , "None"                                  , "CHIEF"                  , True),
"CHIEF OF STAFF"                                                      : ("CHIEF OF STAFF"                                  , "None"                                  , "None"                   , True),
"CHIEF OFFICE ASSISTANT"                                              : ("OFFICE ASSISTANT"                                , "CHIEF"                                 , "None"                   , True),
"CHIEF RACKETS INVESTIGATOR"                                          : ("INVESTIGATOR"                                    , "CHIEF"                                 , "None"                   , True),
"CHIEF RACKETS INVESTIGATOR"                                          : ("INVESTIGATOR"                                    , "RACKETS"                               , "CHIEF"                  , True),
"CHIEF REVIEW ASSESSOR"                                               : ("REVIEW ASSESSOR"                                 , "None"                                  , "CHIEF"                  , True),
"CHIEF SCHOOL BUSINESS EXECUTIVE"                                     : ("SCHOOL BUSINESS EXECUTIVE"                       , "CHIEF"                                 , "None"                   , True),
"CHIEF, PUBLICATIONS & REPORTS"                                       : ("PUBLICATIONS & REPORTS"                          , "CHIEF"                                 , "None"                   , True),
"CHILD AND FAMILY SPECIALIST"                                         : ("CHILD AND FAMILY SPECIALIST"                     , "None"                                  , "None"                   , False),
"CHILD PROTECTIVE SPECIALIST SUPERVISOR"                              : ("CHILD PROTECTIVE SPECIALIST"                     , "None"                                  , "None"                   , True),
"CHILD PROTECTIVE SPECIALIST"                                         : ("CHILD PROTECTIVE SPECIALIST"                     , "None"                                  , "None"                   , False),
"CHILD WELFARE SPECIALIST SUPERVISOR"                                 : ("CHILD WELFARE SPECIALIST"                        , "None"                                  , "None"                   , True),
"CHILD WELFARE SPECIALIST"                                            : ("CHILD WELFARE SPECIALIST"                        , "None"                                  , "None"                   , False),
"CHILDREN'S COUNSELOR"                                                : ("COUNSELOR"                                       , "CHILDREN'S"                            , "None"                   , False),
"CITY ASSESSOR"                                                       : ("CITY ASSESSOR"                                   , "None"                                  , "None"                   , False),
"CITY ATTENDANT"                                                      : ("CITY ATTENDANT"                                  , "None"                                  , "None"                   , False),
"CITY CLERK & CLERK OF COUNCIL"                                       : ("CITY CLERK"                                      , "None"                                  , "None"                   , False),
"CITY CLINICIAN"                                                      : ("CLINICIAN"                                       , "CITY"                                  , "None"                   , False),
"CITY COLLECTOR"                                                      : ("CITY COLLECTOR"                                  , "None"                                  , "None"                   , False),
"CITY CUSTODIAL ASSISTANT"                                            : ("CUSTODIAL ASSISTANT"                             , "CITY"                                  , "None"                   , False),
"CITY DEBRIS REMOVER"                                                 : ("CITY DEBRIS REMOVER"                             , "None"                                  , "None"                   , False),
"CITY DENTIST"                                                        : ("DENTIST"                                         , "CITY"                                  , "None"                   , False),
"CITY DEPUTY MEDICAL DIRECTOR"                                        : ("MEDICAL DIRECTOR"                                , "DEPUTY"                                , "None"                   , False),
"CITY ELEVATOR OPERATOR"                                              : ("ELEVATOR OPERATOR"                               , "CITY"                                  , "None"                   , False),
"CITY LABORER"                                                        : ("CITY LABORER"                                    , "None"                                  , "None"                   , False),
"CITY MEDICAL DIRECTOR"                                               : ("MEDICAL DIRECTOR"                                , "CITY"                                  , "None"                   , True),
"CITY MEDICAL EXAMINER"                                               : ("MEDICAL EXAMINER"                                , "CITY"                                  , "None"                   , False),
"CITY MEDICAL SPECIALIST"                                             : ("MEDICAL SPECIALIST"                              , "CITY"                                  , "None"                   , False),
"CITY MORTUARY TECHNICIAN"                                            : ("MORTUARY TECHNICIAN"                             , "CITY"                                  , "None"                   , False),
"CITY PARK WORKER"                                                    : ("CITY PARK WORKER"                                , "None"                                  , "None"                   , False),
"CITY PARKING EQUIPMENT SERVICE WORKER"                               : ("CITY PARKING EQUIPMENT SERVICE WORKER"           , "None"                                  , "None"                   , False),
"CITY PEST CONTROL AIDE"                                              : ("CITY PEST CONTROL AIDE"                          , "None"                                  , "None"                   , False),
"CITY PLANNER"                                                        : ("CITY PLANNER"                                    , "None"                                  , "None"                   , False),
"CITY PLANNING TECHNICIAN"                                            : ("CITY PLANNING TECHNICIAN"                        , "None"                                  , "None"                   , False),
"CITY REGISTER"                                                       : ("CITY REGISTER"                                   , "None"                                  , "None"                   , False),
"CITY RESEARCH SCIENTIST"                                             : ("RESEARCH SCIENTIST"                              , "CITY"                                  , "None"                   , False),
"CITY SEASONAL AIDE"                                                  : ("AIDE"                                            , "CITY SEASONAL"                         , "None"                   , False),
"CITY SECURITY AIDE"                                                  : ("SECURITY AIDE"                                   , "CITY"                                  , "None"                   , False),
"CITY SERVICE AIDE"                                                   : ("CITY SERVICE AIDE"                               , "None"                                  , "None"                   , False),
"CITY TAX AUDITOR"                                                    : ("TAX AUDITOR"                                     , "CITY"                                  , "None"                   , False),
"CITY VETERINARIAN"                                                   : ("VETERINARIAN"                                    , "CITY"                                  , "None"                   , False),
"CIVIL ENGINEER"                                                      : ("CIVIL ENGINEER"                                  , "None"                                  , "None"                   , False),
"CIVIL ENGINEERING INTERN"                                            : ("CIVIL ENGINEER"                                  , "INTERN"                                , "None"                   , False),
"CIVILIANIZATION MANAGER-PD"                                          : ("CIVILIANIZATION MANAGER"                         , "None"                                  , "None"                   , True),
"CLAIM SPECIALIST"                                                    : ("CLAIM SPECIALIST"                                , "None"                                  , "None"                   , False),
"CLERICAL AIDE"                                                       : ("CLERICAL AIDE"                                   , "None"                                  , "None"                   , False),
"CLERICAL ASSOCIATE MOST MAYORAL AG"                                  : ("CLERICAL ASSOCIATE"                              , "MOST MAYORAL AG"                       , "None"                   , False),
"CLERICAL ASSOCIATE"                                                  : ("CLERICAL ASSOCIATE"                              , "None"                                  , "None"                   , False),
"CLERK TO THE BOARD"                                                  : ("BOARD"                                           , "CLERK OF"                              , "None"                   , False),
"CLIMBER & PRUNER"                                                    : ("CLIMBER & PRUNER"                                , "None"                                  , "None"                   , False),
"CLINICAL PROFESSOR"                                                  : ("PROFESSOR"                                       , "CLINICAL"                              , "None"                   , False),
"CLIP INSTRUCTOR"                                                     : ("INSTRUCTOR"                                      , "CLIP"                                  , "None"                   , False),
"CLOCK REPAIRER"                                                      : ("CLOCK REPAIRER"                                  , "None"                                  , "None"                   , False),
"COACH"                                                               : ("COACH"                                           , "None"                                  , "None"                   , False),
"COLLEGE ACCOUNTANT"                                                  : ("ACCOUNTANT"                                      , "COLLEGE"                               , "None"                   , False),
"COLLEGE ACCOUNTING ASSISTANT"                                        : ("ACCOUNTING ASSISTANT"                            , "COLLEGE"                               , "None"                   , False),
"COLLEGE AIDE - ASSIGNMENT LEVELS II AND III"                         : ("COLLEGE AIDE"                                    , "ASSIGNMENT LEVELS"                     , "LEVELS II AND III"      , False),
"COLLEGE AIDE"                                                        : ("COLLEGE AIDE"                                    , "None"                                  , "None"                   , False),
"COLLEGE ASSISTANT"                                                   : ("COLLEGE ASSISTANT"                               , "None"                                  , "None"                   , False),
"COLLEGE GRAPH DESIGNER"                                              : ("GRAPH DESIGNER"                                  , "COLLEGE"                               , "None"                   , False),
"COLLEGE INTERIOR DESIGNER"                                           : ("INTERIOR DESIGNER"                               , "COLLEGE"                               , "None"                   , False),
"COLLEGE LAB TECHNICIAN"                                              : ("LAB TECHNICIAN"                                  , "COLLEGE"                               , "None"                   , False),
"COLLEGE PRINT SHOP ASSISTANT"                                        : ("COLLEGE PRINT SHOP ASSISTANT"                    , "None"                                  , "None"                   , False),
"COLLEGE PRINT SHOP ASSOCIATE"                                        : ("COLLEGE PRINT SHOP ASSOCIATE"                    , "None"                                  , "None"                   , False),
"COLLEGE PRINT SHOP COORDINATOR"                                      : ("COLLEGE PRINT SHOP COORDINATOR"                  , "None"                                  , "None"                   , False),
"COLLEGE SECURITY DIRECTOR"                                           : ("SECURITY DIRECTOR"                               , "COLLEGE"                               , "None"                   , True),
"COLLEGE SECURITY SPECIALIST"                                         : ("SECURITY SPECIALIST"                             , "COLLEGE"                               , "None"                   , False),
"COMMISSARY MANAGER"                                                  : ("COMMISSARY MANAGER"                              , "None"                                  , "None"                   , True),
"COMMISSIONER DESIGNATED AS VICE CHAIRMAN"                            : ("COMMISSIONER"                                    , "VICE CHAIRMAN"                         , "None"                   , True),
"COMMISSIONER OF BUILDINGS"                                           : ("COMMISSIONER OF BUILDINGS"                       , "None"                                  , "None"                   , True),
"COMMISSIONER OF BUSINESS SERVICES"                                   : ("COMMISSIONER"                                    , "OF BUSINESS SERVICES"                  , "None"                   , True),
"COMMISSIONER OF CHILDREN'S SERVICES"                                 : ("COMMISSIONER OF CHILDREN'S SERVICES"             , "None"                                  , "None"                   , True),
"COMMISSIONER OF CITYWIDE ADMINISTRATIVE SERVICES"                    : ("COMMISSIONER OF CITYWIDE ADMINISTRATIVE SERVICES", "None"                                  , "None"                   , True),
"COMMISSIONER OF COMMUNITY DEVELOPMENT"                               : ("COMMISSIONER OF COMMUNITY DEVELOPMENT"           , "None"                                  , "None"                   , True),
"COMMISSIONER OF CULTURAL AFFAIRS"                                    : ("COMMISSIONER OF CULTURAL AFFAIRS"                , "None"                                  , "None"                   , True),
"COMMISSIONER OF DEPT OF INFO TECHNOLOGY & TELECOMMUNICATIONS"        : ("COMMISSIONER"                                    , "OF DEPT OF INFO TECHNOLOGY & TELECOMMUNICATIONS", "None"                   , True),
"COMMISSIONER OF DESIGN & CONSTRUCTION"                               : ("COMMISSIONER OF DESIGN & CONSTRUCTION"           , "None"                                  , "None"                   , True),
"COMMISSIONER OF EMERGENCY MANAGEMENT"                                : ("EMERGENCY MANAGEMENT COMMISSIONER"               , "None"                                  , "None"                   , True),
"COMMISSIONER OF ENVIRONMENTAL PROTECTION"                            : ("ENVIRONMENTAL PROTECTION COMMISSIONER"           , "None"                                  , "None"                   , True),
"COMMISSIONER OF FINANCE"                                             : ("COMMISSIONER OF FINANCE"                         , "None"                                  , "None"                   , True),
"COMMISSIONER OF HEALTH"                                              : ("COMMISSIONER OF HEALTH"                          , "None"                                  , "None"                   , True),
"COMMISSIONER OF HOMELESS SERVICES"                                   : ("COMMISSIONER OF HOMELESS SERVICES"               , "None"                                  , "None"                   , True),
"COMMISSIONER OF HOUSING PRESERVATION & DEVELOPMENT"                  : ("COMMISSIONER OF HOUSING PRESERVATION & DEVELOPMENT", "None"                                  , "None"                   , True),
"COMMISSIONER OF LABOR RELATIONS"                                     : ("COMMISSIONER OF LABOR RELATIONS"                 , "None"                                  , "None"                   , True),
"COMMISSIONER OF PARKS & RECREATION"                                  : ("COMMISSIONER OF PARKS & RECREATION"              , "None"                                  , "None"                   , True),
"COMMISSIONER OF SANITATION"                                          : ("COMMISSIONER OF SANITATION"                      , "None"                                  , "None"                   , True),
"COMMISSIONER OF TRANSPORTATION"                                      : ("COMMISSIONER OF TRANSPORTATION"                  , "None"                                  , "None"                   , True),
"COMMISSIONER"                                                        : ("COMMISSIONER"                                    , "None"                                  , "None"                   , False),
"COMMISSIONER, UN & CONSULAR CORPS DIPLOMATIC RELATIONS"              : ("COMMISSIONER"                                    , "UN & CONSULAR CORPS DIPLOMATIC RELATIONS", "None"                   , True),
"COMMUNICATION ASSISTANT"                                             : ("COMMUNICATION ASSISTANT"                         , "None"                                  , "None"                   , False),
"COMMUNICATION ELECTRICIAN"                                           : ("ELECTRICIAN"                                     , "COMMUNICATION"                         , "None"                   , False),
"COMMUNITY ASSISTANT"                                                 : ("COMMUNITY ASSISTANT"                             , "None"                                  , "None"                   , False),
"COMMUNITY ASSOCIATE"                                                 : ("COMMUNITY ASSOCIATE"                             , "None"                                  , "None"                   , False),
"COMMUNITY COORDINATOR"                                               : ("COMMUNITY COORDINATOR"                           , "None"                                  , "None"                   , False),
"COMMUNITY LIAISON WORKER"                                            : ("COMMUNITY LIAISON WORKER"                        , "None"                                  , "None"                   , False),
"COMMUNITY PLANNING BOARD COORDINATOR"                                : ("COMMUNITY PLANNING BOARD COORDINATOR"            , "None"                                  , "None"                   , False),
"COMMUNITY SERVICE AIDE"                                              : ("COMMUNITY SERVICE AIDE"                          , "None"                                  , "None"                   , False),
"COMMUNITY SUPERTINDENT"                                              : ("SUPERTINDENT"                                    , "COMMUNITY"                             , "None"                   , True),
"COMPOSITOR"                                                          : ("COMPOSITOR"                                      , "None"                                  , "None"                   , False),
"COMPTROLLER"                                                         : ("COMPTROLLER"                                     , "None"                                  , "None"                   , False),
"COMPUTER AIDE"                                                       : ("COMPUTER AIDE"                                   , "None"                                  , "None"                   , False),
"COMPUTER AIDE-NON-SPVR"                                              : ("COMPUTER AIDE"                                   , "None"                                  , "None"                   , False),
"COMPUTER ASSOCIATE"                                                  : ("COMPUTER ASSOCIATE"                              , "None"                                  , "None"                   , False),
"COMPUTER OPERATIONS MANAGER"                                         : ("COMPUTER OPERATIONS MANAGER"                     , "None"                                  , "None"                   , True),
"COMPUTER OPERATOR MANAGER"                                           : ("COMPUTER OPERATOR"                               , "None"                                  , "None"                   , True),
"COMPUTER OPERATOR"                                                   : ("COMPUTER OPERATOR"                               , "None"                                  , "None"                   , False),
"COMPUTER PROGRAMMER ANALYST TRAINEE"                                 : ("COMPUTER PROGRAMMER ANALYST"                     , "TRAINEE"                               , "None"                   , False),
"COMPUTER PROGRAMMER ANALYST"                                         : ("COMPUTER PROGRAMMER ANALYST"                     , "None"                                  , "None"                   , False),
"COMPUTER SERVICE TECHNICIAN"                                         : ("COMPUTER SERVICE TECHNICIAN"                     , "None"                                  , "None"                   , False),
"COMPUTER SPECIALIST"                                                 : ("COMPUTER SPECIALIST"                             , "None"                                  , "None"                   , False),
"COMPUTER SYSTEM MANAGER"                                             : ("COMPUTER SYSTEM MANAGER"                         , "None"                                  , "None"                   , True),
"COMPUTER SYSTEMS MANAGER"                                            : ("COMPUTER SYSTEMS MANAGER"                        , "None"                                  , "None"                   , True),
"CONFIDENTIAL AGENCY INVESTIGATOR"                                    : ("INVESTIGATOR"                                    , "CONFIDENTIAL AGENCY"                   , "None"                   , False),
"CONFIDENTIAL ASSISTANT TO BORO PRESIDENT"                            : ("BORO PRESIDENT"                                  , "ASSISTANT TO"                          , "None"                   , False),
"CONFIDENTIAL ASSISTANT TO THE COMPTROLLER"                           : ("COMPTROLLER"                                     , "ASSISTANT TO"                          , "None"                   , False),
"CONFIDENTIAL ASST TO THE EXECUTIVE DEPUTY COMMISSIONER"              : ("EXECUTIVE DEPUTY COMMISSIONER"                   , "CONFIDENTIAL ASST TO"                  , "None"                   , False),
"CONFIDENTIAL EXAMINER"                                               : ("EXAMINER"                                        , "CONFIDENTIAL"                          , "None"                   , False),
"CONFIDENTIAL INVESTIGATOR"                                           : ("INVESTIGATOR"                                    , "CONFIDENTIAL"                          , "None"                   , False),
"CONFIDENTIAL SECRETARY OF ADMINISTRATIVE LAW JUDGE"                  : ("ADMINISTRATIVE LAW JUDGE"                        , "CONFIDENTIAL SECRETARY OF"             , "None"                   , False),
"CONFIDENTIAL SECRETARY TO THE DEPUTY COMMISSIONER"                   : ("DEPUTY COMMISSIONER"                             , "CONFIDENTIAL SECRETARY OF"             , "None"                   , False),
"CONFIDENTIAL SECY"                                                   : ("SECRETARY"                                       , "CONFIDENTIAL"                          , "None"                   , False),
"CONFIDENTIAL STRATEGY PLANNER"                                       : ("STRATEGY PLANNER"                                , "CONFIDENTIAL"                          , "None"                   , False),
"CONGREG CARE SPEC - ACS"                                             : ("CARE SPEC"                                       , "ACS"                                   , "None"                   , False),
"CONGREG CARE SPEC- DJJ"                                              : ("CONGREG CARE SPEC"                               , "DJJ"                                   , "None"                   , False),
"CONSTRUCTION LABORER"                                                : ("CONSTRUCTION LABORER"                            , "None"                                  , "None"                   , False),
"CONSTRUCTION PROJECT MANAGER INTERN"                                 : ("CONSTRUCTION PROJECT MANAGER"                    , "INTERN"                                , "None"                   , False),
"CONSTRUCTION PROJECT MANAGER"                                        : ("CONSTRUCTION PROJECT MANAGER"                    , "None"                                  , "None"                   , True),
"CONSULTANT PUBLIC HEALTH NURSE"                                      : ("PUBLIC HEALTH NURSE"                             , "CONSULTANT"                            , "None"                   , False),
"CONSULTANT"                                                          : ("CONSULTANT"                                      , "None"                                  , "None"                   , False),
"CONSULTING ENGINEER"                                                 : ("CONSULTING ENGINEER"                             , "None"                                  , "None"                   , False),
"CONSUMER ADVOCATE"                                                   : ("CONSUMER ADVOCATE"                               , "None"                                  , "None"                   , False),
"CONTINUING EDUCATION TEACHER"                                        : ("TEACHER"                                         , "CONTINUING EDUCATION"                  , "None"                   , False),
"CONTRACT REVIEWER"                                                   : ("CONTRACT REVIEWER"                               , "None"                                  , "None"                   , False),
"CONTRACT SPECIALIST"                                                 : ("CONTRACT SPECIALIST"                             , "None"                                  , "None"                   , False),
"CONTROLLER"                                                          : ("CONTROLLER"                                      , "None"                                  , "None"                   , False),
"COOK"                                                                : ("COOK"                                            , "None"                                  , "None"                   , False),
"COOP EDUCATION TRAINEE COMPUTER TECH AIDE/JR"                        : ("COMPUTER TECH AIDE"                              , "COOP EDUCATION TRAINEE"                , "JR"                     , False),
"COOP EDUCATION TRAINEE COMPUTER TECH AIDE/SR"                        : ("COMPUTER TECH AIDE"                              , "COOP EDUCATION TRAINEE"                , "SENIOR"                 , False),
"COOP TRAINEE TYPIST"                                                 : ("TYPIST"                                          , "COOP TRAINEE"                          , "None"                   , False),
"COOPERATIVE EDUCATION TRAINEE - ENGINEERING AIDE"                    : ("ENGINEERING AIDE"                                , "COOPERATIVE EDUCATION TRAINEE"         , "None"                   , False),
"COOPERATIVE EDUCATION TRAINEE LEGISLATIVE CLERICAL ASSISTANT"        : ("LEGISLATIVE CLERICAL ASSISTANT"                  , "COOPERATIVE EDUCATION TRAINEE"         , "None"                   , False),
"COOPERATIVE EDUCATION TRAINEE-SEASONAL PARK HELP"                    : ("PARK HELP"                                       , "COOPERATIVE EDUCATION TRAINEE"         , "None"                   , False),
"COOPERATIVE EDUCATIONAL TRAINEE"                                     : ("COOPERATIVE EDUCATIONAL TRAINEE"                 , "None"                                  , "None"                   , False),
"COORDINATING MANAGER"                                                : ("MANAGER"                                         , "COORDINATING"                          , "None"                   , True),
"COORDINATOR OF CRIMINAL JUSTICE"                                     : ("COORDINATOR"                                     , "OF CRIMINAL JUSTICE"                   , "None"                   , False),
"COORDINATOR OF VETERANS' AFFAIRS"                                    : ("VETERANS' AFFAIRS COORDINATOR"                   , "None"                                  , "None"                   , False),
"COORDINATOR, ELECTION DAY OPERATIONS"                                : ("ELECTION DAY OPERATIONS COORDINATOR"             , "None"                                  , "None"                   , False),
"COORDINATOR, VOTER REGISTRATION ACTIVITIES"                          : ("VOTER REGISTRATION ACTIVITIES"                   , "COORDINATOR"                           , "None"                   , False),
"CORPORATION COUNSEL"                                                 : ("COUNSEL"                                         , "CORPORATION"                           , "None"                   , False),
"CORRECTION ADMINISTRATIVE AIDE"                                      : ("CORRECTION ADMINISTRATIVE AIDE"                  , "None"                                  , "None"                   , False),
"CORRECTION OFFICER"                                                  : ("CORRECTION OFFICER"                              , "None"                                  , "None"                   , False),
"CORRECTIONAL COUNSELOR"                                              : ("CORRECTIONAL COUNSELOR"                          , "None"                                  , "None"                   , False),
"CORRECTIONAL STANDARDS REVIEW SPECIALIST"                            : ("CORRECTIONAL STANDARDS REVIEW SPECIALIST"        , "None"                                  , "None"                   , False),
"COUNCIL MEMBER"                                                      : ("COUNCIL MEMBER"                                  , "None"                                  , "None"                   , False),
"COUNCILMANIC AIDE"                                                   : ("COUNCILMANIC AIDE"                               , "None"                                  , "None"                   , False),
"COUNSEL TO MINORITY LEADER"                                          : ("MINORITY LEADER"                                 , "COUNSEL OF"                            , "None"                   , False),
"COUNSEL TO THE BOARD"                                                : ("BOARD"                                           , "COUNSEL OF"                            , "None"                   , False),
"COUNSEL TO THE BOROUGH PRESIDENT"                                    : ("BOROUGH PRESIDENT"                               , "COUNSEL OF"                            , "None"                   , False),
"COUNSEL TO THE CHANCELLOR"                                           : ("CHANCELLOR"                                      , "COUNSEL OF"                            , "None"                   , False),
"COUNSEL TO THE POLICE COMMISSIONER"                                  : ("POLICE COMMISSIONER"                             , "COUNSEL OF"                            , "None"                   , False),
"COUNSEL TO THE PUBLIC ADVOCATE"                                      : ("PUBLIC ADVOCATE"                                 , "COUNSEL OF"                            , "None"                   , False),
"COUNSEL"                                                             : ("COUNSEL"                                         , "None"                                  , "None"                   , False),
"COUNSELOR"                                                           : ("COUNSELOR"                                       , "None"                                  , "None"                   , False),
"COUNTY DETECTIVE"                                                    : ("DETECTIVE"                                       , "COUNTY"                                , "None"                   , False),
"CRANE OPERATOR AMPES"                                                : ("CRANE OPERATOR"                                  , "AMPES"                                 , "None"                   , False),
"CRIMINALIST ASSISTANT DIRECTOR OF LABORATORY"                        : ("DIRECTOR OF LABORATORY"                          , "ASSISTANT"                             , "None"                   , True),
"CRIMINALIST DEPUTY DIRECTOR OF LABATORY"                             : ("CRIMINALIST"                                     , "DEPUTY DIRECTOR OF LABATORY"           , "None"                   , True),
"CRIMINALIST DIRECTOR OF LABORATORY"                                  : ("CRIMINALIST"                                     , "DIRECTOR OF LABORATORY"                , "None"                   , True),
"CRIMINALIST"                                                         : ("CRIMINALIST"                                     , "None"                                  , "None"                   , False),
"CSE CHAIRPERSON"                                                     : ("CHAIRPERSON"                                     , "CSE"                                   , "None"                   , True),
"CUNY ADMINISTRATOR ASSISTANT"                                        : ("CUNY ADMINISTRATOR"                              , "ASSISTANT"                             , "None"                   , False),
"CUNY BROADCAST ASSOCIATE"                                            : ("BROADCAST ASSOCIATE"                             , "None"                                  , "None"                   , False),
"CUNY CUSTODIAL ASSISTANT"                                            : ("CUSTODIAL ASSISTANT"                             , "None"                                  , "None"                   , False),
"CUNY OFFICE ASSISTANT"                                               : ("OFFICE ASSISTANT"                                , "None"                                  , "None"                   , False),
"CUNY SECRETARIAL ASSISTANT"                                          : ("SECRETARIAL ASSISTANT"                           , "None"                                  , "None"                   , False),
"CUNY START INSTRUCTOR"                                               : ("INSTRUCTOR"                                      , "CUNY START"                            , "None"                   , False),
"CUNY TECHNICAL SUPPORT AIDE"                                         : ("TECHNICAL SUPPORT AIDE"                          , "None"                                  , "None"                   , False),
"CUSTODIAL ASSISTANT"                                                 : ("CUSTODIAL ASSISTANT"                             , "None"                                  , "None"                   , False),
"CUSTODIAL SUPERVISOR"                                                : ("CUSTODIAL SUPERVISOR"                            , "None"                                  , "None"                   , True),
"CUSTODIAN ENGINEER"                                                  : ("CUSTODIAN ENGINEER"                              , "None"                                  , "None"                   , False),
"CUSTODIAN"                                                           : ("CUSTODIAN"                                       , "None"                                  , "None"                   , False),
"CUSTOMER INFORMATION REPRESENTATIVE MA L 1549"                       : ("CUSTOMER INFORMATION REPRESENTATIVE"             , "None"                                  , "L 1549"                 , False),
"CUSTOMER INFORMATION REPRESENTATIVE"                                 : ("CUSTOMER INFORMATION REPRESENTATIVE"             , "None"                                  , "None"                   , False),
"DEAN"                                                                : ("DEAN"                                            , "None"                                  , "None"                   , True),
"DECEDENT PROPERTY AGENT"                                             : ("DECEDENT PROPERTY AGENT"                         , "None"                                  , "None"                   , False),
"DECKHAND"                                                            : ("DECKHAND"                                        , "None"                                  , "None"                   , False),
"DENTAL HYGIENIST"                                                    : ("DENTAL HYGIENIST"                                , "None"                                  , "None"                   , False),
"DENTIST"                                                             : ("DENTIST"                                         , "None"                                  , "None"                   , False),
"DEP COMMISSIONER FOR COMMUNITY RELATIONS"                            : ("COMMISSIONER"                                    , "FOR COMMUNITY RELATIONS"               , "None"                   , True),
"DEP DIR MTR EQ MNTC"                                                 : ("DIRECTOR"                                        , "DEPUTY"                                , "None"                   , True),
"DEP DIR OF AUDIT & PROG ANALYSIS"                                    : ("DIRECTOR OF AUDIT & PROGRAM ANALYSIS"            , "DEP"                                   , "None"                   , True),
"DEP DIR OF PROGRAM PLANNING"                                         : ("DIRECTOR"                                        , "DEP OF PROGRAM PLANNING"               , "None"                   , True),
"DEP DIR OF SOCIAL & COMM SERV"                                       : ("DIRECTOR"                                        , "DEP OF SOCIAL & COMM SERV"             , "None"                   , True),
"DEP DIR RESEARCH & POL DEVELP"                                       : ("DIRECTOR"                                        , "DEPUTY"                                , "None"                   , True),
"DEP DIRECTOR OF PERSONNEL & LR"                                      : ("DIRECTOR"                                        , "OF PERSONNEL & LR"                     , "None"                   , True),
"DEPARTMENT LIBRARIAN"                                                : ("LIBRARIAN"                                       , "DEPARTMENT"                            , "None"                   , False),
"DEPUTY ADMINISTRATION ASSISTANT"                                     : ("ADMINISTRATION ASSISTANT"                        , "DEPUTY"                                , "None"                   , False),
"DEPUTY ASSISTANT CHIEF OF DEPARTMENT"                                : ("CHIEF OF DEPARTMENT"                             , "DEPUTY ASSISTANT"                      , "None"                   , True),
"DEPUTY ASSISTANT COMMISSIONER"                                       : ("COMMISSIONER"                                    , "DEPUTY ASSISTANT OF"                   , "None"                   , False),
"DEPUTY ASSISTANT COUNSEL"                                            : ("ASSISTANT COUNSEL"                               , "DEPUTY"                                , "None"                   , False),
"DEPUTY ASSISTANT DIRECTOR"                                           : ("ASSISTANT DIRECTOR"                              , "DEPUTY"                                , "None"                   , False),
"DEPUTY ASSISTANT SUPERINTENDENT"                                     : ("ASSISTANT SUPERINTENDENT"                        , "DEPUTY"                                , "None"                   , False),
"DEPUTY ASST ADMIN FOR PERSONNEL ADM & LABOR REL"                     : ("ADMINISTRATOR"                                   , "DEPUTY ASST"                           , "None"                   , True),
"DEPUTY ASST COMMISSIONER FOR PROGRAM DEV & ANAL"                     : ("COMMISSIONER"                                    , "DEPUTY ASST"                           , "None"                   , True),
"DEPUTY AUDITOR GENERAL"                                              : ("AUDITOR GENERAL"                                 , "DEPUTY"                                , "None"                   , True),
"DEPUTY BOROUGH COMMISSIONER"                                         : ("BOROUGH COMMISSIONER"                            , "DEPUTY OF"                             , "None"                   , False),
"DEPUTY BOROUGH PRESIDENT"                                            : ("BOROUGH PRESIDENT"                               , "DEPUTY"                                , "None"                   , True),
"DEPUTY CHANCELLOR FINANCIAL AFFAIRS"                                 : ("CHANCELLOR"                                      , "DEPUTY"                                , "None"                   , True),
"DEPUTY CHANCELLOR"                                                   : ("CHANCELLOR"                                      , "DEPUTY OF"                             , "None"                   , True),
"DEPUTY CHIEF CITY MEDICAL EXAMINER"                                  : ("CITY MEDICAL EXAMINER"                           , "DEPUTY"                                , "None"                   , True),
"DEPUTY CHIEF CLERK"                                                  : ("CHIEF CLERK"                                     , "DEPUTY"                                , "None"                   , False),
"DEPUTY CHIEF OF OPERATIONS"                                          : ("CHIEF OF OPERATIONS"                             , "DEPUTY"                                , "None"                   , True),
"DEPUTY CHIEF OF STAFF"                                               : ("CHIEF OF STAFF"                                  , "DEPUTY"                                , "None"                   , True),
"DEPUTY CHIEF RACKETS INVESTIGATOR"                                   : ("RACKETS INVESTIGATOR"                            , "DEPUTY CHIEF"                          , "None"                   , True),
"DEPUTY CHIEF SURGEON"                                                : ("CHIEF SURGEON"                                   , "DEPUTY"                                , "None"                   , False),
"DEPUTY CHIEF"                                                        : ("CHIEF"                                           , "DEPUTY"                                , "None"                   , True),
"DEPUTY CITY CLERK"                                                   : ("CITY CLERK"                                      , "DEPUTY OF"                             , "None"                   , False),
"DEPUTY CITY SHERIFF - NON-SPVR"                                      : ("CITY SHERIFF"                                    , "DEPUTY"                                , "None"                   , False),
"DEPUTY COMMISIONER"                                                  : ("COMMISSIONER"                                    , "DEPUTY"                                , "None"                   , True),
"DEPUTY COMMISSIONER FOR ADMINISTATION"                               : ("COMMISSIONER"                                    , "DEPUTY"                                , "None"                   , True),
"DEPUTY COMMISSIONER FOR CITY PERSONNEL SERVICES"                     : ("COMMISSIONER"                                    , "DEPUTY"                                , "None"                   , True),
"DEPUTY COMMISSIONER FOR LAW ENFORCEMENT"                             : ("COMMISSIONER"                                    , "DEPUTY"                                , "None"                   , True),
"DEPUTY COMMISSIONER OF LABOR RELATIONS"                              : ("COMMISSIONER OF LABOR RELATIONS"                 , "DEPUTY"                                , "None"                   , True),
"DEPUTY COMMISSIONER"                                                 : ("COMMISSIONER"                                    , "DEPUTY"                                , "None"                   , True),
"DEPUTY COMMUNITY SUPERINTENDENT"                                     : ("COMMUNITY SUPERINTENDENT"                        , "DEPUTY OF"                             , "None"                   , False),
"DEPUTY CONTROLLER"                                                   : ("CONTROLLER"                                      , "DEPUTY OF"                             , "None"                   , False),
"DEPUTY COUNSEL INTERN"                                               : ("COUNSEL"                                         , "INTERN"                                , "DEPUTY"                 , False),
"DEPUTY COUNSEL"                                                      : ("COUNSEL"                                         , "DEPUTY OF"                             , "None"                   , False),
"DEPUTY DIRECTOR OF ADMIN WORKER'S COMP BENEFITS"                     : ("DIRECTOR"                                        , "DEPUTY OF ADMIN WORKER'S COMP BENEFITS", "None"                   , True),
"DEPUTY DIRECTOR OF ADMINISTRATION"                                   : ("DIRECTOR OF ADMINISTRATION"                      , "DEPUTY"                                , "None"                   , True),
"DEPUTY DIRECTOR OF COMMUNITY PLANNING BOARDS"                        : ("DIRECTOR OF COMMUNITY PLANNING BOARDS"           , "DEPUTY"                                , "None"                   , False),
"DEPUTY DIRECTOR OF EQUAL OPPORTUNITY"                                : ("DIRECTOR OF EQUAL OPPORTUNITY"                   , "DEPUTY"                                , "None"                   , False),
"DEPUTY DIRECTOR OF HOUSING APPLICANTS"                               : ("DIRECTOR OF HOUSING APPLICANTS"                  , "DEPUTY"                                , "None"                   , False),
"DEPUTY DIRECTOR OF LABOR RELATIONS"                                  : ("DIRECTOR OF LABOR RELATIONS"                     , "DEPUTY"                                , "None"                   , True),
"DEPUTY DIRECTOR OF MANAGEMENT & BUDGET"                              : ("DIRECTOR OF MANAGEMENT & BUDGET"                 , "DEPUTY"                                , "None"                   , True),
"DEPUTY DIRECTOR OF MANAGEMENT"                                       : ("DIRECTOR OF MANAGEMENT"                          , "DEPUTY"                                , "None"                   , True),
"DEPUTY DIRECTOR OF MEDICOLEGAL INVESTIGATIONS"                       : ("DIRECTOR OF MEDICOLEGAL INVESTIGATIONS"          , "DEPUTY"                                , "None"                   , False),
"DEPUTY DIRECTOR OF METHODS & ANALYSIS"                               : ("DIRECTOR OF METHODS & ANALYSIS"                  , "DEPUTY"                                , "None"                   , True),
"DEPUTY DIRECTOR OF MOTOR EQUIPMENT MAINTENANCE"                      : ("DIRECTOR OF MOTOR EQUIPMENT MAINTENANCE"         , "DEPUTY"                                , "None"                   , False),
"DEPUTY DIRECTOR OF PROBATION"                                        : ("DIRECTOR OF PROBATION"                           , "DEPUTY"                                , "None"                   , True),
"DEPUTY DIRECTOR OF SECURITY"                                         : ("DIRECTOR OF SECURITY"                            , "DEPUTY"                                , "None"                   , True),
"DEPUTY DIRECTOR"                                                     : ("DIRECTOR"                                        , "DEPUTY OF"                             , "None"                   , True),
"DEPUTY DIRECTOR- FINANCE"                                            : ("DIRECTOR- FINANCE"                               , "DEPUTY"                                , "None"                   , True),
"DEPUTY DIRECTOR-LEGAL DIVISION"                                      : ("DIRECTOR"                                        , "LEGAL DIVISION"                        , "DEPUTY"                 , True),
"DEPUTY EXECUTIVE DIRECTOR CAMPAIGN FINANCE BOARD"                    : ("EXECUTIVE DIRECTOR"                              , "DEPUTY"                                , "None"                   , True),
"DEPUTY EXECUTIVE DIRECTOR OF BUSINESS AND ADMINISTRATION"            : ("EXECUTIVE DIRECTOR"                              , "DEPUTY OF"                             , "None"                   , True),
"DEPUTY EXECUTIVE DIRECTOR OF FINANCIAL OPERATIONS"                   : ("EXECUTIVE DIRECTOR OF FINANCIAL OPERATIONS"      , "DEPUTY"                                , "None"                   , True),
"DEPUTY EXECUTIVE DIRECTOR"                                           : ("EXECUTIVE DIRECTOR"                              , "DEPUTY"                                , "None"                   , True),
"DEPUTY GENERAL COUNSEL"                                              : ("GENERAL COUNSEL"                                 , "DEPUTY OF"                             , "None"                   , False),
"DEPUTY INSPECTOR GENERAL"                                            : ("INSPECTOR GENERAL"                               , "DEPUTY"                                , "None"                   , True),
"DEPUTY MAYOR"                                                        : ("MAYOR"                                           , "DEPUTY"                                , "None"                   , True),
"DEPUTY OPERATION SUPPORT MANAGER"                                    : ("OPERATION SUPPORT MANAGER"                       , "DEPUTY"                                , "None"                   , True),
"DEPUTY PUBLIC ADMINISTATOR"                                          : ("PUBLIC ADMINISTATOR"                             , "DEPUTY OF"                             , "None"                   , False),
"DEPUTY REGIONAL SUPERINTENDENT"                                      : ("REGIONAL SUPERINTENDENT"                         , "DEPUTY"                                , "None"                   , False),
"DEPUTY SUPERINTENDENT"                                               : ("SUPERINTENDENT"                                  , "DEPUTY OF"                             , "None"                   , True),
"DEPUTY UNIT CHIEF"                                                   : ("UNIT CHIEF"                                      , "DEPUTY"                                , "None"                   , True),
"DETECTIVE INVESTIGATOR"                                              : ("DETECTIVE"                                       , "INVESTIGATOR"                          , "None"                   , False),
"DIETARY AIDE"                                                        : ("DIETARY AIDE"                                    , "None"                                  , "None"                   , False),
"DIETITIAN"                                                           : ("DIETITIAN"                                       , "None"                                  , "None"                   , False),
"DIR OF ADMINISTRATION, WORKER'S COMP BENEFITS"                       : ("DIRECTOR OF ADMINISTRATION"                      , "WORKER'S COMP BENEFITS"                , "None"                   , True),
"DIR OF AUDIT & PROG ANALYSIS"                                        : ("AUDIT & PROGRAM ANALYSIS"                        , "None"                                  , "DIRECTOR"               , True),
"DIR OF FISCAL AFFAIRS"                                               : ("FISCAL AFFAIRS DIRECTOR"                         , "None"                                  , "None"                   , True),
"DIR OF HEALTHCARE FACILITIES AND PROVIDER READINESS"                 : ("DIRECTOR"                                        , "HEALTHCARE FACILITIES AND PROVIDER READINESS", "None"                   , True),
"DIR OF HOME CARE SERVICES/FAMILY AND ADULT SERVICES"                 : ("DIRECTOR"                                        , "OF HOME CARE SERVICES/FAMILY AND ADULT SERVICES", "None"                   , True),
"DIR OF MATERIALS MANAGEMENT"                                         : ("MATERIALS MANAGEMENT"                            , "DIR OF"                                , "None"                   , True),
"DIR OF MGNT SYSTEM, PLANNING, RESEARCH AND EVAL"                     : ("DIRECTOR"                                        , "OF MGNT SYSTEM, PLANNING, RESEARCH AND EVAL", "None"                   , True),
"DIR OF OPERATIONS"                                                   : ("OPERATIONS DIRECTOR"                             , "None"                                  , "None"                   , True),
"DIR OF PERSONNEL & LABOR RELS"                                       : ("PERSONNEL & LABOR RELATIONS DIRECTOR"            , "None"                                  , "None"                   , True),
"DIR OF QUALITY & COST CONTROL"                                       : ("QUALITY & COST CONTROL"                          , "DIR OF"                                , "None"                   , True),
"DIR POLICY ECONOM RESRCH"                                            : ("DIRECTOR"                                        , "POLICY ECONOM RESRCH"                  , "None"                   , True),
"DIRECTOR BOROUGH PRESIDENT'S OFFICE OF ADMINISTRATION QUEENS"        : ("DIRECTOR"                                        , "BOROUGH PRESIDENT'S OFFICE OF ADMINISTRATION", "QUEENS"                 , True),
"DIRECTOR CORRESPONDENCE SERVICES"                                    : ("DIRECTOR"                                        , "CORRESPONDENCE SERVICES"               , "None"                   , True),
"DIRECTOR EMPLOYEE ASSISTANCE PROGRAM"                                : ("EMPLOYEE ASSISTANCE PROGRAM DIRECTOR"            , "None"                                  , "None"                   , True),
"DIRECTOR EMPLOYEE MANAGEMENT DIVISION"                               : ("DIRECTOR"                                        , "EMPLOYEE MANAGEMENT"                   , "None"                   , True),
"DIRECTOR FOR EQUAL EMPLOYMENT OPPORTUNITY"                           : ("DIRECTOR"                                        , "FOR EQUAL EMPLOYMENT OPPORTUNITY"      , "None"                   , True),
"DIRECTOR LEGAL DIVISION/GEN COUNSEL& SPEC COUNSEL"                   : ("COUNSEL"                                         , "DIRECTOR LEGAL DIVISION/GEN"           , "None"                   , True),
"DIRECTOR MANAGEMENT INFORMATION SYSTEMS"                             : ("MANAGEMENT INFORMATION SYSTEMS"                  , "DIRECTOR OF"                           , "None"                   , True),
"DIRECTOR NYC LOFT BOARD"                                             : ("DIRECTOR"                                        , "NYC LOFT BOARD"                        , "None"                   , True),
"DIRECTOR OF ADMIN"                                                   : ("ADMIN"                                           , "DIRECTOR OF"                           , "None"                   , True),
"DIRECTOR OF ADMINISTRATION"                                          : ("DIRECTOR OF ADMINISTRATION"                      , "None"                                  , "None"                   , True),
"DIRECTOR OF AUDIT AND INVESTIGATION"                                 : ("DIRECTOR OF AUDIT AND INVESTIGATION"             , "None"                                  , "None"                   , True),
"DIRECTOR OF AUDITS"                                                  : ("DIRECTOR OF AUDITS"                              , "None"                                  , "None"                   , True),
"DIRECTOR OF BIODETECTION AND RESPONSE"                               : ("DIRECTOR"                                        , "BIODETECTION AND RESPONSE"             , "None"                   , True),
"DIRECTOR OF BUILDING MANAGEMENT"                                     : ("BUILDING MANAGEMENT"                             , "DIRECTOR OF"                           , "None"                   , True),
"DIRECTOR OF BUREAU OF CONSUMER SERVICES"                             : ("DIRECTOR"                                        , "OF BUREAU OF CONSUMER SERVICES"        , "None"                   , True),
"DIRECTOR OF COMMUNICATIONS"                                          : ("DIRECTOR OF COMMUNICATIONS"                      , "None"                                  , "None"                   , True),
"DIRECTOR OF COMMUNITY ASSISTANCE UNIT"                               : ("DIRECTOR"                                        , "OF COMMUNITY ASSISTANCE UNIT"          , "None"                   , True),
"DIRECTOR OF COMMUNITY INVOLVEMENT"                                   : ("DIRECTOR"                                        , "OF COMMUNITY INVOLVEMENT"              , "None"                   , True),
"DIRECTOR OF COMMUNITY PARTICIPATION PROGRAMS"                        : ("DIRECTOR"                                        , "COMMUNITY PARTICIPATION PROGRAMS"      , "None"                   , True),
"DIRECTOR OF COMMUNITY PLANNING BOARDS - MGL ASSIGN"                  : ("DIRECTOR OF COMMUNITY PLANNING BOARDS"           , "MGL ASSIGN"                            , "None"                   , True),
"DIRECTOR OF COMMUNITY PLANNING BOARDS"                               : ("DIRECTOR"                                        , "OF COMMUNITY PLANNING BOARDS"          , "None"                   , True),
"DIRECTOR OF COMMUNITY PROGRAMS"                                      : ("DIRECTOR"                                        , "OF COMMUNITY PROGRAMS"                 , "None"                   , True),
"DIRECTOR OF COMPLIANCE & LEGISLATIVE DEVELOPMENT"                    : ("DIRECTOR"                                        , "OF COMPLIANCE & LEGISLATIVE DEVELOPMENT", "None"                   , True),
"DIRECTOR OF CONSUMER INFORMATION"                                    : ("DIRECTOR OF CONSUMER INFORMATION"                , "None"                                  , "None"                   , True),
"DIRECTOR OF CORRECTIONAL STANDARDS REVIEW"                           : ("DIRECTOR"                                        , "OF CORRECTIONAL STANDARDS REVIEW"      , "None"                   , True),
"DIRECTOR OF COUNCIL SERVICES BUREAU"                                 : ("DIRECTOR"                                        , "OF COUNCIL SERVICES BUREAU"            , "None"                   , True),
"DIRECTOR OF CRISIS ASSISTANCE AND TRAINING"                          : ("DIRECTOR"                                        , "OF CRISIS ASSISTANCE AND TRAINING"     , "None"                   , True),
"DIRECTOR OF DEPARTMENT ADVOCATES OFFICE"                             : ("DIRECTOR"                                        , "OF DEPARTMENT ADVOCATES OFFICE"        , "None"                   , True),
"DIRECTOR OF DIRECT SERVICE PROGRAMS"                                 : ("DIRECT SERVICE PROGRAMS"                         , "DIRECTOR OF"                           , "None"                   , True),
"DIRECTOR OF EEO"                                                     : ("DIRECTOR"                                        , "OF EEO"                                , "None"                   , True),
"DIRECTOR OF EEO/CONTRACT COMPLIANCE"                                 : ("EEO/CONTRACT COMPLIANCE"                         , "DIRECTOR OF"                           , "None"                   , True),
"DIRECTOR OF ENERGY CONSERVATION"                                     : ("DIRECTOR OF ENERGY CONSERVATION"                 , "None"                                  , "None"                   , True),
"DIRECTOR OF EQUAL OPPORTUNITY"                                       : ("DIRECTOR OF EQUAL OPPORTUNITY"                   , "None"                                  , "None"                   , True),
"DIRECTOR OF EQUIPMENT"                                               : ("DIRECTOR"                                        , "OF EQUIPMENT"                          , "None"                   , True),
"DIRECTOR OF FIELD OPERATIONS"                                        : ("FIELD OPERATIONS"                                , "DIRECTOR OF"                           , "None"                   , True),
"DIRECTOR OF FINANCE"                                                 : ("DIRECTOR OF FINANCE"                             , "None"                                  , "None"                   , True),
"DIRECTOR OF FINANCING POLICY COORDINATION"                           : ("DIRECTOR"                                        , "FINANCING POLICY COORDINATION"         , "None"                   , True),
"DIRECTOR OF FISCAL OPERATIONS"                                       : ("FISCAL OPERATIONS"                               , "DIRECTOR OF"                           , "None"                   , True),
"DIRECTOR OF FOOD STAMP PROGRAM"                                      : ("DIRECTOR"                                        , "OF FOOD STAMP PROGRAM"                 , "None"                   , True),
"DIRECTOR OF FORENSIC BIOLOGY"                                        : ("FORENSIC BIOLOGIST"                              , "None"                                  , "DIRECTOR"               , True),
"DIRECTOR OF GENERAL SERVICES"                                        : ("DIRECTOR OF GENERAL SERVICES"                    , "None"                                  , "None"                   , True),
"DIRECTOR OF HEALTH CARE PROGRAM PLANNING/ANALYSIS"                   : ("DIRECTOR"                                        , "HEALTH CARE PROGRAM PLANNING/ANALYSIS" , "None"                   , True),
"DIRECTOR OF HOUSING APPLICANTIONS"                                   : ("DIRECTOR"                                        , "OF HOUSING APPLICANTIONS"              , "None"                   , True),
"DIRECTOR OF INDEPENDENT BUDGET OFFICE"                               : ("DIRECTOR"                                        , "OF INDEPENDENT BUDGET OFFICE"          , "None"                   , True),
"DIRECTOR OF INFORMATION AND RESEARCH"                                : ("DIRECTOR"                                        , "OF INFORMATION AND RESEARCH"           , "None"                   , True),
"DIRECTOR OF INFORMATION SERVICES"                                    : ("INFORMATION SERVICES DIRECTOR"                   , "None"                                  , "None"                   , True),
"DIRECTOR OF INFORMATION SYSTEMS"                                     : ("INFORMATION SYSTEMS DIRECTOR"                    , "None"                                  , "None"                   , True),
"DIRECTOR OF INTERAGENCY PLANNING"                                    : ("INTERAGENCY PLANNING"                            , "DIRECTOR OF"                           , "None"                   , True),
"DIRECTOR OF INTERGOVERNMENTAL RELATIONS"                             : ("DIRECTOR OF INTERGOVERNMENTAL RELATIONS"         , "None"                                  , "None"                   , True),
"DIRECTOR OF INTERNAL AFFAIRS - PD"                                   : ("DIRECTOR"                                        , "INTERNAL AFFAIRS"                      , "None"                   , True),
"DIRECTOR OF INVESTMENTS"                                             : ("INVESTMENTS"                                     , "DIRECTOR OF"                           , "None"                   , True),
"DIRECTOR OF LABOR RELATIONS"                                         : ("DIRECTOR OF LABOR RELATIONS"                     , "None"                                  , "None"                   , True),
"DIRECTOR OF LAND USE DIVISION"                                       : ("DIRECTOR"                                        , "LAND USE"                              , "None"                   , True),
"DIRECTOR OF LEASED HOUSING"                                          : ("DIRECTOR OF LEASED HOUSING"                      , "None"                                  , "None"                   , True),
"DIRECTOR OF LEGISLATIVE COORDINATION"                                : ("LEGISLATIVE COORDINATOR"                         , "DIRECTOR OF"                           , "None"                   , True),
"DIRECTOR OF MANAGEMENT & BUDGET"                                     : ("DIRECTOR"                                        , "OF MANAGEMENT & BUDGET"                , "None"                   , True),
"DIRECTOR OF MANAGEMENT INFORMATION SERVICES"                         : ("MANAGEMENT INFORMATION SERVICES"                 , "DIRECTOR OF"                           , "None"                   , True),
"DIRECTOR OF MANAGEMENT PLANNING SS"                                  : ("DIRECTOR"                                        , "OF MANAGEMENT PLANNING"                , "None"                   , True),
"DIRECTOR OF MEDICAL ASSISTANCE PROGRAM"                              : ("DIRECTOR"                                        , "OF MEDICAL ASSISTANCE PROGRAM"         , "None"                   , True),
"DIRECTOR OF METHODS & ANALYSIS"                                      : ("DIRECTOR"                                        , "OF METHODS & ANALYSIS"                 , "None"                   , True),
"DIRECTOR OF MOTOR EQUIPMENT MAINTENANCE"                             : ("MOTOR EQUIPMENT MAINTENANCE"                     , "DIRECTOR OF"                           , "None"                   , True),
"DIRECTOR OF MOTOR EQUIPMENT MAINTENANCE-NM"                          : ("MOTOR EQUIPMENT MAINTENANCE"                     , "DIRECTOR OF"                           , "None"                   , True),
"DIRECTOR OF MOTOR EQUIPMENT"                                         : ("DIRECTOR"                                        , "OF MOTOR EQUIPMENT"                    , "None"                   , True),
"DIRECTOR OF MOTOR TRANSPORT"                                         : ("DIRECTOR OF MOTOR TRANSPORT"                     , "None"                                  , "None"                   , True),
"DIRECTOR OF NEIGHBORHOOD PARK RESTORATION"                           : ("DIRECTOR"                                        , "NEIGHBORHOOD PARK RESTORATION"         , "None"                   , True),
"DIRECTOR OF NOISE ABATEMENT"                                         : ("DIRECTOR"                                        , "OF NOISE ABATEMENT"                    , "None"                   , True),
"DIRECTOR OF NURSING SERVICES"                                        : ("NURSING SERVICES DIRECTOR"                       , "None"                                  , "None"                   , True),
"DIRECTOR OF NYC MEDICAL RESERVE CORPS"                               : ("DIRECTOR"                                        , "OF NYC MEDICAL RESERVE CORPS"          , "None"                   , True),
"DIRECTOR OF OPERATIONS"                                              : ("DIRECTOR OF OPERATIONS"                          , "None"                                  , "None"                   , True),
"DIRECTOR OF PHOTOGRAPHIC SERVICES-PD"                                : ("DIRECTOR OF PHOTOGRAPHIC SERVICES"               , "None"                                  , "None"                   , True),
"DIRECTOR OF PLANT OPERATIONS & MAINTENANCE"                          : ("DIRECTOR OF PLANT OPERATIONS & MAINTENANCE"      , "None"                                  , "None"                   , True),
"DIRECTOR OF POLICY ANALYSIS"                                         : ("POLICY ANALYSIS"                                 , "DIRECTOR OF"                           , "None"                   , True),
"DIRECTOR OF PROBATION"                                               : ("DIRECTOR OF PROBATION"                           , "None"                                  , "None"                   , True),
"DIRECTOR OF PROGRAM PLANNING"                                        : ("PROGRAM PLANNING DIRECTOR"                       , "None"                                  , "None"                   , True),
"DIRECTOR OF PSYCHOLOGICAL SERVICES"                                  : ("PSYCHOLOGICAL SERVICES"                          , "DIRECTOR OF"                           , "None"                   , True),
"DIRECTOR OF PUBLIC & COMMUNITY RELATIONS"                            : ("DIRECTOR OF PUBLIC & COMMUNITY RELATIONS"        , "None"                                  , "None"                   , True),
"DIRECTOR OF PUBLIC AFFAIRS"                                          : ("DIRECTOR OF PUBLIC AFFAIRS"                      , "None"                                  , "None"                   , True),
"DIRECTOR OF PUBLIC HEALTH EMERGENCY PLANNING"                        : ("DIRECTOR OF PUBLIC HEALTH EMERGENCY PLANNING"    , "None"                                  , "None"                   , True),
"DIRECTOR OF PUBLIC INFORMATION"                                      : ("DIRECTOR OF PUBLIC INFORMATION"                  , "None"                                  , "None"                   , True),
"DIRECTOR OF PUBLIC RELATIONS"                                        : ("PUBLIC RELATIONS DIRECTOR"                       , "None"                                  , "None"                   , True),
"DIRECTOR OF PUPPETRY"                                                : ("DIRECTOR"                                        , "PUPPETRY"                              , "None"                   , True),
"DIRECTOR OF PURCHASE"                                                : ("DIRECTOR OF PURCHASE"                            , "None"                                  , "None"                   , True),
"DIRECTOR OF REGIONAL JOINT INTEREST PARK"                            : ("DIRECTOR"                                        , "OF REGIONAL JOINT INTEREST PARK"       , "None"                   , True),
"DIRECTOR OF RESEARCH & POLICY DEVELOPMENT"                           : ("DIRECTOR"                                        , "RESEARCH & POLICY DEVELOPMENT"         , "None"                   , True),
"DIRECTOR OF RESEARCH PLANNING AND POLICY ANALYSIS"                   : ("DIRECTOR"                                        , "RESEARCH PLANNING AND POLICY ANALYSIS" , "None"                   , True),
"DIRECTOR OF SCHEDULING"                                              : ("SCHEDULING DIRECTOR"                             , "None"                                  , "None"                   , True),
"DIRECTOR OF SECURITY"                                                : ("DIRECTOR OF SECURITY"                            , "None"                                  , "None"                   , True),
"DIRECTOR OF STORES"                                                  : ("DIRECTOR"                                        , "OF STORES"                             , "None"                   , True),
"DIRECTOR OF SUPPORT SERVICES-PD"                                     : ("DIRECTOR"                                        , "OF SUPPORT SERVICES"                   , "None"                   , True),
"DIRECTOR OF TECHNICAL SERVICES"                                      : ("TECHNICAL SERVICES"                              , "DIRECTOR OF"                           , "None"                   , True),
"DIRECTOR OF TECHNOLOGY DEVELOPMENT-PD"                               : ("DIRECTOR OF TECHNOLOGY DEVELOPMENT"              , "PD"                                    , "None"                   , True),
"DIRECTOR OF TELEVISION"                                              : ("DIRECTOR"                                        , "OF TELEVISION"                         , "None"                   , True),
"DIRECTOR OF THE CITY RECORD"                                         : ("CITY RECORD"                                     , "DIRECTOR OF"                           , "None"                   , True),
"DIRECTOR OF THE OFFICE OF COLLECTIVE BARGAINING"                     : ("DIRECTOR"                                        , "OF THE OFFICE OF COLLECTIVE BARGAINING", "None"                   , True),
"DIRECTOR OF THE OMBUDSMAN PROGRAM"                                   : ("OMBUDSMAN PROGRAM DIRECTOR"                      , "None"                                  , "None"                   , True),
"DIRECTOR OF TRAINING"                                                : ("DIRECTOR OF TRAINING"                            , "None"                                  , "None"                   , True),
"DIRECTOR OF URBAN ARCHAEOLOGY"                                       : ("URBAN ARCHAEOLOGIST"                             , "None"                                  , "DIRECTOR"               , True),
"DIRECTOR OF URBAN PARK RANGER PROGRAM"                               : ("URBAN PARK RANGER"                               , "DIRECTOR OF"                           , "None"                   , True),
"DIRECTOR OR MANAGEMENT"                                              : ("DIRECTOR"                                        , "OR"                                    , "None"                   , True),
"DIRECTOR URBAN ARCHAEOLOGY - LPC"                                    : ("URBAN ARCHAEOLOGY"                               , "DIRECTOR"                              , "None"                   , True),
"DIRECTOR"                                                            : ("DIRECTOR"                                        , "None"                                  , "None"                   , True),
"DIRECTOR, DATA PROCESSING COORDINATION"                              : ("DATA PROCESSING COORDINATOR"                     , "None"                                  , "DIRECTOR"               , True),
"DIRECTOR, MEDICAL AFFAIRS"                                           : ("MEDICAL AFFAIRS"                                 , "None"                                  , "DIRECTOR"               , True),
"DIRECTOR, NYC ART COMMISSION"                                        : ("ART COMMISSION DIRECTOR"                         , "None"                                  , "None"                   , True),
"DIRECTOR, OFFICE OF OPERATIONS"                                      : ("DIRECTOR"                                        , "OFFICE OF OPERATIONS"                  , "None"                   , True),
"DIRECTOR, PUBLIC AFFAIRS AND COMMUNICATION"                          : ("DIRECTOR"                                        , "PUBLIC AFFAIRS AND COMMUNICATION"      , "None"                   , True),
"DIRECTOR,CITYWIDE OCCUPATIONAL SAFETY & HEALTH PROGRAM"              : ("DIRECTOR"                                        , "CITYWIDE OCCUPATIONAL SAFETY & HEALTH PROGRAM", "None"                   , True),
"DIRECTOR-OFFICE OF COMMUNICATIONS"                                   : ("DIRECTOR"                                        , "OFFICE OF COMMUNICATIONS"              , "None"                   , True),
"DIRECTOR-OFFICE OF POLICY"                                           : ("DIRECTOR"                                        , "OFFICE OF POLICY"                      , "None"                   , True),
"DISABILITY ACCOMMODATIONS SPECIALIST"                                : ("DISABILITY ACCOMMODATIONS SPECIALIST"            , "None"                                  , "None"                   , False),
"DISTINGUISHED LECTURER"                                              : ("LECTURER"                                        , "DISTINGUISHED"                         , "None"                   , False),
"DISTINGUISHED PROFESSOR"                                             : ("PROFESSOR"                                       , "DISTINGUISHED"                         , "None"                   , False),
"DISTRICT ATTORNEY"                                                   : ("DISTRICT ATTORNEY"                               , "None"                                  , "None"                   , True),
"DISTRICT MANAGER OF ADMINISTRATION AND BUSINESS AFFAIRS"             : ("DISTRICT MANAGER"                                , "OF ADMINISTRATION AND BUSINESS AFFAIRS", "None"                   , True),
"DISTRICT MANAGER"                                                    : ("DISTRICT MANAGER"                                , "None"                                  , "None"                   , True),
"DISTRICT SUPERVISOR"                                                 : ("DISTRICT SUPERVISOR"                             , "None"                                  , "None"                   , True),
"DOCKBUILDER"                                                         : ("DOCKBUILDER"                                     , "None"                                  , "None"                   , False),
"DOCKMASTER"                                                          : ("DOCKMASTER"                                      , "None"                                  , "None"                   , False),
"ECONOMIST"                                                           : ("ECONOMIST"                                       , "None"                                  , "None"                   , False),
"EDUCATION & VOCAT COUNSELOR"                                         : ("EDUCATION & VOCAT COUNSELOR"                     , "None"                                  , "None"                   , False),
"EDUCATION ANALYST TRAINEE"                                           : ("EDUCATION ANALYST"                               , "TRAINEE"                               , "None"                   , False),
"EDUCATION ANALYST"                                                   : ("EDUCATION ANALYST"                               , "None"                                  , "None"                   , False),
"EDUCATION OFFICER"                                                   : ("EDUCATION OFFICER"                               , "None"                                  , "None"                   , False),
"EDUCATIONAL ADMINISTRATOR CSA"                                       : ("EDUCATIONAL ADMINISTRATOR"                       , "CSA"                                   , "None"                   , False),
"EDUCATIONAL ADMINISTRATOR UFT"                                       : ("EDUCATIONAL ADMINISTRATOR"                       , "UFT"                                   , "None"                   , False),
"EDUCATIONAL ADMINISTRATOR"                                           : ("EDUCATIONAL ADMINISTRATOR"                       , "None"                                  , "None"                   , False),
"EDUCATIONAL MANAGEMENT ASSOCIATE"                                    : ("EDUCATIONAL MANAGEMENT ASSOCIATE"                , "None"                                  , "None"                   , False),
"ELECTION TRAINER"                                                    : ("ELECTION TRAINER"                                , "None"                                  , "None"                   , False),
"ELECTION WORKER"                                                     : ("ELECTION WORKER"                                 , "None"                                  , "None"                   , False),
"ELECTRICAL ENGINEER"                                                 : ("ELECTRICAL ENGINEER"                             , "None"                                  , "None"                   , False),
"ELECTRICAL ENGINEERING INTERN"                                       : ("ELECTRICAL ENGINEER"                             , "INTERN"                                , "None"                   , False),
"ELECTRICIAN"                                                         : ("ELECTRICIAN"                                     , "None"                                  , "None"                   , False),
"ELECTRICIAN'S HELPER"                                                : ("ELECTRICIAN"                                     , "HELPER OF"                             , "None"                   , False),
"ELEVATOR MECHANIC HELPER"                                            : ("ELEVATOR MECHANIC"                               , "HELPER"                                , "None"                   , False),
"ELEVATOR MECHANIC"                                                   : ("ELEVATOR MECHANIC"                               , "None"                                  , "None"                   , False),
"ELEVATOR OPERATOR"                                                   : ("ELEVATOR OPERATOR"                               , "None"                                  , "None"                   , False),
"ELIGIBILITY SPECIALIST"                                              : ("ELIGIBILITY SPECIALIST"                          , "None"                                  , "None"                   , False),
"EMERGENCY MEDICAL SPECIALIST TRAINEE"                                : ("EMERGENCY MEDICAL SPECIALIST"                    , "TRAINEE"                               , "None"                   , False),
"EMERGENCY MEDICAL SPECIALIST-EMT"                                    : ("EMERGENCY MEDICAL SPECIALIST"                    , "EMT"                                   , "None"                   , False),
"EMERGENCY MEDICAL SPECIALIST-PARAMEDIC"                              : ("EMERGENCY MEDICAL SPECIALIST"                    , "PARAMEDIC"                             , "None"                   , False),
"EMERGENCY PREPAREDNESS MANAGER"                                      : ("EMERGENCY PREPAREDNESS MANAGER"                  , "None"                                  , "None"                   , True),
"EMERGENCY PREPAREDNESS SPECIALIST"                                   : ("EMERGENCY PREPAREDNESS SPECIALIST"               , "None"                                  , "None"                   , False),
"EMERGENCY SERVICE AIDE"                                              : ("EMERGENCY SERVICE AIDE"                          , "None"                                  , "None"                   , False),
"EMPLOYEE ASSISTANCE PROGRAM SPECIALIST"                              : ("EMPLOYEE ASSISTANCE PROGRAM SPECIALIST"          , "None"                                  , "None"                   , False),
"EMPLOYEE HEALTH BENEFITS PROGRAM DIRECTOR"                           : ("HEALTH BENEFITS PROGRAM DIRECTOR"                , "None"                                  , "None"                   , True),
"ENGINEERING TECHNICIAN"                                              : ("ENGINEERING TECHNICIAN"                          , "None"                                  , "None"                   , False),
"ENVIRONMENTAL ENGINEER"                                              : ("ENVIRONMENTAL ENGINEER"                          , "None"                                  , "None"                   , False),
"ENVIRONMENTAL ENGINEERING INTERN"                                    : ("ENVIRONMENTAL ENGINEER"                          , "INTERN"                                , "None"                   , False),
"ENVIRONMENTAL HEALTH TECHNICIAN"                                     : ("ENVIRONMENTAL HEALTH TECHNICIAN"                 , "None"                                  , "None"                   , False),
"ENVIRONMENTAL POLICE OFFICER"                                        : ("ENVIRONMENTAL POLICE OFFICER"                    , "None"                                  , "None"                   , False),
"ENVIRONMENTAL POLICE OFFICER-MANAGERIAL"                             : ("ENVIRONMENTAL POLICE OFFICER"                    , "None"                                  , "None"                   , True),
"EOC ACCOUNTANT"                                                      : ("ACCOUNTANT"                                      , "EOC"                                   , "None"                   , False),
"EOC ACCOUNTING ASSISTANT"                                            : ("ACCOUNTING ASSISTANT"                            , "EOC"                                   , "None"                   , False),
"EOC ADJUNCT COLLEGE LAB TECH"                                        : ("COLLEGE LAB TECH"                                , "ADJUNCT"                               , "None"                   , False),
"EOC ADJUNCT LECTURER"                                                : ("LECTURER"                                        , "ADJUNCT"                               , "None"                   , False),
"EOC ADMINISTRATIVE ASSISTANT"                                        : ("ADMINISTRATIVE ASSISTANT"                        , "EOC"                                   , "None"                   , False),
"EOC ASSISTANT TO HEO"                                                : ("HEO"                                             , "ASSISTANT TO"                          , "None"                   , False),
"EOC ASSISTANT"                                                       : ("EOC ASSISTANT"                                   , "None"                                  , "None"                   , False),
"EOC COLLEGE LAB TECHNICIAN"                                          : ("LAB TECHNICIAN"                                  , "EOC COLLEGE"                           , "None"                   , False),
"EOC CUSTODIAL ASSISTANT"                                             : ("CUSTODIAL ASSISTANT"                             , "EOC"                                   , "None"                   , False),
"EOC CUSTODIAL SUPERVISOR"                                            : ("CUSTODIAL SUPERVISOR"                            , "EOC"                                   , "None"                   , True),
"EOC HIGHER EDUCATION ASSISTANT"                                      : ("HIGHER EDUCATION ASSISTANT"                      , "EOC"                                   , "None"                   , False),
"EOC HIGHER EDUCATION ASSOCIATE"                                      : ("HIGHER EDUCATION ASSOCIATE"                      , "EOC"                                   , "None"                   , False),
"EOC HIGHER EDUCATION OFFICER"                                        : ("HIGHER EDUCATION OFFICER"                        , "EOC"                                   , "None"                   , False),
"EOC LECTURER"                                                        : ("LECTURER"                                        , "EOC"                                   , "None"                   , False),
"EOC LECTURER/ DOCTRAL SCHEDULE"                                      : ("LECTURER"                                        , "DOCTRAL SCHEDULE"                      , "None"                   , False),
"EOC MAIL/MESSAGE SERVICES WORKER"                                    : ("MAIL/MESSAGE SERVICES WORKER"                    , "EOC"                                   , "None"                   , False),
"EOC OFFICE ASSISTANT"                                                : ("OFFICE ASSISTANT"                                , "EOC"                                   , "None"                   , False),
"EQUAL RIGHTS COMPL SPEC"                                             : ("EQUAL RIGHTS COMPL SPEC"                         , "None"                                  , "None"                   , False),
"EQUAL RIGHTS COMPLIANCE SPECIALIST"                                  : ("EQUAL RIGHTS COMPLIANCE SPECIALIST"              , "None"                                  , "None"                   , False),
"ESTIMATOR"                                                           : ("ESTIMATOR"                                       , "None"                                  , "None"                   , False),
"EVIDENCE AND PROPERTY CONTROL SPECIALIST"                            : ("EVIDENCE AND PROPERTY CONTROL SPECIALIST"        , "None"                                  , "None"                   , False),
"EXAMINER OF ACCOUNTS"                                                : ("EXAMINER"                                        , "OF ACCOUNTS"                           , "None"                   , False),
"EXAMINING ATTORNEY"                                                  : ("ATTORNEY"                                        , "EXAMINING"                             , "None"                   , False),
"EXEC ASST TO THE COMMISSIONER OF FINANCE"                            : ("COMMISSIONER OF FINANCE"                         , "EXEC ASST OF"                          , "None"                   , False),
"EXEC ASST TO THE DEPUTY COM FOR CITYWIDE PERSONNEL SVCS-DCAS"        : ("DEPUTY COMMISSIONER"                             , "EXEC ASST TO"                          , "None"                   , False),
"EXEC ASST TO THE DEPUTY COMMISSIONER"                                : ("DEPUTY COMMISSIONER"                             , "EXEC ASST TO"                          , "None"                   , False),
"EXEC DIRECTOR OF GRANTS MANAGEMENT AND ADMINISTRATION"               : ("EXEC DIRECTOR"                                   , "OF GRANTS MANAGEMENT AND ADMINISTRATION", "None"                   , True),
"EXEC SEC TO ADVISORY BD FOR LOCATIONAL AND EXTENSION EDUC"           : ("ADVISORY BOARD"                                  , "EXEC SEC TO"                           , "None"                   , False),
"EXECUTIVE ADMINISTRATOR OF GRACIE MANSION"                           : ("EXECUTIVE ADMINISTRATOR"                         , "OF GRACIE MANSION"                     , "None"                   , False),
"EXECUTIVE AGENCY COUNSEL"                                            : ("AGENCY COUNSEL"                                  , "EXECUTIVE"                             , "None"                   , True),
"EXECUTIVE ASSISTANT AND COUNSEL TO THE PUBLIC ADVOCATE"              : ("PUBLIC ADVOCATE"                                 , "EXECUTIVE ASSISTANT AND COUNSEL OF"    , "None"                   , False),
"EXECUTIVE ASSISTANT FOR PLANNING"                                    : ("EXECUTIVE ASSISTANT"                             , "FOR PLANNING"                          , "None"                   , False),
"EXECUTIVE ASSISTANT FOR PROGRAMS & POLICY"                           : ("EXECUTIVE ASSISTANT"                             , "FOR PROGRAMS & POLICY"                 , "None"                   , False),
"EXECUTIVE ASSISTANT TO COMMISSIONER OF TRANSPORTATION"               : ("COMMISSIONER OF TRANSPORTATION"                  , "EXECUTIVE ASSISTANT OF"                , "None"                   , False),
"EXECUTIVE ASSISTANT TO THE CHAIRMAN"                                 : ("CHAIRMAN"                                        , "EXECUTIVE ASSISTANT OF"                , "None"                   , False),
"EXECUTIVE ASSISTANT TO THE CHANCELLOR"                               : ("CHANCELLOR"                                      , "EXECUTIVE ASSISTANT OF"                , "None"                   , False),
"EXECUTIVE ASSISTANT TO THE CITY CLERK"                               : ("CITY CLERK"                                      , "EXECUTIVE ASSISTANT OF"                , "None"                   , False),
"EXECUTIVE ASSISTANT TO THE COMMISSIONER"                             : ("COMMISSIONER"                                    , "EXECUTIVE ASSISTANT OF"                , "None"                   , False),
"EXECUTIVE ASSISTANT TO THE COMPTROLLER"                              : ("COMPTROLLER"                                     , "EXECUTIVE ASSISTANT OF"                , "None"                   , False),
"EXECUTIVE ASSISTANT TO THE DEPUTY COMMISSIONER"                      : ("DEPUTY COMMISSIONER"                             , "EXECUTIVE ASSISTANT OF"                , "None"                   , False),
"EXECUTIVE ASSISTANT TO THE DIRECTOR OF MANAGEMENT & BUDGET"          : ("DIRECTOR OF MANAGEMENT & BUDGET"                 , "EXECUTIVE ASSISTANT OF"                , "None"                   , False),
"EXECUTIVE ASSISTANT TO THE DIRECTOR"                                 : ("DIRECTOR"                                        , "EXECUTIVE ASSISTANT OF"                , "None"                   , False),
"EXECUTIVE ASSISTANT TO THE EXECUTIVE DEPUTY ADM"                     : ("EXECUTIVE DEPUTY ADM"                            , "EXECUTIVE ASSISTANT TO"                , "None"                   , False),
"EXECUTIVE ASSISTANT TO THE FIRST DEPUTY CLERK"                       : ("FIRST DEPUTY CLERK"                              , "EXECUTIVE ASSISTANT OF"                , "None"                   , False),
"EXECUTIVE ASSISTANT TO THE HUMAN RESOURCES ADMIN"                    : ("HUMAN RESOURCES ADMIN"                           , "EXECUTIVE ASSISTANT TO"                , "None"                   , False),
"EXECUTIVE ASSISTANT TO THE PRESIDENT OF THE TAX COMMISSION"          : ("PRESIDENT OF THE TAX COMMISSION"                 , "EXECUTIVE ASSISTANT OF"                , "None"                   , False),
"EXECUTIVE ASSISTANT"                                                 : ("EXECUTIVE"                                       , "ASSISTANT"                             , "None"                   , False),
"EXECUTIVE ASSISTANT-MIDTOWN ENFORCEMENT"                             : ("EXECUTIVE ASSISTANT"                             , "MIDTOWN ENFORCEMENT"                   , "None"                   , False),
"EXECUTIVE ASST TO THE COMMISSIONER"                                  : ("COMMISSIONER"                                    , "EXECUTIVE ASST OF"                     , "None"                   , False),
"EXECUTIVE ASST-DOMESTIC VIOLENCE-HRA"                                : ("EXECUTIVE ASST"                                  , "DOMESTIC VIOLENCE-HRA"                 , "None"                   , False),
"EXECUTIVE CHIEF OF STAFF"                                            : ("CHIEF OF STAFF"                                  , "EXECUTIVE"                             , "None"                   , True),
"EXECUTIVE COOK"                                                      : ("COOK"                                            , "EXECUTIVE"                             , "None"                   , True),
"EXECUTIVE DEPUTY ADMINISTRATOR"                                      : ("DEPUTY ADMINISTRATOR"                            , "EXECUTIVE"                             , "None"                   , True),
"EXECUTIVE DEPUTY COMMISSIONER FOR MENTAL HYGIENE"                    : ("COMMISSIONER FOR MENTAL HYGIENE"                 , "EXECUTIVE DEPUTY"                      , "None"                   , True),
"EXECUTIVE DEPUTY COMMISSIONER"                                       : ("COMMISSIONER"                                    , "EXECUTIVE DEPUTY"                      , "None"                   , True),
"EXECUTIVE DIRECTOR OF ADMINISTRATION"                                : ("DIRECTOR OF ADMINISTRATION"                      , "EXECUTIVE"                             , "None"                   , True),
"EXECUTIVE DIRECTOR OF FOOD SERVICES"                                 : ("DIRECTOR OF FOOD SERVICES"                       , "EXECUTIVE"                             , "None"                   , True),
"EXECUTIVE DIRECTOR OF MATERIALS FOR THE ARTS"                        : ("EXECUTIVE DIRECTOR"                              , "OF MATERIALS FOR THE ARTS"             , "None"                   , True),
"EXECUTIVE DIRECTOR OF PROFESSIONAL DEVELOPMENT"                      : ("EXECUTIVE DIRECTOR"                              , "OF PROFESSIONAL DEVELOPMENT"           , "None"                   , True),
"EXECUTIVE DIRECTOR OF STANDARDS AND APPEALS"                         : ("DIRECTOR"                                        , "OF STANDARDS AND APPEALS"              , "EXECUTIVE"              , True),
"EXECUTIVE DIRECTOR TO THE ENVIRONMENTAL CONTROL BOARD"               : ("ENVIRONMENTAL CONTROL BOARD"                     , "EXECUTIVE DIRECTOR OF"                 , "None"                   , True),
"EXECUTIVE DIRECTOR"                                                  : ("EXECUTIVE DIRECTOR"                              , "None"                                  , "None"                   , True),
"EXECUTIVE DIRECTOR, NEW YORK CITY LOFT BOARD"                        : ("EXECUTIVE DIRECTOR"                              , "NEW YORK CITY LOFT BOARD"              , "None"                   , True),
"EXECUTIVE DIRECTOR, TECHNOLOGICAL DEVELOPMENT"                       : ("DIRECTOR, TECHNOLOGICAL DEVELOPMENT"             , "EXECUTIVE"                             , "None"                   , True),
"EXECUTIVE INSPECTOR"                                                 : ("INSPECTOR"                                       , "EXECUTIVE"                             , "None"                   , True),
"EXECUTIVE LEGISLATIVE COORDINATOR"                                   : ("LEGISLATIVE COORDINATOR"                         , "EXECUTIVE"                             , "None"                   , True),
"EXECUTIVE LEGISLATIVE SECRETARY"                                     : ("LEGISLATIVE SECRETARY"                           , "EXECUTIVE"                             , "None"                   , False),
"EXECUTIVE PROGRAM SPECIALIST"                                        : ("PROGRAM SPECIALIST"                              , "EXECUTIVE"                             , "None"                   , True),
"EXECUTIVE SECRETARY TO CITY CLERK"                                   : ("CITY CLERK"                                      , "EXECUTIVE SECRETARY OF"                , "None"                   , False),
"EXECUTIVE SECRETARY TO THE PUBLIC ADVOCATE"                          : ("PUBLIC ADVOCATE"                                 , "EXECUTIVE SECRETARY OF"                , "None"                   , False),
"EXECUTIVE SECRETARY"                                                 : ("EXECUTIVE SECRETARY"                             , "None"                                  , "None"                   , False),
"EXTERMINATOR"                                                        : ("EXTERMINATOR"                                    , "None"                                  , "None"                   , False),
"F/T DC 37 FAMILY PARA"                                               : ("FAMILY PARA"                                     , "None"                                  , "None"                   , False),
"F/T FILM INSPECTION ASST."                                           : ("FILM INSPECTOR"                                  , "ASSISTANT"                             , "None"                   , False),
"F/T GROUP LEADER"                                                    : ("GROUP LEADER"                                    , "None"                                  , "None"                   , True),
"F/T HEALTH SERVICE AIDE"                                             : ("HEALTH SERVICE AIDE"                             , "None"                                  , "None"                   , False),
"F/T SCHOOL AIDE"                                                     : ("SCHOOL AIDE"                                     , "None"                                  , "None"                   , False),
"F/T SCHOOL GUARD"                                                    : ("SCHOOL GUARD"                                    , "None"                                  , "None"                   , False),
"F/T SCHOOL LUNCH HELPER"                                             : ("SCHOOL LUNCH HELPER"                             , "None"                                  , "None"                   , False),
"F/T SCHOOL SAFETY OFFICER"                                           : ("SCHOOL SAFETY OFFICER"                           , "None"                                  , "None"                   , False),
"F/T SR. SCHOOL LUNCH HELPER"                                         : ("SCHOOL LUNCH HELPER"                             , "None"                                  , "SENIOR"                 , False),
"FACILITIES COORDINATOR"                                              : ("FACILITIES COORDINATOR"                          , "None"                                  , "None"                   , False),
"FAMILY PUB HEALTH NURSE"                                             : ("HEALTH NURSE"                                    , "FAMILY PUB"                            , "None"                   , False),
"FERRY TERMINAL SUPERVISOR"                                           : ("FERRY TERMINAL SUPERVISOR"                       , "None"                                  , "None"                   , True),
"FIELD INVESTIGATION SPECIALIST"                                      : ("INVESTIGATOR"                                    , "FIELD"                                 , "None"                   , False),
"FIELD SUPERVISOR"                                                    : ("FIELD SUPERVISOR"                                , "None"                                  , "None"                   , True),
"FILM MANAGER"                                                        : ("FILM MANAGER"                                    , "None"                                  , "None"                   , True),
"FINANCE OFFICER"                                                     : ("FINANCE OFFICER"                                 , "None"                                  , "None"                   , False),
"FINANCIAL CLERK"                                                     : ("FINANCIAL CLERK"                                 , "None"                                  , "None"                   , False),
"FINGERPRINT TECHNICIAN TRAINEE"                                      : ("FINGERPRINT TECHNICIAN"                          , "TRAINEE"                               , "None"                   , False),
"FIRE ALARM DISPATCHER"                                               : ("FIRE ALARM DISPATCHER"                           , "None"                                  , "None"                   , False),
"FIRE MARSHAL"                                                        : ("FIRE MARSHAL"                                    , "None"                                  , "None"                   , False),
"FIRE MEDICAL OFFICER"                                                : ("FIRE MEDICAL OFFICER"                            , "None"                                  , "None"                   , False),
"FIRE PROTECTION INSPECTOR"                                           : ("FIRE PROTECTION INSPECTOR"                       , "None"                                  , "None"                   , False),
"FIRE SAFETY CADET"                                                   : ("FIRE SAFETY CADET"                               , "None"                                  , "None"                   , False),
"FIREFIGHTER"                                                         : ("FIREFIGHTER"                                     , "None"                                  , "None"                   , False),
"FIRST ASSISTANT CORPORATION COUNSEL"                                 : ("CORPORATION COUNSEL"                             , "FIRST ASSISTANT"                       , "None"                   , False),
"FIRST ASSISTANT TO THE PUBLIC ADVOCATE"                              : ("PUBLIC ADVOCATE"                                 , "FIRST ASSISTANT OF"                    , "None"                   , False),
"FIRST ASST MARINE ENGINEER"                                          : ("MARINE ENGINEER"                                 , "ASST"                                  , "FIRST"                  , False),
"FIRST DEPUTY COMMISSIONER"                                           : ("COMMISSIONER"                                    , "FIRST DEPUTY"                          , "None"                   , True),
"FIRST DEPUTY COMPTROLLER"                                            : ("COMPTROLLER"                                     , "DEPUTY"                                , "FIRST"                  , True),
"FIRST DEPUTY MAYOR"                                                  : ("MAYOR"                                           , "DEPUTY"                                , "FIRST"                  , True),
"FISCAL AND POLICY ANALYST"                                           : ("FISCAL AND POLICY ANALYST"                       , "None"                                  , "None"                   , False),
"FISCAL RESPONSIBILITY OFFICER"                                       : ("FISCAL RESPONSIBILITY OFFICER"                   , "None"                                  , "None"                   , False),
"FITNESS INSTRUCTOR"                                                  : ("FITNESS INSTRUCTOR"                              , "None"                                  , "None"                   , False),
"FOOD SERVICE ADMINISTRATOR"                                          : ("FOOD SERVICE ADMINISTRATOR"                      , "None"                                  , "None"                   , False),
"FOOD SERVICE MANAGER"                                                : ("FOOD SERVICE MANAGER"                            , "None"                                  , "None"                   , True),
"FORENS MORT TECH-COORD MRT SRV"                                      : ("FORENSIC MORTUARY TECHNICIAN"                    , "COORD"                                 , "SRV"                    , True),
"FORENSIC MORTUARY TECHNICIAN"                                        : ("FORENSIC MORTUARY TECHNICIAN"                    , "None"                                  , "None"                   , False),
"FORENSIC MORTUARY TECHNICIAN-COORD OF MORTUARY SERVICES"             : ("FORENSIC MORTUARY TECHNICIAN"                    , "COORD OF MORTUARY SERVICES"            , "None"                   , True),
"FORESTER"                                                            : ("FORESTER"                                        , "None"                                  , "None"                   , False),
"FOSTER GRANDPARENT"                                                  : ("FOSTER GRANDPARENT"                              , "None"                                  , "None"                   , False),
"FRAUD INVESTIGATOR"                                                  : ("FRAUD INVESTIGATOR"                              , "None"                                  , "None"                   , False),
"FURNITURE MAINTAINER"                                                : ("FURNITURE MAINTAINER"                            , "None"                                  , "None"                   , False),
"GARDENER"                                                            : ("GARDENER"                                        , "None"                                  , "None"                   , False),
"GASOLINE ROLLER ENGINEER"                                            : ("GASOLINE ROLLER ENGINEER"                        , "None"                                  , "None"                   , False),
"GASOLINE ROLLER ENGINEER-L15"                                        : ("GASOLINE ROLLER ENGINEER"                        , "None"                                  , "L15"                    , False),
"GENERAL COUNSEL"                                                     : ("COUNSEL"                                         , "GENERAL"                               , "None"                   , False),
"GENERAL INSPECTOR"                                                   : ("INSPECTOR"                                       , "GENERAL"                               , "None"                   , False),
"GENERAL SUPERINTENDENT"                                              : ("SUPERINTENDENT"                                  , "None"                                  , "None"                   , True),
"GEOLOGIST"                                                           : ("GEOLOGIST"                                       , "None"                                  , "None"                   , False),
"GLAZIER"                                                             : ("GLAZIER"                                         , "None"                                  , "None"                   , False),
"GRAD ASSISTANT C"                                                    : ("GRAD ASSISTANT"                                  , "None"                                  , "C"                      , False),
"GRAPHIC ARTIST"                                                      : ("GRAPHIC ARTIST"                                  , "None"                                  , "None"                   , False),
"GUIDANCE COUNSELOR ASSIGNED A"                                       : ("GUIDANCE COUNSELOR"                              , "ASSIGNED"                              , "A"                      , False),
"GUIDANCE COUNSELOR"                                                  : ("GUIDANCE COUNSELOR"                              , "None"                                  , "None"                   , False),
"GUIDANCE COUNSELOR-REG SUB"                                          : ("GUIDANCE COUNSELOR"                              , "REG SUB"                               , "None"                   , False),
"HCCS SR COLLEGE LAB TECH"                                            : ("COLLEGE LAB TECH"                                , "SR"                                    , "None"                   , False),
"HCPPA"                                                               : ("HCPPA"                                           , "None"                                  , "None"                   , False),
"HEAD NURSE"                                                          : ("NURSE"                                           , "None"                                  , "None"                   , True),
"HEALTH CARE PROG PLAN/ANALYST"                                       : ("HEALTH CARE PROGRAM ANALYST"                     , "None"                                  , "None"                   , False),
"HEALTH SERVICES MANAGER"                                             : ("HEALTH SERVICES MANAGER"                         , "None"                                  , "None"                   , True),
"HEARING DIRECTOR"                                                    : ("HEARING DIRECTOR"                                , "None"                                  , "None"                   , False),
"HEARING EXAMINER"                                                    : ("HEARING EXAMINER"                                , "None"                                  , "None"                   , False),
"HEARING OFFICER"                                                     : ("HEARING OFFICER"                                 , "None"                                  , "None"                   , False),
"HEATING PLANT TECHNICIAN"                                            : ("HEATING PLANT TECHNICIAN"                        , "None"                                  , "None"                   , False),
"HIGH PRESSURE PLANT TENDER"                                          : ("PLANT TENDER"                                    , "HIGH PRESSURE"                         , "None"                   , False),
"HIGH SCHOOL ELEM COUNSELOR"                                          : ("COUNSELOR"                                       , "HIGH SCHOOL ELEM"                      , "None"                   , False),
"HIGH SCHOOL STUDENT AIDE"                                            : ("STUDENT AIDE"                                    , "HIGH SCHOOL"                           , "None"                   , False),
"HIGHER EDUCATION ASSISTANT"                                          : ("HIGHER EDUCATION ASSISTANT"                      , "None"                                  , "HIGHER"                 , False),
"HIGHER EDUCATION ASSOCIATE"                                          : ("HIGHER EDUCATION ASSOCIATE"                      , "None"                                  , "None"                   , False),
"HIGHER EDUCATION OFFICER"                                            : ("HIGHER EDUCATION OFFICER"                        , "None"                                  , "None"                   , False),
"HIGHER EDUCATION OFFICER/ASST ADMINISTRATOR"                         : ("HIGHER EDUCATION OFFICER"                        , "ASST"                                  , "None"                   , False),
"HIGHWAY REPAIRER"                                                    : ("HIGHWAY REPAIRER"                                , "None"                                  , "None"                   , False),
"HIGHWAY TRANSPORTATION SPECIALIST"                                   : ("HIGHWAY TRANSPORTATION SPECIALIST"               , "None"                                  , "None"                   , False),
"HIGHWAYS AND SEWERS INSPECTOR"                                       : ("HIGHWAYS AND SEWERS INSPECTOR"                   , "None"                                  , "None"                   , False),
"HOME AIDE"                                                           : ("HOME AIDE"                                       , "None"                                  , "None"                   , False),
"HOMEMAKER"                                                           : ("HOMEMAKER"                                       , "None"                                  , "None"                   , False),
"HORSESHOER"                                                          : ("HORSESHOER"                                      , "None"                                  , "None"                   , False),
"HOSTLER"                                                             : ("HOSTLER"                                         , "None"                                  , "None"                   , False),
"HOUSEKEEPER"                                                         : ("HOUSEKEEPER"                                     , "None"                                  , "None"                   , False),
"HOUSEPARENT AIDE"                                                    : ("HOUSEPARENT"                                     , "AIDE"                                  , "None"                   , False),
"HOUSING ASSISTANT"                                                   : ("HOUSING ASSISTANT"                               , "None"                                  , "None"                   , False),
"HOUSING CARETAKER"                                                   : ("HOUSING CARETAKER"                               , "None"                                  , "None"                   , False),
"HOUSING CONSULTANT"                                                  : ("HOUSING CONSULTANT"                              , "None"                                  , "None"                   , False),
"HOUSING DEVELOPMENT SPECIALIST TRAINEE"                              : ("HOUSING DEVELOPMENT SPECIALIST"                  , "TRAINEE"                               , "None"                   , False),
"HOUSING DEVELOPMENT SPECIALIST"                                      : ("HOUSING DEVELOPMENT SPECIALIST"                  , "None"                                  , "None"                   , False),
"HOUSING EXTERMINATOR"                                                : ("EXTERMINATOR"                                    , "HOUSING"                               , "None"                   , False),
"HOUSING MANAGER"                                                     : ("HOUSING MANAGER"                                 , "None"                                  , "None"                   , True),
"HOUSING STOCK WORKER"                                                : ("HOUSING STOCK WORKER"                            , "None"                                  , "None"                   , False),
"HUMAN RESOURCES ADMINISTRATOR"                                       : ("HUMAN RESOURCES ADMINISTRATOR"                   , "None"                                  , "None"                   , False),
"HUMAN RESOURCES TECHNICIAN"                                          : ("HUMAN RESOURCES TECHNICIAN"                      , "None"                                  , "None"                   , False),
"HUMAN RIGHTS SPECIALIST"                                             : ("HUMAN RIGHTS SPECIALIST"                         , "None"                                  , "None"                   , False),
"INCINERATOR FACILITY MANAGER"                                        : ("INCINERATOR FACILITY MANAGER"                    , "None"                                  , "None"                   , True),
"INDUSTRIAL HYGIENIST"                                                : ("INDUSTRIAL HYGIENIST"                            , "None"                                  , "None"                   , False),
"INFORMATION SYSTEMS AIDE"                                            : ("INFORMATION SYSTEMS AIDE"                        , "None"                                  , "None"                   , False),
"INSPECTOR GENERAL"                                                   : ("INSPECTOR GENERAL"                               , "None"                                  , "None"                   , True),
"INSPECTOR"                                                           : ("INSPECTOR"                                       , "None"                                  , "None"                   , False),
"INSTITUTIONAL AIDE"                                                  : ("INSTITUTIONAL AIDE"                              , "None"                                  , "None"                   , False),
"INSTRUCTOR"                                                          : ("INSTRUCTOR"                                      , "None"                                  , "None"                   , False),
"INSTRUMENTAL SPECIALIST TRAINEE"                                     : ("INSTRUMENTAL SPECIALIST"                         , "TRAINEE"                               , "None"                   , False),
"INSTRUMENTAL SPECIALIST"                                             : ("INSTRUMENTAL SPECIALIST"                         , "None"                                  , "None"                   , False),
"INSURANCE ADVISOR"                                                   : ("INSURANCE ADVISOR"                               , "None"                                  , "None"                   , False),
"INTELLIGENCE RESEARCH MANAGER"                                       : ("INTELLIGENCE RESEARCH MANAGER"                   , "None"                                  , "None"                   , True),
"INTELLIGENCE RESEARCH MANAGER-PD"                                    : ("INTELLIGENCE RESEARCH MANAGER"                   , "None"                                  , "None"                   , True),
"INTELLIGENCE RESEARCH SPECIALIST"                                    : ("INTELLIGENCE RESEARCH SPECIALIST"                , "None"                                  , "None"                   , False),
"INTELLIGENCE RESEARCH SPECIALIST-PD"                                 : ("INTELLIGENCE RESEARCH SPECIALIST"                , "None"                                  , "None"                   , False),
"INTERPRETER"                                                         : ("INTERPRETER"                                     , "None"                                  , "None"                   , False),
"INTERPRETER/TRANSLATOR DOE"                                          : ("INTERPRETER/TRANSLATOR"                          , "None"                                  , "None"                   , False),
"INTERPRETER/TRANSLATOR"                                              : ("INTERPRETER/TRANSLATOR"                          , "None"                                  , "None"                   , False),
"INVESTIGATIVE MANAGER"                                               : ("INVESTIGATIVE MANAGER"                           , "None"                                  , "None"                   , True),
"INVESTIGATOR EMPL DISC"                                              : ("INVESTIGATOR"                                    , "EMPL DISC"                             , "None"                   , False),
"INVESTIGATOR TRAINEE"                                                : ("INVESTIGATOR"                                    , "TRAINEE"                               , "None"                   , False),
"INVESTIGATOR"                                                        : ("INVESTIGATOR"                                    , "None"                                  , "None"                   , False),
"INVESTMENT ANALYST"                                                  : ("INVESTMENT ANALYST"                              , "None"                                  , "None"                   , False),
"INVESTMENT MANAGER"                                                  : ("INVESTMENT MANAGER"                              , "None"                                  , "None"                   , True),
"INVESTMENT TRUSTEE - DEFERRED COMPENSATION"                          : ("INVESTMENT TRUSTEE"                              , "DEFERRED COMPENSATION"                 , "None"                   , False),
"IT ASSISTANT"                                                        : ("IT ASSISTANT"                                    , "None"                                  , "None"                   , False),
"IT ASSOCIATE"                                                        : ("IT ASSOCIATE"                                    , "None"                                  , "None"                   , False),
"IT AUTOMATION AND MONITORING ENGINEER"                               : ("IT ENGINEER"                                     , "AUTOMATION AND MONITORING"             , "None"                   , False),
"IT INFRASTRUCTURE ENGINEER"                                          : ("IT INFRASTRUCTURE ENGINEER"                      , "None"                                  , "None"                   , False),
"IT PROJECT SPECIALIST"                                               : ("IT PROJECT SPECIALIST"                           , "None"                                  , "None"                   , False),
"IT SECURITY SPECIALIST"                                              : ("IT SECURITY SPECIALIST"                          , "None"                                  , "None"                   , False),
"IT SENIOR ASSOCIATE"                                                 : ("IT ASSOCIATE"                                    , "None"                                  , "SENIOR"                 , False),
"IT SERVICE MANAGEMENT SPECIALIST"                                    : ("IT SERVICE MANAGEMENT SPECIALIST"                , "None"                                  , "None"                   , False),
"IT SUPPORT ASSISTANT"                                                : ("IT SUPPORT"                                      , "ASSISTANT"                             , "None"                   , False),
"JOB OPPORTUNITY SPECIALIST"                                          : ("SPECIALIST"                                      , "JOB OPPORTUNITY"                       , "None"                   , False),
"JOB TRAINING PARTICIPANT"                                            : ("JOB TRAINING PARTICIPANT"                        , "None"                                  , "None"                   , False),
"JR PH NURSE"                                                         : ("PH NURSE"                                        , "JR"                                    , "None"                   , False),
"JR PHNURSE"                                                          : ("PHNURSE"                                         , "JR"                                    , "None"                   , False),
"JUNIOR PUBLIC HEALTH NURSE"                                          : ("PUBLIC HEALTH NURSE"                             , "JUNIOR"                                , "None"                   , False),
"JUVENILE COUNSELOR"                                                  : ("JUVENILE COUNSELOR"                              , "None"                                  , "None"                   , False),
"LAB SPECIALIST/ASSISTANT"                                            : ("LAB SPECIALIST"                                  , "ASSISTANT OF"                          , "None"                   , False),
"LAB SPECIALIST/ASSISTANT-REG SUB"                                    : ("LAB SPECIALIST/ASSISTANT"                        , "REG SUB"                               , "None"                   , False),
"LABOR RELATIONS ANALYST TRAINEE"                                     : ("LABOR RELATIONS ANALYST"                         , "TRAINEE"                               , "None"                   , False),
"LABOR RELATIONS ANALYST"                                             : ("LABOR RELATIONS ANALYST"                         , "None"                                  , "None"                   , False),
"LABOR RELATIONS TRIAL EXAMINER"                                      : ("LABOR RELATIONS EXAMINER"                        , "TRIAL"                                 , "None"                   , False),
"LABORATORY ASSOCIATE"                                                : ("LABORATORY ASSOCIATE"                            , "None"                                  , "None"                   , False),
"LABORATORY HELPER"                                                   : ("LABORATORY HELPER"                               , "None"                                  , "None"                   , False),
"LABORATORY MICROBIOLOGIST"                                           : ("LABORATORY MICROBIOLOGIST"                       , "None"                                  , "None"                   , False),
"LABORER"                                                             : ("LABORER"                                         , "None"                                  , "None"                   , False),
"LANDMARKS PRESERVATIONIST"                                           : ("LANDMARKS PRESERVATIONIST"                       , "None"                                  , "None"                   , False),
"LANDSCAPE ARCHITECT INTERN"                                          : ("LANDSCAPE ARCHITECT"                             , "INTERN"                                , "None"                   , False),
"LANDSCAPE ARCHITECT"                                                 : ("LANDSCAPE ARCHITECT"                             , "None"                                  , "None"                   , False),
"LEAD ABATEMENT WORKER"                                               : ("ABATEMENT WORKER"                                , "None"                                  , "None"                   , True),
"LEAD THEATRE TECHNICIAN"                                             : ("THEATRE TECHNICIAN"                              , "None"                                  , "LEAD"                   , True),
"LECTURER"                                                            : ("LECTURER"                                        , "None"                                  , "None"                   , False),
"LECTURER/DOCTORAL SCHEDULE"                                          : ("LECTURER"                                        , "DOCTORAL SCHEDULE"                     , "None"                   , False),
"LEGAL COORDINATOR"                                                   : ("LEGAL COORDINATOR"                               , "None"                                  , "None"                   , False),
"LEGAL SECRETARIAL ASSISTANT - AL 1 ONLY"                             : ("LEGAL SECRETARIAL ASSISTANT"                     , "None"                                  , "GRADE 1"                , False),
"LEGAL SECRETARIAL ASSISTANT AL 2,3,4 ONLY"                           : ("LEGAL SECRETARIAL ASSISTANT"                     , "None"                                  , "GRADE 2,3,4"            , False),
"LEGAL SECRETARIAL ASSISTANT"                                         : ("LEGAL SECRETARY"                                 , "ASSISTANT"                             , "None"                   , False),
"LEGISLATIVE ADMINISTRATIVE ASSISTANT"                                : ("LEGISLATIVE ADMINISTRATIVE ASSISTANT"            , "None"                                  , "None"                   , False),
"LEGISLATIVE ADMINISTRATIVE MANAGER"                                  : ("LEGISLATIVE ADMINISTRATIVE MANAGER"              , "None"                                  , "None"                   , True),
"LEGISLATIVE ADMINISTRATIVE SECRETARY"                                : ("LEGISLATIVE SECRETARY"                           , "ADMINISTRATIVE"                        , "None"                   , False),
"LEGISLATIVE AIDE"                                                    : ("LEGISLATIVE AIDE"                                , "None"                                  , "None"                   , False),
"LEGISLATIVE ANALYST"                                                 : ("LEGISLATIVE ANALYST"                             , "None"                                  , "None"                   , False),
"LEGISLATIVE ASSISTANT"                                               : ("LEGISLATIVE ASSISTANT"                           , "None"                                  , "None"                   , False),
"LEGISLATIVE ATTORNEY"                                                : ("LEGISLATIVE ATTORNEY"                            , "None"                                  , "None"                   , False),
"LEGISLATIVE CLERK"                                                   : ("LEGISLATIVE CLERK"                               , "None"                                  , "None"                   , False),
"LEGISLATIVE COMPUTER SUPPORT SPECIALIST"                             : ("COMPUTER SUPPORT SPECIALIST"                     , "LEGISLATIVE"                           , "None"                   , False),
"LEGISLATIVE COORDINATOR"                                             : ("LEGISLATIVE COORDINATOR"                         , "None"                                  , "None"                   , False),
"LEGISLATIVE COUNSEL"                                                 : ("LEGISLATIVE COUNSEL"                             , "None"                                  , "None"                   , False),
"LEGISLATIVE FINANCIAL ANALYST"                                       : ("LEGISLATIVE FINANCIAL ANALYST"                   , "None"                                  , "None"                   , False),
"LEGISLATIVE FISCAL OFFICER"                                          : ("LEGISLATIVE FISCAL OFFICER"                      , "None"                                  , "None"                   , False),
"LEGISLATIVE INFORMATION OFFICER"                                     : ("LEGISLATIVE INFORMATION OFFICER"                 , "None"                                  , "None"                   , False),
"LEGISLATIVE INTERN"                                                  : ("LEGISLATIVE INTERN"                              , "None"                                  , "None"                   , False),
"LEGISLATIVE INVESTIGATOR"                                            : ("INVESTIGATOR"                                    , "LEGISLATIVE"                           , "None"                   , False),
"LEGISLATIVE MESSENGER"                                               : ("LEGISLATIVE MESSENGER"                           , "None"                                  , "None"                   , False),
"LEGISLATIVE POLICY ANALYST"                                          : ("LEGISLATIVE POLICY ANALYST"                      , "None"                                  , "None"                   , False),
"LEGISLATIVE PROGRAMMER/ANALYST"                                      : ("LEGISLATIVE PROGRAMMER/ANALYST"                  , "None"                                  , "None"                   , False),
"LEGISLATIVE PROJECT MANAGER"                                         : ("LEGISLATIVE PROJECT MANAGER"                     , "None"                                  , "None"                   , False),
"LEGISLATIVE SUPPORT SERVICE COORD"                                   : ("LEGISLATIVE SUPPORT SERVICE COORD"               , "None"                                  , "None"                   , False),
"LEGISLATIVE SYSTEMS MANAGER"                                         : ("LEGISLATIVE SYSTEMS MANAGER"                     , "None"                                  , "None"                   , True),
"LEGISTATIVE MANAGER-OFFICE OF COMMUNICATION"                         : ("LEGISLATIVE MANAGER"                             , "OFFICE OF COMMUNICATION"               , "None"                   , True),
"LETTERER AND SIGN PAINTER"                                           : ("LETTERER AND SIGN PAINTER"                       , "None"                                  , "None"                   , False),
"LICENSED BARBER"                                                     : ("BARBER"                                          , "LICENSED"                              , "None"                   , False),
"LIEUTENANT D/A COMMANDER OF DETECTIVE SQUAD"                         : ("LIEUTENANT"                                      , "COMMANDER OF DETECTIVE SQUAD"          , "None"                   , True),
"LIEUTENANT D/A SPECIAL ASSIGNMENT"                                   : ("LIEUTENANT"                                      , "D/A SPECIAL ASSIGNMENT"                , "None"                   , False),
"LIEUTENANT"                                                          : ("LIEUTENANT"                                      , "None"                                  , "None"                   , True),
"LIFE GUARD"                                                          : ("LIFE GUARD"                                      , "None"                                  , "None"                   , False),
"LOCAL INSTRUCTIONAL SUPERVISOR"                                      : ("INSTRUCTIONAL SUPERVISOR"                        , "LOCAL"                                 , "None"                   , True),
"LOCKSMITH"                                                           : ("LOCKSMITH"                                       , "None"                                  , "None"                   , False),
"MACHINIST"                                                           : ("MACHINIST"                                       , "None"                                  , "None"                   , False),
"MACHINIST'S HELPER"                                                  : ("MACHINIST"                                       , "HELPER OF"                             , "None"                   , False),
"MAIL/MESSAGE SERVICES WORKER"                                        : ("MAIL/MESSAGE SERVICES WORKER"                    , "None"                                  , "None"                   , False),
"MAINTENANCE WORKER"                                                  : ("MAINTENANCE WORKER"                              , "None"                                  , "None"                   , False),
"MANAGEMENT AUDITOR TRAINEE"                                          : ("MANAGEMENT AUDITOR"                              , "TRAINEE"                               , "None"                   , False),
"MANAGEMENT AUDITOR"                                                  : ("MANAGEMENT AUDITOR"                              , "None"                                  , "None"                   , False),
"MANAGER OF PARK LICENSE AGREEMENTS"                                  : ("PARK LICENSE AGREEMENTS"                         , "None"                                  , "None"                   , True),
"MANAGER OF RADIO REPAIR OPERATIONS"                                  : ("RADIO REPAIR OPERATIONS"                         , "None"                                  , "None"                   , True),
"MANAGER, WATER AND SEWER SYSTEMS"                                    : ("WATER AND SEWER SYSTEMS"                         , "None"                                  , "None"                   , True),
"MANAGER, WATER SUPPLY"                                               : ("WATER SUPPLY"                                    , "None"                                  , "None"                   , True),
"MARINE ELECTRONICS TECHNICIAN"                                       : ("MARINE ELECTRONICS TECHNICIAN"                   , "None"                                  , "None"                   , False),
"MARINE ENGINEER"                                                     : ("MARINE ENGINEER"                                 , "None"                                  , "None"                   , False),
"MARINE MAINTENANCE MECHANIC"                                         : ("MARINE MAINTENANCE MECHANIC"                     , "None"                                  , "None"                   , False),
"MARINE OILER"                                                        : ("MARINE OILER"                                    , "None"                                  , "None"                   , False),
"MARINER"                                                             : ("MARINER"                                         , "None"                                  , "None"                   , False),
"MARKET AGENT"                                                        : ("MARKET AGENT"                                    , "None"                                  , "None"                   , False),
"MASONS HELPER"                                                       : ("MASON"                                           , "HELPER"                                , "None"                   , False),
"MATE"                                                                : ("MATE"                                            , "None"                                  , "None"                   , False),
"MAYOR"                                                               : ("MAYOR"                                           , "None"                                  , "None"                   , True),
"MAYORAL OFFICE ASSISTANT"                                            : ("MAYORAL OFFICE ASSISTANT"                        , "None"                                  , "None"                   , False),
"MAYORAL PROGRAM COORDINATOR"                                         : ("PROGRAM COORDINATOR"                             , "MAYORAL"                               , "None"                   , False),
"MECHANICAL ENGINEER"                                                 : ("MECHANICAL ENGINEER"                             , "None"                                  , "None"                   , False),
"MECHANICAL ENGINEERING INTERN"                                       : ("MECHANICAL ENGINEER"                             , "INTERN"                                , "None"                   , False),
"MEDIA SERVICES TECHNICIAN"                                           : ("MEDIA SERVICES TECHNICIAN"                       , "None"                                  , "None"                   , False),
"MEDICAL DIRECTOR"                                                    : ("MEDICAL DIRECTOR"                                , "None"                                  , "None"                   , True),
"MEDICAL INVESTIGATOR"                                                : ("MEDICAL INVESTIGATOR"                            , "None"                                  , "None"                   , False),
"MEDICAL OFFICER"                                                     : ("MEDICAL OFFICER"                                 , "None"                                  , "None"                   , False),
"MEDICAL RECORD LIBRARIAN"                                            : ("MEDICAL RECORD LIBRARIAN"                        , "None"                                  , "None"                   , False),
"MEDICAL SPECIALIST"                                                  : ("MEDICAL SPECIALIST"                              , "None"                                  , "None"                   , False),
"MEDICOLEGAL ANALYST"                                                 : ("MEDICOLEGAL ANALYST"                             , "None"                                  , "None"                   , False),
"MEDICOLEGAL INVESTIGATOR"                                            : ("MEDICOLEGAL INVESTIGATOR"                        , "None"                                  , "None"                   , False),
"MEMBER OF THE ENVIRONMENTAL CONTROL BOARD - OATH"                    : ("ENVIRONMENTAL CONTROL BOARD MEMBER"              , "None"                                  , "None"                   , False),
"MEMBER OF THE HOUSING AUTHORITY"                                     : ("HOUSING AUTHORITY MEMBER"                        , "None"                                  , "None"                   , False),
"MEMBER, CIVILIAN COMPLAINT REVIEW BOARD"                             : ("CIVILIAN COMPLAINT REVIEW BOARD MEMBER"          , "None"                                  , "None"                   , False),
"MENTAL HEALTH WORKER"                                                : ("MENTAL HEALTH WORKER"                            , "None"                                  , "None"                   , False),
"METAL WORK MECHANIC"                                                 : ("METAL WORK MECHANIC"                             , "None"                                  , "None"                   , False),
"MINORITY LEADER"                                                     : ("LEADER"                                          , "MINORITY"                              , "None"                   , True),
"MONITOR"                                                             : ("MONITOR"                                         , "None"                                  , "None"                   , False),
"MORTGAGE TAX EXAMINER"                                               : ("MORTGAGE TAX EXAMINER"                           , "None"                                  , "None"                   , False),
"MOTOR GRADER OPERATOR"                                               : ("MOTOR GRADER OPERATOR"                           , "None"                                  , "None"                   , False),
"MOTOR VEHICLE OPERATOR"                                              : ("MOTOR VEHICLE OPERATOR"                          , "None"                                  , "None"                   , False),
"MOTOR VEHICLE SUPERVISOR"                                            : ("MOTOR VEHICLE SUPERVISOR"                        , "None"                                  , "None"                   , True),
"MULTIPLE DWELLING SPECIALIST"                                        : ("MULTIPLE DWELLING SPECIALIST"                    , "None"                                  , "None"                   , False),
"NEW LEADER"                                                          : ("LEADER"                                          , "None"                                  , "None"                   , True),
"NEW PRINCIPAL INTERN"                                                : ("PRINCIPAL INTERN"                                , "NEW"                                   , "None"                   , False),
"NEW YORK CITY FELLOW"                                                : ("FELLOW"                                          , "NEW YORK CITY"                         , "None"                   , False),
"NEW YORK CITY URBAN FELLOW"                                          : ("URBAN FELLOW"                                    , "None"                                  , "None"                   , False),
"NON-TEACHING ADJUNCT I"                                              : ("ADJUNCT"                                         , "NON-TEACHING"                          , "I"                      , False),
"NON-TEACHING ADJUNCT II"                                             : ("ADJUNCT"                                         , "NON-TEACHING"                          , "II"                     , False),
"NON-TEACHING ADJUNCT III"                                            : ("ADJUNCT"                                         , "NON-TEACHING"                          , "III"                    , False),
"NON-TEACHING ADJUNCT IV"                                             : ("ADJUNCT"                                         , "NON-TEACHING"                          , "IV"                     , False),
"NON-TEACHING ADJUNCT V"                                              : ("ADJUNCT"                                         , "NON-TEACHING"                          , "V"                      , False),
"NURSE PRACTICIONER"                                                  : ("NURSE PRACTICIONER"                              , "None"                                  , "None"                   , False),
"NURSE'S AIDE"                                                        : ("NURSE'S AIDE"                                    , "None"                                  , "None"                   , False),
"NUTRITION CONSULTANT"                                                : ("NUTRITION CONSULTANT"                            , "None"                                  , "None"                   , False),
"NUTRITIONIST"                                                        : ("NUTRITIONIST"                                    , "None"                                  , "None"                   , False),
"NYCAPS PROCESS ANALYST MANAGER"                                      : ("PROCESS ANALYST"                                 , "None"                                  , "None"                   , True),
"NYCAPS PROCESS ANALYST"                                              : ("PROCESS ANALYST"                                 , "NYCAPS"                                , "None"                   , False),
"None"                                                                : ("None"                                            , "None"                                  , "None"                   , False),
"OCCUPATIONAL THERAPIST"                                              : ("OCCUPATIONAL THERAPIST"                          , "None"                                  , "None"                   , False),
"OFFICE ASSISTANT"                                                    : ("OFFICE ASSISTANT"                                , "None"                                  , "None"                   , False),
"OFFICE MACHINE AIDE"                                                 : ("OFFICE MACHINE AIDE"                             , "None"                                  , "None"                   , False),
"OFFICE MANAGER"                                                      : ("OFFICE MANAGER"                                  , "None"                                  , "None"                   , True),
"OIL BURNER SPECIALIST"                                               : ("OIL BURNER SPECIALIST"                           , "None"                                  , "None"                   , False),
"OILER"                                                               : ("OILER"                                           , "None"                                  , "None"                   , False),
"OMBUDSMAN"                                                           : ("OMBUDSMAN"                                       , "None"                                  , "None"                   , False),
"OPERATIONS COMMUNICATIONS SPECIALIST"                                : ("OPERATIONS COMMUNICATIONS SPECIALIST"            , "None"                                  , "None"                   , False),
"P.O. DA DET GR3"                                                     : ("POLICE OFFICER"                                  , "DA DETECTIVE"                          , "GRADE 3"                , False),
"P.O. DET SPEC"                                                       : ("POLICE OFFICER"                                  , "DETECTIVE SPECIALIST"                  , "None"                   , False),
"P/T DC 37 FAMILY PARA"                                               : ("FAMILY PARA"                                     , "P/T DC 37"                             , "None"                   , False),
"P/T SCHOOL AIDE"                                                     : ("SCHOOL AIDE"                                     , "P/T"                                   , "None"                   , False),
"PAINTER"                                                             : ("PAINTER"                                         , "None"                                  , "None"                   , False),
"PARALEGAL AIDE"                                                      : ("PARALEGAL AIDE"                                  , "None"                                  , "None"                   , False),
"PARK BOROUGH COMMISSIONER"                                           : ("PARK COMMISSIONER"                               , "BOROUGH"                               , "None"                   , True),
"PARK SERVICES WORKER"                                                : ("PARK SERVICES WORKER"                            , "None"                                  , "None"                   , False),
"PARK SUPERVISOR"                                                     : ("PARK SUPERVISOR"                                 , "None"                                  , "None"                   , True),
"PARKING CONTROL SPECIALIST"                                          : ("PARKING CONTROL SPECIALIST"                      , "None"                                  , "None"                   , False),
"PEER COUNSELOR"                                                      : ("PEER COUNSELOR"                                  , "None"                                  , "None"                   , False),
"PENSION INVESTMENT ADVISOR"                                          : ("PENSION INVESTMENT ADVISOR"                      , "None"                                  , "None"                   , False),
"PER-SESSION ED PARA"                                                 : ("ED PARA"                                         , "PER-SESSION"                           , "None"                   , False),
"PERSONNEL DIRECTOR - HEALTH CARE FACILITIES"                         : ("PERSONNEL DIRECTOR"                              , "HEALTH CARE FACILITIES"                , "None"                   , True),
"PERSONNEL PROGRAM DEVELOPEMENT SPECIALIST"                           : ("PERSONNEL PROGRAM DEVELOPEMENT SPECIALIST"       , "None"                                  , "None"                   , False),
"PHARMACY TECHNICIAN"                                                 : ("PHARMACY TECHNICIAN"                             , "None"                                  , "None"                   , False),
"PHOTOGRAPHER"                                                        : ("PHOTOGRAPHER"                                    , "None"                                  , "None"                   , False),
"PHYSICAL THERAPIST"                                                  : ("PHYSICAL THERAPIST"                              , "None"                                  , "None"                   , False),
"PHYSICIAN"                                                           : ("PHYSICIAN"                                       , "None"                                  , "None"                   , False),
"PHYSICIAN'S ASSISTANT"                                               : ("PHYSICIAN"                                       , "ASSISTANT OF"                          , "None"                   , False),
"PHYSICIST"                                                           : ("PHYSICIST"                                       , "None"                                  , "None"                   , False),
"PILOT"                                                               : ("PILOT"                                           , "None"                                  , "None"                   , False),
"PIPE CAULKER"                                                        : ("PIPE CAULKER"                                    , "None"                                  , "None"                   , False),
"PLAN EXAMINER"                                                       : ("PLAN EXAMINER"                                   , "None"                                  , "None"                   , False),
"PLANNER: PRODUCTION CNTRL & SCHEDULING"                              : ("PLANNER"                                         , "PRODUCTION CNTRL & SCHEDULING"         , "None"                   , False),
"PLANT MAINTAINER"                                                    : ("PLANT MAINTAINER"                                , "None"                                  , "None"                   , False),
"PLASTERER"                                                           : ("PLASTERER"                                       , "None"                                  , "None"                   , False),
"PLAYGROUND ASSOCIATE"                                                : ("PLAYGROUND ASSOCIATE"                            , "None"                                  , "None"                   , False),
"PLUMBER"                                                             : ("PLUMBER"                                         , "None"                                  , "None"                   , False),
"PLUMBER'S HELPER"                                                    : ("PLUMBER"                                         , "HELPER OF"                             , "None"                   , False),
"POISON INFORMATION SPECIALIST"                                       : ("POISON INFORMATION SPECIALIST"                   , "None"                                  , "None"                   , False),
"POLICE ADMINISTRATIVE AIDE"                                          : ("POLICE ADMINISTRATIVE AIDE"                      , "None"                                  , "None"                   , False),
"POLICE ATTENDANT"                                                    : ("POLICE ATTENDANT"                                , "None"                                  , "None"                   , False),
"POLICE CADET"                                                        : ("POLICE CADET"                                    , "None"                                  , "None"                   , False),
"POLICE CAPTAIN"                                                      : ("POLICE CAPTAIN"                                  , "None"                                  , "None"                   , False),
"POLICE COMMUNICATIONS TECHNICIAN"                                    : ("POLICE COMMUNICATIONS TECHNICIAN"                , "None"                                  , "None"                   , False),
"POLICE OFFICER D/A DETECTIVE 1ST GR"                                 : ("POLICE OFFICER"                                  , "D/A DETECTIVE"                         , "GRADE 1"                , False),
"POLICE OFFICER D/A DETECTIVE 2ND GR"                                 : ("POLICE OFFICER"                                  , "D/A DETECTIVE"                         , "GRADE 2"                , False),
"POLICE OFFICER"                                                      : ("POLICE OFFICER"                                  , "None"                                  , "None"                   , False),
"POLICE OFFICER, SPECIAL ASSIGNMENT"                                  : ("POLICE OFFICER"                                  , "SPECIAL ASSIGNMENT"                    , "None"                   , False),
"POLICE SURGEON"                                                      : ("POLICE SURGEON"                                  , "None"                                  , "None"                   , False),
"PORT MARINE ENGINEER"                                                : ("MARINE ENGINEER"                                 , "PORT"                                  , "None"                   , False),
"PRE-SES ED PARA"                                                     : ("EDUCATION PARA"                                  , "PRE-SES"                               , "None"                   , False),
"PRECINCT COMMUNITY RELATIONS AIDE"                                   : ("COMMUNITY RELATIONS AIDE"                        , "None"                                  , "None"                   , False),
"PRECINCT RECEPTIONIST"                                               : ("RECEPTIONIST"                                    , "PRECINCT"                              , "None"                   , False),
"PRESIDENT"                                                           : ("PRESIDENT"                                       , "None"                                  , "None"                   , True),
"PRESS AND SPEECH AIDE"                                               : ("PRESS AND SPEECH AIDE"                           , "None"                                  , "None"                   , False),
"PRESS OFFICER"                                                       : ("PRESS OFFICER"                                   , "None"                                  , "None"                   , False),
"PRINCIPAL ACCOUNTANT INVESTIGATOR"                                   : ("ACCOUNTANT INVESTIGATOR"                         , "None"                                  , "PRINCIPAL"              , False),
"PRINCIPAL ADMINISTRATIVE ASSOCIATE - LEV 1 & 2 NON SUPVR"            : ("ADMINISTRATIVE ASSOCIATE"                        , "None"                                  , "PRINCIPAL"              , False),
"PRINCIPAL ADMINISTRATIVE ASSOCIATE"                                  : ("ADMINISTRATIVE ASSOCIATE"                        , "None"                                  , "PRINCIPAL"              , False),
"PRINCIPAL APPRAISER"                                                 : ("APPRAISER"                                       , "None"                                  , "PRINCIPAL"              , False),
"PRINCIPAL ASSIGNED"                                                  : ("None"                                            , "None"                                  , "PRINCIPAL"              , False),
"PRINCIPAL CUSTODIAL SUPERVISOR"                                      : ("CUSTODIAL SUPERVISOR"                            , "None"                                  , "PRINCIPAL"              , False),
"PRINCIPAL FINGERPRINT TECHNICIAN"                                    : ("FINGERPRINT TECHNICIAN"                          , "None"                                  , "PRINCIPAL"              , False),
"PRINCIPAL HUMAN RIGHTS SPECIALIST"                                   : ("HUMAN RIGHTS SPECIALIST"                         , "None"                                  , "PRINCIPAL"              , False),
"PRINCIPAL INVESTIGATOR"                                              : ("INVESTIGATOR"                                    , "None"                                  , "PRINCIPAL"              , False),
"PRINCIPAL INVESTMENT OFFICER"                                        : ("INVESTMENT OFFICER"                              , "None"                                  , "PRINCIPAL"              , False),
"PRINCIPAL LEGISLATIVE FINANCIAL ANALYST"                             : ("LEGISLATIVE FINANCIAL ANALYST"                   , "None"                                  , "PRINCIPAL"              , False),
"PRINCIPAL MULTIPLE DWELLING SPECIALIST"                              : ("MULTIPLE DWELLING SPECIALIST"                    , "None"                                  , "PRINCIPAL"              , False),
"PRINCIPAL NUTRITION CONSULTANT"                                      : ("NUTRITION CONSULTANT"                            , "None"                                  , "PRINCIPAL"              , False),
"PRINCIPAL PARK SUPERVISOR"                                           : ("PARK SUPERVISOR"                                 , "None"                                  , "PRINCIPAL"              , False),
"PRINCIPAL PLANNING CONSULTANT"                                       : ("PLANNING CONSULTANT"                             , "None"                                  , "PRINCIPAL"              , False),
"PRINCIPAL POLICE COMMUNICATION TECHNICIAN"                           : ("POLICE COMMUNICATIONS TECHNICIAN"                , "None"                                  , "PRINCIPAL"              , False),
"PRINCIPAL SCHOOL-NEIGHBORHOOD WORKER"                                : ("SCHOOL-NEIGHBORHOOD WORKER"                      , "None"                                  , "PRINCIPAL"              , False),
"PRINCIPAL TITLE EXAMINER"                                            : ("TITLE EXAMINER"                                  , "None"                                  , "PRINCIPAL"              , False),
"PRINCIPAL"                                                           : ("PRINCIPAL"                                       , "None"                                  , "None"                   , False),
"PRINTING PRESS OPERATOR"                                             : ("PRINTING PRESS OPERATOR"                         , "None"                                  , "None"                   , False),
"PRIVATE SECRETARY"                                                   : ("SECRETARY"                                       , "PRIVATE"                               , "None"                   , False),
"PROBATION ASSISTANT"                                                 : ("PROBATION ASSISTANT"                             , "None"                                  , "None"                   , False),
"PROBATION OFFICER TRAINEE"                                           : ("PROBATION OFFICER"                               , "TRAINEE"                               , "None"                   , False),
"PROBATION OFFICER"                                                   : ("PROBATION OFFICER"                               , "None"                                  , "None"                   , False),
"PROCUREMENT ANALYST"                                                 : ("PROCUREMENT ANALYST"                             , "None"                                  , "None"                   , False),
"PROCURMENT ANALYST LEVEL III"                                        : ("PROCURMENT ANALYST"                              , "None"                                  , "LEVEL III"              , False),
"PROCURMENT ANALYST LEVEL III, OJ"                                    : ("PROCURMENT ANALYST"                              , "None"                                  , "LEVEL III"              , False),
"PROFESSOR"                                                           : ("PROFESSOR"                                       , "None"                                  , "None"                   , False),
"PROGRAM EVALUATOR"                                                   : ("PROGRAM EVALUATOR"                               , "None"                                  , "None"                   , False),
"PROGRAM OFFICER"                                                     : ("PROGRAM OFFICER"                                 , "None"                                  , "None"                   , False),
"PROGRAM PRODUCER"                                                    : ("PROGRAM PRODUCER"                                , "None"                                  , "None"                   , False),
"PROGRAM RESEARCH ANALYST TO THE PUBLIC ADVOCATE"                     : ("PROGRAM RESEARCH ANALYST"                        , "TO THE PUBLIC ADVOCATE"                , "None"                   , False),
"PROGRAM SPECIALIST  CORRECTION"                                      : ("PROGRAM SPECIALIST"                              , "CORRECTION"                            , "None"                   , False),
"PROJECT COORDINATOR"                                                 : ("PROJECT COORDINATOR"                             , "None"                                  , "None"                   , False),
"PROJECT MANAGER INTERN#"                                             : ("PROJECT MANAGER"                                 , "INTERN"                                , "None"                   , False),
"PROJECT MANAGER"                                                     : ("PROJECT MANAGER"                                 , "None"                                  , "None"                   , True),
"PROJECT PLANNER"                                                     : ("PROJECT PLANNER"                                 , "None"                                  , "None"                   , False),
"PROTECTION AGENT"                                                    : ("PROTECTION AGENT"                                , "None"                                  , "None"                   , False),
"PSYCHIATRIST"                                                        : ("PSYCHIATRIST"                                    , "None"                                  , "None"                   , False),
"PSYCHOLOGIST IN TRAIN - REG SUB"                                     : ("PSYCHOLOGIST"                                    , "IN TRAIN"                              , "None"                   , False),
"PSYCHOLOGIST IN TRAINING"                                            : ("PSYCHOLOGIST"                                    , "IN TRAINING"                           , "None"                   , False),
"PSYCHOLOGIST"                                                        : ("PSYCHOLOGIST"                                    , "None"                                  , "None"                   , False),
"PUBLIC ADMINISTRATOR"                                                : ("PUBLIC ADMINISTRATOR"                            , "None"                                  , "None"                   , False),
"PUBLIC ADVOCATE"                                                     : ("PUBLIC ADVOCATE"                                 , "None"                                  , "None"                   , False),
"PUBLIC HEALTH ADVISER"                                               : ("PUBLIC HEALTH ADVISER"                           , "None"                                  , "None"                   , False),
"PUBLIC HEALTH ASSISTANT"                                             : ("PUBLIC HEALTH ASSISTANT"                         , "None"                                  , "None"                   , False),
"PUBLIC HEALTH EDUCATION TRAINEE"                                     : ("PUBLIC HEALTH EDUCATION TRAINEE"                 , "None"                                  , "None"                   , False),
"PUBLIC HEALTH EDUCATOR"                                              : ("PUBLIC HEALTH EDUCATOR"                          , "None"                                  , "None"                   , False),
"PUBLIC HEALTH EMERGENCY PREPAREDNESS SPECIALIST"                     : ("PUBLIC HEALTH EMERGENCY PREPAREDNESS SPECIALIST" , "None"                                  , "None"                   , False),
"PUBLIC HEALTH EPIDEMIOLOGIST"                                        : ("EPIDEMIOLOGIST"                                  , "PUBLIC HEALTH"                         , "None"                   , False),
"PUBLIC HEALTH NURSE"                                                 : ("PUBLIC HEALTH NURSE"                             , "None"                                  , "None"                   , False),
"PUBLIC HEALTH SANITARIAN"                                            : ("PUBLIC HEALTH SANITARIAN"                        , "None"                                  , "None"                   , False),
"PUBLIC HEALTH/PREVENTIVE MEDICINE RESIDENT"                          : ("PUBLIC HEALTH/PREVENTIVE MEDICINE RESIDENT"      , "None"                                  , "None"                   , False),
"PUBLIC INFORMATION OFFICER"                                          : ("PUBLIC INFORMATION OFFICER"                      , "None"                                  , "None"                   , False),
"PUBLIC RECORDS AIDE"                                                 : ("PUBLIC RECORDS AIDE"                             , "None"                                  , "None"                   , False),
"PUBLIC RECORDS OFFICER"                                              : ("PUBLIC RECORDS OFFICER"                          , "None"                                  , "None"                   , False),
"PUBLIC RELATIONS ADVISER"                                            : ("PUBLIC RELATIONS ADVISER"                        , "None"                                  , "None"                   , False),
"PUBLIC RELATIONS ASSISTANT"                                          : ("PUBLIC RELATIONS"                                , "ASSISTANT"                             , "None"                   , False),
"PUBLIC SERVICE INTERNE"                                              : ("INTERNE"                                         , "PUBLIC SERVICE"                        , "None"                   , False),
"PUBLISHING ASSOCIATE"                                                : ("PUBLISHING ASSOCIATE"                            , "None"                                  , "None"                   , False),
"PUPPETEER"                                                           : ("PUPPETEER"                                       , "None"                                  , "None"                   , False),
"PURCHASING AGENT"                                                    : ("PURCHASING AGENT"                                , "None"                                  , "None"                   , False),
"QUALITY ASSURANCE SPECIALIST TRAINEE"                                : ("QUALITY ASSURANCE SPECIALIST"                    , "TRAINEE"                               , "None"                   , False),
"QUALITY ASSURANCE SPECIALIST"                                        : ("QUALITY ASSURANCE SPECIALIST"                    , "None"                                  , "None"                   , False),
"RACKETS INVESTIGATOR"                                                : ("INVESTIGATOR"                                    , "RACKETS"                               , "None"                   , False),
"RADIO AND TELEVISION OPERATOR"                                       : ("RADIO AND TELEVISION OPERATOR"                   , "None"                                  , "None"                   , False),
"RADIO AND TEVEVISION OPERATOR"                                       : ("RADIO AND TEVEVISION OPERATOR"                   , "None"                                  , "None"                   , False),
"RADIO REPAIR MECHANIC"                                               : ("RADIO REPAIR MECHANIC"                           , "None"                                  , "None"                   , False),
"REAL PROPERTY ASSISTANT"                                             : ("REAL PROPERTY ASSISTANT"                         , "None"                                  , "None"                   , False),
"REAL PROPERTY MANAGER"                                               : ("REAL PROPERTY MANAGER"                           , "None"                                  , "None"                   , False),
"RECREATION DIRECTOR"                                                 : ("RECREATION DIRECTOR"                             , "None"                                  , "None"                   , True),
"RECREATION SPECIALIST"                                               : ("RECREATION SPECIALIST"                           , "None"                                  , "None"                   , False),
"RECREATION SUPERVISOR"                                               : ("RECREATION SUPERVISOR"                           , "None"                                  , "None"                   , True),
"REGIONAL INSTRUCTIONAL SUPERINTENDENT"                               : ("INSTRUCTIONAL SUPERINTENDENT"                    , "REGIONAL"                              , "None"                   , True),
"REPAIR CREW CHIEF"                                                   : ("REPAIR CREW CHIEF"                               , "None"                                  , "None"                   , True),
"REPAIR CREW WORKER"                                                  : ("REPAIR CREW WORKER"                              , "None"                                  , "None"                   , False),
"REPAIR SHOP MANAGER"                                                 : ("REPAIR SHOP MANAGER"                             , "None"                                  , "None"                   , True),
"REPORTER/ STENOGRAPHER"                                              : ("REPORTER"                                        , "STENOGRAPHER"                          , "None"                   , False),
"RESEARCH & LIAISON COORDINATOR"                                      : ("RESEARCH & LIAISON COORDINATOR"                  , "None"                                  , "None"                   , False),
"RESEARCH ANALYST TO THE PUBLIC ADVOCATE"                             : ("PUBLIC ADVOCATE"                                 , "RESEARCH ANALYST OF"                   , "None"                   , False),
"RESEARCH AND LIAISON COORDINATOR"                                    : ("RESEARCH AND LIAISON COORDINATOR"                , "None"                                  , "None"                   , False),
"RESEARCH AND LIAISON SPECIALIST"                                     : ("RESEARCH AND LIAISON SPECIALIST"                 , "None"                                  , "None"                   , False),
"RESEARCH ASSISTANT BEHAV SCI"                                        : ("RESEARCH ASSISTANT"                              , "BEHAV SCI"                             , "None"                   , False),
"RESEARCH ASSISTANT"                                                  : ("RESEARCH ASSISTANT"                              , "None"                                  , "None"                   , False),
"RESEARCH LIAISON ADN GOVERNMENTAL COORDINATOR"                       : ("RESEARCH LIAISON"                                , "ADN GOVERNMENTAL COORDINATOR"          , "None"                   , False),
"RESEARCH PROJECTS COORD"                                             : ("RESEARCH PROJECTS COORD"                         , "None"                                  , "None"                   , False),
"RESEARCH PROJECTS COORDINATOR"                                       : ("RESEARCH PROJECTS COORDINATOR"                   , "None"                                  , "None"                   , False),
"RESEARCH SCIENTIST"                                                  : ("RESEARCH SCIENTIST"                              , "None"                                  , "None"                   , False),
"RESIDENT BUILDING SUPERINTENDENT"                                    : ("BUILDING SUPERINTENDENT"                         , "RESIDENT"                              , "None"                   , True),
"RIGGER"                                                              : ("RIGGER"                                          , "None"                                  , "None"                   , False),
"ROOFER"                                                              : ("ROOFER"                                          , "None"                                  , "None"                   , False),
"RUBBER TIRE REPAIRER"                                                : ("RUBBER TIRE REPAIRER"                            , "None"                                  , "None"                   , False),
"SAFETY SPECIALIST"                                                   : ("SAFETY SPECIALIST"                               , "None"                                  , "None"                   , False),
"SANITATION COMPLIANCE AGENT"                                         : ("SANITATION COMPLIANCE AGENT"                     , "None"                                  , "None"                   , False),
"SANITATION ENFORCEMENT AGENT"                                        : ("SANITATION ENFORCEMENT AGENT"                    , "None"                                  , "None"                   , False),
"SANITATION WORKER"                                                   : ("SANITATION WORKER"                               , "None"                                  , "None"                   , False),
"SCHOOL BUSINESS MANAGER"                                             : ("SCHOOL BUSINESS MANAGER"                         , "None"                                  , "None"                   , True),
"SCHOOL COMPUTER TECHNOLOGY SPECIALIST"                               : ("COMPUTER TECHNOLOGY SPECIALIST"                  , "SCHOOL"                                , "None"                   , False),
"SCHOOL CROSSING GUARD"                                               : ("SCHOOL CROSSING GUARD"                           , "None"                                  , "None"                   , False),
"SCHOOL EQUIPMENT MAINTAINER"                                         : ("SCHOOL EQUIPMENT MAINTAINER"                     , "None"                                  , "None"                   , False),
"SCHOOL FOOD SERVICE MANAGER"                                         : ("FOOD SERVICE MANAGER"                            , "SCHOOL"                                , "None"                   , True),
"SCHOOL LUNCH AIDE"                                                   : ("SCHOOL LUNCH AIDE"                               , "None"                                  , "None"                   , False),
"SCHOOL LUNCH ASSISTANT COOK"                                         : ("SCHOOL COOK"                                     , "ASSISTANT"                             , "None"                   , False),
"SCHOOL LUNCH ASSISTANT"                                              : ("SCHOOL LUNCH ASSISTANT"                          , "None"                                  , "None"                   , False),
"SCHOOL LUNCH HELPER"                                                 : ("SCHOOL LUNCH HELPER"                             , "None"                                  , "None"                   , False),
"SCHOOL LUNCH LOADER AND HANDLER"                                     : ("SCHOOL LUNCH LOADER AND HANDLER"                 , "None"                                  , "None"                   , False),
"SCHOOL MEDICAL INSPECTOR - REG SUB"                                  : ("SCHOOL MEDICAL INSPECTOR"                        , "REG SUB"                               , "None"                   , False),
"SCHOOL MEDICAL INSPECTOR"                                            : ("MEDICAL INSPECTOR"                               , "SCHOOL"                                , "None"                   , False),
"SCHOOL PLANT MANAGER"                                                : ("SCHOOL PLANT MANAGER"                            , "None"                                  , "None"                   , True),
"SCHOOL PSYCHIATRIST"                                                 : ("SCHOOL PSYCHIATRIST"                             , "None"                                  , "None"                   , False),
"SCHOOL PSYCHOLGIST"                                                  : ("SCHOOL PSYCHOLOGIST"                             , "None"                                  , "None"                   , False),
"SCHOOL PSYCHOLOGIST - REG SUB"                                       : ("SCHOOL PSYCHOLOGIST"                             , "REG SUB"                               , "None"                   , False),
"SCHOOL PSYCHOLOGIST"                                                 : ("SCHOOL PSYCHOLOGIST"                             , "None"                                  , "None"                   , False),
"SCHOOL SAFETY AGENT"                                                 : ("SCHOOL SAFETY AGENT"                             , "None"                                  , "None"                   , False),
"SCHOOL SECRETARY PER SESSION"                                        : ("SCHOOL SECRETARY"                                , "PER SESSION"                           , "None"                   , False),
"SCHOOL SECRETARY"                                                    : ("SCHOOL SECRETARY"                                , "None"                                  , "None"                   , False),
"SCHOOL SECRETARY-REG SUB"                                            : ("SCHOOL SECRETARY"                                , "REG SUB"                               , "None"                   , False),
"SCHOOL SOCIAL WORKER - REG SUB"                                      : ("SCHOOL SOCIAL WORKER"                            , "REG SUB"                               , "None"                   , False),
"SCHOOL SOCIAL WORKER"                                                : ("SCHOOL SOCIAL WORKER"                            , "None"                                  , "None"                   , False),
"SCHOOL-NEIGHBORHOOD WORKER"                                          : ("SCHOOL-NEIGHBORHOOD WORKER"                      , "None"                                  , "None"                   , False),
"SCIENTIST"                                                           : ("SCIENTIST"                                       , "None"                                  , "None"                   , False),
"SECOND DEPUTY COMPTROLLER"                                           : ("DEPUTY COMPTROLLER"                              , "SECOND"                                , "None"                   , False),
"SECRETARY OF COMM"                                                   : ("SECRETARY"                                       , "OF COMM"                               , "None"                   , False),
"SECRETARY OF COMMISSIONER"                                           : ("COMMISSIONER"                                    , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY OF THE AUTHORITY"                                          : ("AUTHORITY"                                       , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY OF THE COMMISSION"                                         : ("COMMISSION"                                      , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY OF THE DEPARTMENT"                                         : ("DEPARTMENT HEAD"                                 , "SECRETARY OF"                          , "None"                   , True),
"SECRETARY OF THE TAX COMMISSION"                                     : ("TAX COMMISSION"                                  , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO ASSISTANT TO PRESIDENT"                                 : ("PRESIDENT"                                       , "SECRETARY TO ASSISTANT OF"             , "None"                   , False),
"SECRETARY TO COMMISSIONER OF BLDGS"                                  : ("COMMISSIONER OF BLDGS"                           , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO COMMISSIONER"                                           : ("COMMISSIONER"                                    , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO COMMUNITY SCHOOL BOARD"                                 : ("COMMUNITY SCHOOL BOARD"                          , "SECRETARY TO"                          , "None"                   , False),
"SECRETARY TO CONFLICTS OF INTEREST BOARD"                            : ("SECRETARY"                                       , "TO CONFLICTS OF INTEREST BOARD"        , "None"                   , False),
"SECRETARY TO DEPARTMENT"                                             : ("DEPARTMENT HEAD"                                 , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO DEPT CHANCELLOR"                                        : ("DEPT CHANCELLOR"                                 , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO DEPUTY COMMISSIONER"                                    : ("DEPUTY COMMISSIONER"                             , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO DEPUTY EXECUTIVE DIRECTOR"                              : ("DEPUTY EXECUTIVE DIRECTOR"                       , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO DISTRICT ATTORNEY"                                      : ("DISTRICT ATTORNEY"                               , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO MEMBER"                                                 : ("MEMBER"                                          , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO PUBLIC ADMINISTRATOR"                                   : ("PUBLIC ADMINISTRATOR"                            , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO THE BOARD OF CORRECTION"                                : ("BOARD OF CORRECTION"                             , "SECRETARY TO"                          , "None"                   , False),
"SECRETARY TO THE CHAIR"                                              : ("CHAIR"                                           , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO THE CHAIR, LANDMARKS PRESERVATION COMMISSION"           : ("LANDMARKS PRESERVATION COMMISSION"               , "SECRETARY TO THE CHAIR"                , "None"                   , False),
"SECRETARY TO THE CHAIRMAN"                                           : ("CHAIRMAN"                                        , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO THE CHANCELLOR"                                         : ("CHANCELLOR"                                      , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO THE CHIEF ACTUARY"                                      : ("CHIEF ACTUARY"                                   , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO THE COMMISSIONER OF BLDGS"                              : ("COMMISSIONER OF BLDGS"                           , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO THE COMMISSIONER OF DDC"                                : ("COMMISSIONER OF DDC"                             , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO THE COMMISSIONER OF LABOR RELATIONS"                    : ("COMMISSIONER OF LABOR RELATIONS"                 , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO THE COMMISSIONER"                                       : ("COMMISSIONER"                                    , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO THE COMPTROLLER"                                        : ("COMPTROLLER"                                     , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO THE CORPORATION COUNSEL"                                : ("CORPORATION COUNSEL"                             , "SECRETARY TO"                          , "None"                   , False),
"SECRETARY TO THE COUNSEL TO THE CHANCELLOR"                          : ("COUNSEL TO THE CHANCELLOR"                       , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO THE DEPARTMENT"                                         : ("DEPARTMENT HEAD"                                 , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO THE DEPUTY BORO PRESIDENT"                              : ("DEPUTY BORO PRESIDENT"                           , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO THE DEPUTY CHAIR"                                       : ("DEPUTY CHAIR"                                    , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO THE DEPUTY CHANCELLOR"                                  : ("DEPUTY CHANCELLOR"                               , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO THE DEPUTY COMMISSIONER"                                : ("DEPUTY COMMISSIONER"                             , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO THE EXCUTIVE ASSISTANT"                                 : ("EXCUTIVE ASSISTANT"                              , "SECRETARY TO"                          , "None"                   , False),
"SECRETARY TO THE EXEC DEPUTY COMMISSIONER"                           : ("EXEC DEPUTY COMMISSIONER"                        , "SECRETARY TO"                          , "None"                   , False),
"SECRETARY TO THE EXECUTIVE DIRECTOR OF RETIREMENT SYSTEM"            : ("EXECUTIVE DIRECTOR OF RETIREMENT SYSTEM"         , "SECRETARY TO"                          , "None"                   , False),
"SECRETARY TO THE EXECUTIVE DIRECTOR"                                 : ("EXECUTIVE DIRECTOR"                              , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO THE FIRE COMMISSIONER"                                  : ("FIRE COMMISSIONER"                               , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO THE FIRST ASSISTANT CORPORATION COUNSEL"                : ("ASSISTANT CORPORATION COUNSEL"                   , "SECRETARY TO THE"                      , "FIRST"                  , False),
"SECRETARY TO THE FIRST DEPUTY COMMISSIONER"                          : ("FIRST DEPUTY COMMISSIONER"                       , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO THE FIRST DEPUTY COMMISSIONER-PD"                       : ("FIRST DEPUTY COMMISSIONER"                       , "SECRETARY TO"                          , "None"                   , False),
"SECRETARY TO THE PRESIDENT"                                          : ("PRESIDENT"                                       , "SECRETARY OF"                          , "None"                   , False),
"SECRETARY TO THE SPECIAL COMMISSIONER INVESTIGATION NYC SCH"         : ("SPECIAL COMMISSIONER"                            , "SECRETARY TO"                          , "None"                   , False),
"SECRETARY"                                                           : ("SECRETARY"                                       , "None"                                  , "None"                   , False),
"SECY TO THE DIRECTOR OF COLLECTIVE BARGAINING"                       : ("DIRECTOR OF COLLECTIVE BARGAINING"               , "SECY TO"                               , "None"                   , False),
"SENIOR ACCOUNTANT INVESTIGATOR"                                      : ("ACCOUNTANT INVESTIGATOR"                         , "None"                                  , "SENIOR"                 , False),
"SENIOR ADMINISRATOR"                                                 : ("ADMINISTRATOR"                                   , "None"                                  , "SENIOR"                 , True),
"SENIOR ADVISER TO THE COMMISSIONER"                                  : ("COMMISSIONER"                                    , "ADVISER"                               , "SENIOR"                 , False),
"SENIOR AUTOMOTIVE SERVICE WORKER"                                    : ("AUTOMOTIVE SERVICE WORKER"                       , "None"                                  , "SENIOR"                 , False),
"SENIOR AUTOMOTIVE SPECIALIST"                                        : ("AUTOMOTIVE SPECIALIST"                           , "None"                                  , "SENIOR"                 , False),
"SENIOR BAKER"                                                        : ("BAKER"                                           , "None"                                  , "SENIOR"                 , False),
"SENIOR COLLEGE LAB TECH"                                             : ("COLLEGE LAB TECH"                                , "None"                                  , "SENIOR"                 , False),
"SENIOR COMPUTER PROGRAMMER"                                          : ("COMPUTER PROGRAMMER"                             , "None"                                  , "SENIOR"                 , False),
"SENIOR CONSULTANT"                                                   : ("CONSULTANT"                                      , "None"                                  , "SENIOR"                 , False),
"SENIOR COOK"                                                         : ("COOK"                                            , "None"                                  , "SENIOR"                 , False),
"SENIOR CUSTODIAL SUPERVISOR"                                         : ("CUSTODIAL SUPERVISOR"                            , "None"                                  , "SENIOR"                 , True),
"SENIOR DETECTIVE INVESTIGATOR"                                       : ("DETECTIVE INVESTIGATOR"                          , "None"                                  , "SENIOR"                 , False),
"SENIOR ESTIMATOR"                                                    : ("ESTIMATOR"                                       , "None"                                  , "SENIOR"                 , False),
"SENIOR EXECUTIVE SECRETARY"                                          : ("EXECUTIVE SECRETARY"                             , "None"                                  , "SENIOR"                 , False),
"SENIOR FIELD SUPERVISOR"                                             : ("FIELD SUPERVISOR"                                , "None"                                  , "SENIOR"                 , True),
"SENIOR HOUSEKEEPER"                                                  : ("HOUSEKEEPER"                                     , "None"                                  , "SENIOR"                 , False),
"SENIOR INSTITUTIONAL TRADES INSTRUCTOR"                              : ("INSTITUTIONAL TRADES INSTRUCTOR"                 , "None"                                  , "SENIOR"                 , False),
"SENIOR INTERGROUP RELATIONS OFFICER"                                 : ("INTERGROUP RELATIONS OFFICER"                    , "None"                                  , "SENIOR"                 , False),
"SENIOR IT ARCHITECT"                                                 : ("IT ARCHITECT"                                    , "None"                                  , "SENIOR"                 , False),
"SENIOR LEGISLATIVE FINANCIAL ANALYST"                                : ("LEGISLATIVE FINANCIAL ANALYST"                   , "None"                                  , "SENIOR"                 , False),
"SENIOR LEGISLATIVE POLICY ANALYST"                                   : ("LEGISLATIVE POLICY ANALYST"                      , "None"                                  , "SENIOR"                 , False),
"SENIOR MANAGEMENT CONSULTANT"                                        : ("MANAGEMENT CONSULTANT"                           , "None"                                  , "SENIOR"                 , True),
"SENIOR MEDICAL SPECIALIST"                                           : ("MEDICAL SPECIALIST"                              , "None"                                  , "SENIOR"                 , False),
"SENIOR MENTAL HEALTH WORKER"                                         : ("MENTAL HEALTH WORKER"                            , "None"                                  , "SENIOR"                 , False),
"SENIOR OCCUPATIONAL THERAPIST"                                       : ("OCCUPATIONAL THERAPIST"                          , "None"                                  , "SENIOR"                 , False),
"SENIOR OFFICE APPLIANCE MAINTAINER"                                  : ("OFFICE APPLIANCE MAINTAINER"                     , "None"                                  , "SENIOR"                 , False),
"SENIOR PHOTOGRAPHER"                                                 : ("PHOTOGRAPHER"                                    , "None"                                  , "SENIOR"                 , False),
"SENIOR PHYSICAL THERAPIST"                                           : ("PHYSICAL THERAPIST"                              , "None"                                  , "SENIOR"                 , False),
"SENIOR POLICE ADMINISTRATIVE AIDE"                                   : ("POLICE ADMINISTRATIVE AIDE"                      , "None"                                  , "SENIOR"                 , False),
"SENIOR RACKETS INVESTIGATOR"                                         : ("RACKETS INVESTIGATOR"                            , "None"                                  , "SENIOR"                 , False),
"SENIOR REGISTRAR"                                                    : ("REGISTRAR"                                       , "None"                                  , "SENIOR"                 , False),
"SENIOR REPAIR CREW CHIEF"                                            : ("REPAIR CREW CHIEF"                               , "None"                                  , "SENIOR"                 , True),
"SENIOR SALVAGE APPRAISER"                                            : ("SALVAGE APPRAISER"                               , "None"                                  , "SENIOR"                 , False),
"SENIOR SCHOOL LUNCH AIDE"                                            : ("SCHOOL LUNCH AIDE"                               , "None"                                  , "SENIOR"                 , False),
"SENIOR SCHOOL-NEIGHBORHOOD WORKER"                                   : ("SCHOOL-NEIGHBORHOOD WORKER"                      , "None"                                  , "SENIOR"                 , False),
"SENIOR SECRETARY"                                                    : ("SECRETARY"                                       , "None"                                  , "SENIOR"                 , False),
"SENIOR SERVICE INSPECTOR"                                            : ("SERVICE INSPECTOR"                               , "None"                                  , "SENIOR"                 , False),
"SENIOR SEWAGE TREATMENT WORKER"                                      : ("SEWAGE TREATMENT WORKER"                         , "None"                                  , "SENIOR"                 , False),
"SENIOR STAFF OFFICER"                                                : ("STAFF OFFICER"                                   , "None"                                  , "SENIOR"                 , False),
"SENIOR STATIONARY ENGINEER"                                          : ("STATIONARY ENGINEER"                             , "None"                                  , "SENIOR"                 , False),
"SENIOR STUDENT LEGAL SPECIALIST"                                     : ("STUDENT LEGAL SPECIALIST"                        , "None"                                  , "SENIOR"                 , False),
"SENIOR SUPERVISOR COMMUNICATION ELECTRICIAN"                         : ("COMMUNICATION ELECTRICIAN"                       , "None"                                  , "SENIOR"                 , True),
"SENIOR SYSTEMS ANALYST"                                              : ("SYSTEMS ANALYST"                                 , "None"                                  , "SENIOR"                 , False),
"SENIOR SYSTEMS ANALYSTS"                                             : ("SYSTEMS ANALYSTS"                                , "None"                                  , "SENIOR"                 , False),
"SENIOR TITLE EXAMINER"                                               : ("TITLE EXAMINER"                                  , "None"                                  , "SENIOR"                 , False),
"SENIOR VICE PRESIDENT"                                               : ("VICE PRESIDENT"                                  , "None"                                  , "SENIOR"                 , True),
"SERGEANT AT ARMS"                                                    : ("SERGEANT AT ARMS"                                , "None"                                  , "None"                   , False),
"SERGEANT D/A SPECIAL ASSIGNMENT"                                     : ("SERGEANT"                                        , "SPECIAL ASSIGNMENT"                    , "None"                   , False),
"SERGEANT D/A SUPERVISOR DETECTIVE SQUAD"                             : ("SERGEANT"                                        , "D/A SUPERVISOR DETECTIVE SQUAD"        , "None"                   , True),
"SERGEANT"                                                            : ("SERGEANT"                                        , "None"                                  , "None"                   , True),
"SERGEANTD/A SPECIAL ASSIGNMENT"                                      : ("SERGEANT"                                        , "SPECIAL ASSIGNMENT"                    , "None"                   , False),
"SERGEANTD/A SUPERVISOR DETECTIVE SQUAD"                              : ("SERGEANT"                                        , "D/A SUPERVISOR DETECTIVE SQUAD"        , "None"                   , True),
"SERVICE INSPECTOR"                                                   : ("SERVICE INSPECTOR"                               , "None"                                  , "None"                   , False),
"SEWAGE TREATMENT WORKER"                                             : ("SEWAGE TREATMENT WORKER"                         , "None"                                  , "None"                   , False),
"SHEET METAL WORKER"                                                  : ("SHEET METAL WORKER"                              , "None"                                  , "None"                   , False),
"SHIP CARPENTER"                                                      : ("SHIP CARPENTER"                                  , "None"                                  , "None"                   , False),
"SIGN LANGUAGE INTERPRETER 1,2,3-10M"                                 : ("SIGN LANGUAGE INTERPRETER"                       , "None"                                  , "1-10M"                  , False),
"SIGN LANGUAGE INTERPRETER"                                           : ("SIGN LANGUAGE INTERPRETER"                       , "None"                                  , "None"                   , False),
"SOCIAL WORKER"                                                       : ("SOCIAL WORKER"                                   , "None"                                  , "None"                   , False),
"SPACE ANALYST"                                                       : ("SPACE ANALYST"                                   , "None"                                  , "None"                   , False),
"SPEAKER/MAJORITY LEADER"                                             : ("SPEAKER"                                         , "MAJORITY LEADER"                       , "None"                   , True),
"SPEC CONSULTANT"                                                     : ("CONSULTANT"                                      , "SPEC"                                  , "None"                   , False),
"SPECIAL ADVISOR TO CITY CLERK"                                       : ("CITY CLERK"                                      , "SPECIAL ADVISOR OF"                    , "None"                   , False),
"SPECIAL ADVISOR TO THE DIRECTOR"                                     : ("DIRECTOR"                                        , "SPECIAL ADVISOR OF"                    , "None"                   , False),
"SPECIAL ASSISTANT DISTRICT ATTORNEY"                                 : ("DISTRICT ATTORNEY"                               , "ASSISTANT"                             , "SPECIAL"                , False),
"SPECIAL ASSISTANT TO DA"                                             : ("DA"                                              , "SPECIAL ASSISTANT OF"                  , "None"                   , False),
"SPECIAL ASSISTANT TO DEPUTY EXECUTIVE DIRECTOR"                      : ("DEPUTY EXECUTIVE DIRECTOR"                       , "SPECIAL ASSISTANT OF"                  , "None"                   , False),
"SPECIAL ASSISTANT TO THE BOROUGH PRESIDENT"                          : ("BOROUGH PRESIDENT"                               , "SPECIAL ASSISTANT OF"                  , "None"                   , False),
"SPECIAL ASSISTANT TO THE CHANCELLOR"                                 : ("CHANCELLOR"                                      , "ASSISTANT OF"                          , "None"                   , False),
"SPECIAL ASSISTANT TO THE COMMISSIONER"                               : ("COMMISSIONER"                                    , "SPECIAL ASSISTANT OF"                  , "None"                   , False),
"SPECIAL ASSISTANT TO THE DISTRICT ATTORNEY"                          : ("DISTRICT ATTORNEY"                               , "SPECIAL ASSISTANT OF"                  , "None"                   , False),
"SPECIAL ASSISTANT TO THE FIRE COMMISSIONER"                          : ("FIRE COMMISSIONER"                               , "SPECIAL ASSISTANT OF"                  , "None"                   , False),
"SPECIAL ASSISTANT TO THE HUMAN RESOURCES ADMINISTRATOR"              : ("HUMAN RESOURCES ADMINISTRATOR"                   , "SPECIAL ASSISTANT OF"                  , "None"                   , False),
"SPECIAL ASSISTANT"                                                   : ("ASSISTANT"                                       , "None"                                  , "None"                   , False),
"SPECIAL ASST FOR INVESTIGATIONS/INSPECTOR GEN"                       : ("INSPECTOR GENERAL"                               , "SPECIAL ASST FOR INVESTIGATIONS"       , "None"                   , False),
"SPECIAL ASST TO EXEC DIR"                                            : ("EXEC DIR"                                        , "SPECIAL ASST TO"                       , "None"                   , False),
"SPECIAL COMMISSIONER OF INVESTIGATION-NYC SCHOOL DIST. DOI"          : ("COMMISSIONER OF INVESTIGATION"                   , "SPECIAL"                               , "None"                   , True),
"SPECIAL CONSULTANT"                                                  : ("CONSULTANT"                                      , "None"                                  , "None"                   , False),
"SPECIAL DEPUTY COMPTROLLER"                                          : ("COMPTROLLER"                                     , "SPECIAL DEPUTY"                        , "None"                   , False),
"SPECIAL EXAMINER"                                                    : ("EXAMINER"                                        , "SPECIAL"                               , "None"                   , False),
"SPECIAL INVESTIGATOR"                                                : ("INVESTIGATOR"                                    , "SPECIAL"                               , "None"                   , False),
"SPECIAL OFFICER"                                                     : ("SPECIAL OFFICER"                                 , "None"                                  , "None"                   , False),
"SPVSG FAD-ASST COMMISS DETAIL"                                       : ("COMMISSIONER"                                    , "ASST DETAIL"                           , "None"                   , False),
"SPVSG FAD-DEP DIR DSPTCH DTAIL"                                      : ("DIRECTOR"                                        , "FAD-DEP DSPTCH"                        , "None"                   , True),
"SPVSG FAD-DIR DSPTCH OP DETAIL"                                      : ("OPERATIONS DETAIL"                               , "SPVSG FAD-DIR DSPTCH"                  , "None"                   , True),
"SR ASSOCIATE HEALTH PROGRAM DIRECTOR"                                : ("HEALTH PROGRAM DIRECTOR"                         , "SR ASSOCIATE"                          , "None"                   , False),
"SR COLLEGE LABORATORY TECH"                                          : ("COLLEGE LABORATORY TECH"                         , "None"                                  , "SENIOR"                 , False),
"SR COMMUNITY ORGANIZATION SPECIALIST- URBAN RENEWAL"                 : ("COMMUNITY ORGANIZATION SPECIALIST"               , "URBAN RENEWAL"                         , "SENIOR"                 , False),
"SR HEALTHCARE PROG PLAN ANLYST"                                      : ("HEALTHCARE PROG PLAN ANLYST"                     , "None"                                  , "SENIOR"                 , False),
"SR MANAGEMENT CONSULTANT"                                            : ("MANAGEMENT CONSULTANT"                           , "None"                                  , "SENIOR"                 , True),
"SR PROJECT PLANNER"                                                  : ("PROJECT PLANNER"                                 , "None"                                  , "SENIOR"                 , False),
"SR SYSTEMS ANALYST - EDP"                                            : ("SYSTEMS ANALYST"                                 , "EDP"                                   , "SENIOR"                 , False),
"STAFF ANALYST TRAINEE"                                               : ("STAFF ANALYST"                                   , "TRAINEE"                               , "None"                   , False),
"STAFF ANALYST"                                                       : ("STAFF ANALYST"                                   , "None"                                  , "None"                   , False),
"STAFF ASSISTANT"                                                     : ("STAFF ASSISTANT"                                 , "None"                                  , "None"                   , False),
"STAFF AUDIOLOGIST"                                                   : ("AUDIOLOGIST"                                     , "STAFF"                                 , "None"                   , False),
"STAFF NURSE"                                                         : ("STAFF NURSE"                                     , "None"                                  , "None"                   , False),
"STATIONARY ENGINEER"                                                 : ("STATIONARY ENGINEER"                             , "None"                                  , "None"                   , False),
"STATISTICAL SECRETARY"                                               : ("SECRETARY"                                       , "STATISTICAL"                           , "None"                   , False),
"STATISTICIAN"                                                        : ("STATISTICIAN"                                    , "None"                                  , "None"                   , False),
"STEAM FITTER"                                                        : ("STEAM FITTER"                                    , "None"                                  , "None"                   , False),
"STEAM FITTER'S HELPER"                                               : ("STEAM FITTER"                                    , "HELPER"                                , "None"                   , False),
"STENOGRAPHER TO EACH DEPUTY COMMISSIONER"                            : ("DEPUTY COMMISSIONER"                             , "STENOGRAPHER"                          , "None"                   , False),
"STENOGRAPHER TO THE CORPORATION COUNSEL"                             : ("CORPORATION COUNSEL"                             , "STENOGRAPHER OF"                       , "None"                   , False),
"STENOGRAPHIC SPECIALIST"                                             : ("STENOGRAPHIC SPECIALIST"                         , "None"                                  , "None"                   , False),
"STENOGRAPHIC/SECRETARIAL ASSOCIATE"                                  : ("STENOGRAPHIC/SECRETARIAL ASSOCIATE"              , "None"                                  , "None"                   , False),
"STOCK WORKER"                                                        : ("STOCK WORKER"                                    , "None"                                  , "None"                   , False),
"STRATEGI INITIATIVE SPECIALIST"                                      : ("INITIATIVE SPECIALIST"                           , "STRATEGI"                              , "None"                   , False),
"STRATEGIC INITIATIVE SPECIALIST"                                     : ("STRATEGIC INITIATIVE SPECIALIST"                 , "None"                                  , "None"                   , False),
"STUDENT AIDE VOCATIONAL"                                             : ("STUDENT AIDE"                                    , "VOCATIONAL"                            , "None"                   , False),
"STUDENT AIDE"                                                        : ("STUDENT AIDE"                                    , "None"                                  , "None"                   , False),
"STUDENT LEGAL SPECIALIST"                                            : ("LEGAL SPECIALIST"                                , "STUDENT"                               , "None"                   , False),
"SUBSTANCE ABUSE PREVENTION & INTERVENTION SPECIALIST"                : ("SUBSTANCE ABUSE PREVENTION & INTERVENTION SPECIALIST", "None"                                  , "None"                   , False),
"SUBSTITUTE ED PARA"                                                  : ("ED PARA"                                         , "SUBSTITUTE"                            , "None"                   , False),
"SUBSTITUTE HEALTH SVC. AIDE"                                         : ("HEALTH SVC. AIDE"                                , "SUBSTITUTE"                            , "None"                   , False),
"SUBSTITUTE PRINCIPAL"                                                : ("PRINCIPAL"                                       , "SUBSTITUTE"                            , "None"                   , False),
"SUBSTITUTE RECREATION ASSISTANT"                                     : ("RECREATION ASSISTANT"                            , "SUBSTITUTE"                            , "None"                   , False),
"SUBSTITUTE SCHOOL AIDE"                                              : ("SCHOOL AIDE"                                     , "SUBSTITUTE"                            , "None"                   , False),
"SUBSTITUTE SCHOOL GUARD"                                             : ("SCHOOL GUARD"                                    , "SUBSTITUTE"                            , "None"                   , False),
"SUBSTITUTE SCHOOL LUNCH HELPER"                                      : ("SCHOOL LUNCH HELPER"                             , "SUBSTITUTE"                            , "None"                   , False),
"SUBSTITUTE SR. SCHOOL LUNCH HELPER"                                  : ("SCHOOL LUNCH HELPER"                             , "SUBSTITUTE"                            , "SENIOR"                 , False),
"SUBSTITUTE TEACHER"                                                  : ("TEACHER"                                         , "SUBSTITUTE"                            , "None"                   , False),
"SUMMER COLLEGE INTERN"                                               : ("COLLEGE INTERN"                                  , "SUMMER"                                , "None"                   , False),
"SUMMER GRADUATE INTERN"                                              : ("GRADUATE INTERN"                                 , "SUMMER"                                , "None"                   , False),
"SUPERINTENDENT OF ADULT INSTITUTIONS"                                : ("SUPERINTENDENT"                                  , "OF ADULT INSTITUTIONS"                 , "None"                   , True),
"SUPERINTENDENT OF LAUNDRIES"                                         : ("LAUNDRY SUPERINTENDENT"                          , "None"                                  , "None"                   , True),
"SUPERINTENDENT OF WATER AND SEWER SYSTEMS"                           : ("SUPERINTENDENT"                                  , "OF WATER AND SEWER SYSTEMS"            , "None"                   , True),
"SUPERINTENDENT"                                                      : ("SUPERINTENDENT"                                  , "None"                                  , "None"                   , True),
"SUPERVISING ACCOUNTANT INVESTIGATOR"                                 : ("ACCOUNTANT INVESTIGATOR"                         , "None"                                  , "None"                   , True),
"SUPERVISING BLASTING INSPECTOR"                                      : ("BLASTING INSPECTOR"                              , "None"                                  , "None"                   , True),
"SUPERVISING BOOKBINDER"                                              : ("BOOKBINDER"                                      , "None"                                  , "None"                   , True),
"SUPERVISING CHIEF SURGEON"                                           : ("CHIEF SURGEON"                                   , "None"                                  , "None"                   , True),
"SUPERVISING COMMUNICATION ELECTRICIAN"                               : ("COMMUNICATION ELECTRICIAN"                       , "None"                                  , "None"                   , True),
"SUPERVISING COMPUTER SERVICE TECHNICIAN"                             : ("COMPUTER SERVICE TECHNICIAN"                     , "None"                                  , "None"                   , True),
"SUPERVISING DEMOLITION INSPECTOR"                                    : ("DEMOLITION INSPECTOR"                            , "None"                                  , "None"                   , True),
"SUPERVISING DEPUTY SHERIFF - AL 1 ONLY 40 HR"                        : ("DEPUTY SHERIFF"                                  , "None"                                  , "None"                   , True),
"SUPERVISING DOCKMASTER"                                              : ("DOCKMASTER"                                      , "None"                                  , "None"                   , True),
"SUPERVISING EMERGENCY MEDICAL SERVICE SPECIALIST"                    : ("EMERGENCY MEDICAL SERVICE SPECIALIST"            , "None"                                  , "None"                   , True),
"SUPERVISING EMERGENCY MEDICAL SERVICE SPECIALIST-DPTY CHIEFS"        : ("EMERGENCY MEDICAL SERVICE SPECIALIST"            , "DPTY CHIEFS"                           , "None"                   , True),
"SUPERVISING FIRE ALARM DISPATCHER"                                   : ("FIRE ALARM DISPATCHER"                           , "None"                                  , "None"                   , True),
"SUPERVISING FIRE MARSHAL"                                            : ("FIRE MARSHAL"                                    , "None"                                  , "None"                   , True),
"SUPERVISING FM-MGL DET: CHIEF OF STAFF TO FIRE COMM"                 : ("FIRE COMMISSIONER"                               , "CHIEF OF STAFF"                        , "SUPERVISING"            , True),
"SUPERVISING HOUSEKEEPER"                                             : ("HOUSEKEEPER"                                     , "None"                                  , "None"                   , True),
"SUPERVISING HOUSING GROUNDSKEEPER"                                   : ("HOUSING GROUNDSKEEPER"                           , "None"                                  , "None"                   , True),
"SUPERVISING HULL & MACHINERY INSPECTOR"                              : ("HULL & MACHINERY INSPECTOR"                      , "None"                                  , "None"                   , True),
"SUPERVISING HUMAN RIGHTS SPECIALIST"                                 : ("HUMAN RIGHTS SPECIALIST"                         , "None"                                  , "None"                   , True),
"SUPERVISING INSPECTOR GENERAL"                                       : ("INSPECTOR GENERAL"                               , "None"                                  , "None"                   , True),
"SUPERVISING POLICE COMMUNICATIONS TECHNICIAN"                        : ("POLICE COMMUNICATIONS TECHNICIAN"                , "None"                                  , "None"                   , True),
"SUPERVISING PROBATION OFFICER"                                       : ("PROBATION OFFICER"                               , "None"                                  , "None"                   , True),
"SUPERVISING PUBLIC HEALTH ADVISER"                                   : ("PUBLIC HEALTH ADVISER"                           , "None"                                  , "None"                   , True),
"SUPERVISING RACKETS INVESTIGATOR"                                    : ("RACKETS INVESTIGATOR"                            , "None"                                  , "None"                   , True),
"SUPERVISING SCHOOL AIDE"                                             : ("SCHOOL AIDE"                                     , "None"                                  , "None"                   , True),
"SUPERVISING SPECIAL OFFICER"                                         : ("SPECIAL OFFICER"                                 , "None"                                  , "None"                   , True),
"SUPERVISING THERAPIST"                                               : ("THERAPIST"                                       , "None"                                  , "None"                   , True),
"SUPERVISOR ASSIGNED"                                                 : ("SUPERVISOR"                                      , "None"                                  , "None"                   , True),
"SUPERVISOR BOILER MAKER"                                             : ("BOILER MAKER"                                    , "None"                                  , "None"                   , True),
"SUPERVISOR BRICKLAYER"                                               : ("BRICKLAYER"                                      , "None"                                  , "None"                   , True),
"SUPERVISOR BRIDGE PAINTER"                                           : ("BRIDGE PAINTER"                                  , "None"                                  , "None"                   , True),
"SUPERVISOR BRIDGE REPAIRER AND RIVETER"                              : ("BRIDGE REPAIRER AND RIVETER"                     , "None"                                  , "None"                   , True),
"SUPERVISOR CARPENTER"                                                : ("CARPENTER"                                       , "None"                                  , "None"                   , True),
"SUPERVISOR DOCKBUILDER"                                              : ("DOCKBUILDER"                                     , "None"                                  , "None"                   , True),
"SUPERVISOR ELECTRICIAN"                                              : ("ELECTRICIAN"                                     , "None"                                  , "None"                   , True),
"SUPERVISOR ELEVATOR MECHANIC"                                        : ("ELEVATOR MECHANIC"                               , "None"                                  , "None"                   , True),
"SUPERVISOR GLAZIER"                                                  : ("GLAZIER"                                         , "None"                                  , "None"                   , True),
"SUPERVISOR HIGHWAY REPAIRER"                                         : ("HIGHWAY REPAIRER"                                , "None"                                  , "None"                   , True),
"SUPERVISOR I SOCIAL WORK"                                            : ("SOCIAL WORKER"                                   , "I"                                     , "SUPERVISOR"             , True),
"SUPERVISOR I"                                                        : ("SUPERVISOR"                                      , "None"                                  , "I"                      , True),
"SUPERVISOR II SOCIAL WORK"                                           : ("SOCIAL WORKER"                                   , "None"                                  , "II"                     , True),
"SUPERVISOR II"                                                       : ("SUPERVISOR"                                      , "None"                                  , "II"                     , True),
"SUPERVISOR III SOCIAL WORK"                                          : ("SOCIAL WORKER"                                   , "None"                                  , "III"                    , True),
"SUPERVISOR III"                                                      : ("SUPERVISOR"                                      , "None"                                  , "III"                    , True),
"SUPERVISOR LOCKSMITH"                                                : ("LOCKSMITH"                                       , "None"                                  , "None"                   , True),
"SUPERVISOR OF BRIDGE OPERATIONS"                                     : ("BRIDGE OPERATIONS"                               , "None"                                  , "None"                   , True),
"SUPERVISOR OF CHILD CARE"                                            : ("CHILD CARE WORKER"                               , "None"                                  , "None"                   , True),
"SUPERVISOR OF ELECTRICAL INSTALLATIONS & MAINTENANCE"                : ("ELECTRICAL INSTALLATIONS & MAINTENANCE"          , "None"                                  , "None"                   , True),
"SUPERVISOR OF ELEVATOR MAINTENANCE"                                  : ("ELEVATOR MAINTENANCE"                            , "None"                                  , "None"                   , True),
"SUPERVISOR OF HOUSING CARETAKER"                                     : ("HOUSING CARETAKER"                               , "None"                                  , "None"                   , True),
"SUPERVISOR OF HOUSING STOCK WORKERS"                                 : ("HOUSING STOCK WORKERS"                           , "None"                                  , "None"                   , True),
"SUPERVISOR OF INVESTIGATORS"                                         : ("INVESTIGATOR"                                    , "None"                                  , "None"                   , True),
"SUPERVISOR OF IRONWORK"                                              : ("IRONWORKER"                                      , "OF"                                    , "None"                   , True),
"SUPERVISOR OF MECHANICAL INSTALLATIONS & MAINTENANCE"                : ("MECHANICAL INSTALLATIONS & MAINTENANCE"          , "SUPERVISOR OF"                         , "None"                   , True),
"SUPERVISOR OF MECHANICS"                                             : ("MECHANIC"                                        , "None"                                  , "None"                   , True),
"SUPERVISOR OF MOTOR TRANSPORT"                                       : ("MOTOR TRANSPORT"                                 , "None"                                  , "None"                   , True),
"SUPERVISOR OF NURSES"                                                : ("NURSE"                                           , "None"                                  , "None"                   , True),
"SUPERVISOR OF OFFICE MACHINE OPERATIONS"                             : ("OFFICE MACHINE OPERATIONS"                       , "None"                                  , "None"                   , True),
"SUPERVISOR OF RADIO AND TELEVISION OPERATORS"                        : ("RADIO AND TELEVISION OPERATOR"                   , "None"                                  , "None"                   , True),
"SUPERVISOR OF RADIO REPAIR OPERATIONS"                               : ("RADIO REPAIR OPERATIONS"                         , "None"                                  , "None"                   , True),
"SUPERVISOR OF SCHOOL SECURITY"                                       : ("SCHOOL SECURITY"                                 , "None"                                  , "None"                   , True),
"SUPERVISOR OF STOCK WORKERS"                                         : ("STOCK WORKER"                                    , "None"                                  , "None"                   , True),
"SUPERVISOR OF TRAFFIC DEVICE MAINTAINERS L2 & L3"                    : ("TRAFFIC DEVICE MAINTAINER"                       , "None"                                  , "None"                   , True),
"SUPERVISOR OF TRAFFIC DEVICE MAINTAINERS"                            : ("TRAFFIC DEVICE MAINTAINER"                       , "None"                                  , "None"                   , True),
"SUPERVISOR PAINTER"                                                  : ("PAINTER"                                         , "None"                                  , "None"                   , True),
"SUPERVISOR PER SESSION"                                              : ("SUPERVISOR"                                      , "PER SESSION"                           , "None"                   , True),
"SUPERVISOR PLASTERER"                                                : ("PLASTERER"                                       , "None"                                  , "None"                   , True),
"SUPERVISOR PLUMBER"                                                  : ("PLUMBER"                                         , "None"                                  , "None"                   , True),
"SUPERVISOR ROOFER"                                                   : ("ROOFER"                                          , "None"                                  , "None"                   , True),
"SUPERVISOR SHEET METAL WORKER"                                       : ("SHEET METAL WORKER"                              , "None"                                  , "None"                   , True),
"SUPERVISOR SHIP CARPENTER"                                           : ("SHIP CARPENTER"                                  , "None"                                  , "None"                   , True),
"SUPERVISOR STEAMFITTER"                                              : ("STEAMFITTER"                                     , "None"                                  , "None"                   , True),
"SUPERVISOR THERMOSTAT REPAIR"                                        : ("THERMOSTAT REPAIR"                               , "None"                                  , "None"                   , True),
"SUPERVISOR"                                                          : ("SUPERVISOR"                                      , "None"                                  , "None"                   , True),
"SUPV OF HOUSING EXTERMINATOR"                                        : ("HOUSING EXTERMINATOR"                            , "None"                                  , "None"                   , True),
"SUPV OF HOUSING EXTERMINATORS"                                       : ("HOUSING EXTERMINATOR"                            , "None"                                  , "None"                   , True),
"SUPVSNG SYSTEMS ANALYST"                                             : ("SYSTEMS ANALYST"                                 , "None"                                  , "None"                   , True),
"SURGEON DETAILED AS DEPUTY CHIEF SURGEON"                            : ("SURGEON"                                         , "DEPUTY CHIEF"                          , "None"                   , True),
"SURGEON"                                                             : ("SURGEON"                                         , "None"                                  , "None"                   , False),
"SURVEYOR"                                                            : ("SURVEYOR"                                        , "None"                                  , "None"                   , False),
"SYSTEMS ADMINISTRATOR-CAM FIN BD"                                    : ("SYSTEMS ADMINISTRATOR"                           , "None"                                  , "None"                   , False),
"SYSTEMS ANALYST"                                                     : ("SYSTEMS ANALYST"                                 , "None"                                  , "None"                   , False),
"TAX MAP CARTOGRAPHER"                                                : ("TAX MAP CARTOGRAPHER"                            , "None"                                  , "None"                   , False),
"TAXI AND LIMOUSINE INSPECTOR"                                        : ("TAXI AND LIMOUSINE INSPECTOR"                    , "None"                                  , "None"                   , False),
"TEACH ASST VOCATION - REG SUB"                                       : ("TEACHER"                                         , "VOCATION"                              , "None"                   , False),
"TEACHER AIDE"                                                        : ("TEACHER AIDE"                                    , "None"                                  , "None"                   , False),
"TEACHER ASSIGNED A"                                                  : ("TEACHER"                                         , "ASSIGNED"                              , "A"                      , False),
"TEACHER ASSIGNED B"                                                  : ("TEACHER"                                         , "ASSIGNED"                              , "B"                      , False),
"TEACHER ASSISTANT - REG SUB"                                         : ("TEACHER ASSISTANT"                               , "REG SUB"                               , "None"                   , False),
"TEACHER ATTENDANCE"                                                  : ("TEACHER"                                         , "None"                                  , "None"                   , False),
"TEACHER ATTENDANCE-REG SUB"                                          : ("TEACHER"                                         , "REG SUB"                               , "None"                   , False),
"TEACHER OF LIBRARY"                                                  : ("TEACHER"                                         , "OF LIBRARY"                            , "None"                   , False),
"TEACHER RESOURCE ROOM"                                               : ("TEACHER"                                         , "RESOURCE ROOM"                         , "None"                   , False),
"TEACHER SPECIAL EDUCATION"                                           : ("TEACHER"                                         , "SPECIAL EDUCATION"                     , "None"                   , False),
"TEACHER SPECIAL EDUCATION-REG SUB"                                   : ("TEACHER SPECIAL EDUCATION"                       , "REG SUB"                               , "None"                   , False),
"TEACHER TRAINER"                                                     : ("TEACHER"                                         , "TRAINER"                               , "None"                   , False),
"TEACHER"                                                             : ("TEACHER"                                         , "None"                                  , "None"                   , False),
"TEACHER- PER SESSION"                                                : ("TEACHER"                                         , "PER SESSION"                           , "None"                   , False),
"TEACHER-GENERAL ED"                                                  : ("TEACHER"                                         , "GENERAL ED"                            , "None"                   , False),
"TEACHER-REG SUB"                                                     : ("TEACHER"                                         , "REG SUB"                               , "None"                   , False),
"TEACHER-SPECIAL ED"                                                  : ("TEACHER"                                         , "SPECIAL ED"                            , "None"                   , False),
"TELECOMMUNICATION MANAGER"                                           : ("TELECOMMUNICATION MANAGER"                       , "None"                                  , "None"                   , True),
"TELECOMMUNICATIONS ASSOCIATE"                                        : ("TELECOMMUNICATIONS ASSOCIATE"                    , "None"                                  , "None"                   , False),
"TELECOMMUNICATIONS SPECIALIST"                                       : ("TELECOMMUNICATIONS SPECIALIST"                   , "None"                                  , "None"                   , False),
"TELEPHONE SERVICE TECHNICIAN"                                        : ("TELEPHONE SERVICE TECHNICIAN"                    , "None"                                  , "None"                   , False),
"TELEVISON EQUIPMENT OPERATOR"                                        : ("TELEVISON EQUIPMENT OPERATOR"                    , "None"                                  , "None"                   , False),
"TEMPORARY CLERK"                                                     : ("CLERK"                                           , "TEMPORARY"                             , "None"                   , False),
"TESTS AND MEASUREMENT SPECIALIST"                                    : ("TESTS AND MEASUREMENT SPECIALIST"                , "None"                                  , "None"                   , False),
"THERMOSTAT REPAIRER"                                                 : ("THERMOSTAT REPAIRER"                             , "None"                                  , "None"                   , False),
"THIRD ASSISTANT MARINE ENGINEER"                                     : ("MARINE ENGINEER"                                 , "ASSISTANT"                             , "THIRD"                  , False),
"TITLE EXAMINER"                                                      : ("TITLE EXAMINER"                                  , "None"                                  , "None"                   , False),
"TRACTOR OPERATOR"                                                    : ("TRACTOR OPERATOR"                                , "None"                                  , "None"                   , False),
"TRAFFIC CONTROL INSPECTOR"                                           : ("TRAFFIC CONTROL INSPECTOR"                       , "None"                                  , "None"                   , False),
"TRAFFIC DEVICE MAINTAINER"                                           : ("TRAFFIC DEVICE MAINTAINER"                       , "None"                                  , "None"                   , False),
"TRAFFIC ENFORC AGENT"                                                : ("TRAFFIC ENFORCEMENT AGENT"                       , "None"                                  , "None"                   , False),
"TRAFFIC ENFORCEMENT AGENT"                                           : ("TRAFFIC ENFORCEMENT AGENT"                       , "None"                                  , "None"                   , False),
"TRAINER ASSISTANT"                                                   : ("TRAINER"                                         , "ASSISTANT"                             , "None"                   , False),
"TRANSPORTATION BOROUGH COMMISSIONER"                                 : ("BOROUGH COMMISSIONER"                            , "TRANSPORTATION"                        , "None"                   , True),
"TRANSPORTATION INSPECTOR"                                            : ("TRANSPORTATION INSPECTOR"                        , "None"                                  , "None"                   , False),
"UFT SCHOOL SECRETARY ASSIGNED - NON SCHOOL BASED- 12 MONTH"          : ("SCHOOL SECRETARY"                                , "None"                                  , "None"                   , False),
"UNIVERSITY ARCHITECT"                                                : ("UNIVERSITY ARCHITECT"                            , "None"                                  , "None"                   , False),
"UNIVERSITY ASSISTANT ARCHITECT"                                      : ("ARCHITECT"                                       , "ASSISTANT"                             , "None"                   , False),
"UNIVERSITY ASSISTANT ENGINEER"                                       : ("ASSISTANT ENGINEER"                              , "UNIVERSITY"                            , "None"                   , False),
"UNIVERSITY ENGINEER TECH"                                            : ("ENGINEER TECH"                                   , "UNIVERSITY"                            , "None"                   , False),
"UNIVERSITY ENGINEER"                                                 : ("ENGINEER"                                        , "UNIVERSITY"                            , "None"                   , False),
"URBAN ARCHEOLOGIST"                                                  : ("URBAN ARCHEOLOGIST"                              , "None"                                  , "None"                   , False),
"URBAN PARK RANGER"                                                   : ("PARK RANGER"                                     , "URBAN"                                 , "None"                   , False),
"VICE PRESIDENT"                                                      : ("VICE PRESIDENT"                                  , "None"                                  , "None"                   , True),
"VOTING MACHINE TECHNICIAN"                                           : ("VOTING MACHINE TECHNICIAN"                       , "None"                                  , "None"                   , False),
"WARDEN"                                                              : ("WARDEN"                                          , "None"                                  , "None"                   , True),
"WARDEN-ASSISTANT DEPUTY WARDEN TED < 11/1/92"                        : ("WARDEN"                                          , "ASSISTANT DEPUTY"                      , "None"                   , True),
"WARDEN-ASSISTANT DEPUTY WARDEN"                                      : ("DEPUTY WARDEN"                                   , "ASSISTANT"                             , "None"                   , False),
"WARDEN-DEPUTY WARDEN IN COMM TED < 11/1/92"                          : ("WARDEN"                                          , "DEPUTY"                                , "None"                   , True),
"WARDEN-DEPUTY WARDEN IN COMM"                                        : ("WARDEN"                                          , "DEPUTY"                                , "None"                   , True),
"WARDEN-DEPUTY WARDEN TED < 11/1/92"                                  : ("WARDEN"                                          , "DEPUTY"                                , "None"                   , True),
"WARDEN-DEPUTY WARDEN"                                                : ("WARDEN"                                          , "DEPUTY"                                , "None"                   , True),
"WATCHPERSON"                                                         : ("WATCHPERSON"                                     , "None"                                  , "None"                   , False),
"WATER USE INSPECTOR"                                                 : ("WATER USE INSPECTOR"                             , "None"                                  , "None"                   , False),
"WATERSHED MAINTAINER"                                                : ("WATERSHED MAINTAINER"                            , "None"                                  , "None"                   , False),
"WELDER"                                                              : ("WELDER"                                          , "None"                                  , "None"                   , False),
"WIPER"                                                               : ("WIPER"                                           , "None"                                  , "None"                   , False),
"WORKER'S COMPENSATION BENEFITS EXAMINER"                             : ("WORKER'S COMPENSATION BENEFITS EXAMINER"         , "None"                                  , "None"                   , False),
"X-RAY TECHNICIAN"                                                    : ("X-RAY TECHNICIAN"                                , "None"                                  , "None"                   , False),
"YOUTH COORDINATOR"                                                   : ("YOUTH COORDINATOR"                               , "None"                                  , "None"                   , False),
}